In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from mlflow.data.pandas_dataset import PandasDataset
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from dotenv import load_dotenv

print(" Environment Variables Loaded:")
print(f"AWS_ACCESS_KEY_ID: {'***' + os.getenv('MINIO_ACCESS_KEY', 'NOT SET')[-4:] if os.getenv('MINIO_ACCESS_KEY') else 'NOT SET'}")
print(f"AWS_SECRET_ACCESS_KEY: {'***' + os.getenv('MINIO_SECRET_ACCESS_KEY', 'NOT SET')[-4:] if os.getenv('MINIO_SECRET_ACCESS_KEY') else 'NOT SET'}")
print(f"AWS_DEFAULT_REGION: {os.getenv('AWS_DEFAULT_REGION', 'NOT SET')}")

file_path = 'C:/Users/ldmag/Documents/GitHub/Code-Assignments-Projects/Projects/MLOps Drift Detection and Pipeline Optimization/data/Telco-Churn.csv'
BASE = pd.read_csv(file_path)

 Environment Variables Loaded:
AWS_ACCESS_KEY_ID: ***dmin
AWS_SECRET_ACCESS_KEY: ***dmin
AWS_DEFAULT_REGION: NOT SET


## Train a baseline model

In [3]:
# These functions are for training a relatively robust random forest model; no adversarial injection

def load_and_preprocess_data(df_filepath):
    df = pd.read_csv(df_filepath)

    dataset: PandasDataset = mlflow.data.from_pandas(df)

    print('Loaded Telco data to dataframe')

    numeric = []
    categorical = []
    numeric_features = ['tenure', 'MonthlyCharges', 'TotalCharges']
    categorical_features = [
            'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
            'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
            'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
            'PaperlessBilling', 'PaymentMethod', 'SeniorCitizen'
        ]

    df.drop(columns=['customerID'])

    from sklearn.impute import SimpleImputer

    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

    imputer = SimpleImputer(strategy='median')
    df[numeric_features] = imputer.fit_transform(df[numeric_features])

    if 'MonthlyCharges' in df.columns and 'TotalCharges' in df.columns:
        df['monthly_total_ratio'] = df['MonthlyCharges'] / (df['TotalCharges'] + 1)
        numeric.append('monthly_total_ratio')
        print("Added monthly_total_ratio")
    
    if 'TotalCharges' in df.columns and 'tenure' in df.columns:
        df['charge_per_month'] = df['TotalCharges'] / (df['tenure'] + 1)
        numeric.append('charge_per_month')
        print("Added charge_per_month")
    
    # Service engagement score (aggregated feature)
    service_cols = ['PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                   'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
    available_services = [col for col in service_cols if col in df.columns]
    
    if available_services:
        service_count = sum((df[col] == 'Yes').astype(int) for col in available_services)
        df['service_engagement'] = service_count
        numeric.append('service_engagement')
        print(f"Added service_engagement from {len(available_services)} services")
    
    # Binned features (less sensitive to outliers)
    if 'tenure' in df.columns:
        df['tenure_tier'] = pd.qcut(df['tenure'], 
                                         q=4, labels=['New', 'Short', 'Medium', 'Long'], 
                                         duplicates='drop').astype(str)
        categorical.append('tenure_tier')
        print("Added tenure_tier")
    
    if 'MonthlyCharges' in df.columns:
        df['value_tier'] = pd.qcut(df['MonthlyCharges'], 
                                        q=3, labels=['Budget', 'Standard', 'Premium'], 
                                        duplicates='drop').astype(str)
        categorical.append('value_tier')
        print("Added value_tier")
    
    # Composite stability score
    stability_score = np.zeros(len(df))
    if 'Contract' in df.columns:
        stability_score += (df['Contract'] == 'Two year').astype(int) * 2
        stability_score += (df['Contract'] == 'One year').astype(int) * 1
    
    if 'PaymentMethod' in df.columns:
        auto_pay = df['PaymentMethod'].str.contains('automatic', case=False, na=False)
        stability_score += auto_pay.astype(int)
    
    df['stability_score'] = stability_score
    numeric.append('stability_score')
    print("Added stability_score")
    
    print(f"Added {len(numeric)} numeric and {len(categorical)} categorical features")

    target = 'Churn'
    y = df[target].apply(lambda x: 1 if x == 'Yes' else 0)
    X = df.drop(columns=[target])

    numeric_features = [f for f in numeric_features if f in X.columns]
    categorical_features = [f for f in categorical_features if f in X.columns]

    categorical_columns = categorical + categorical_features
    numeric_columns = numeric + numeric_features

    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    numerical_transformer = RobustScaler()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numeric_columns),
            ('cat', categorical_transformer, categorical_columns)
        ]
    )
    return X, y, preprocessor

def train_randomforest_baseline(X, y, preprocessor, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

    dataset: PandasDataset = mlflow.data.from_pandas(X_train)

    classifier = RandomForestClassifier(
        n_estimators=150,
        max_depth=12,
        min_samples_split=5, 
        min_samples_leaf=3,  
        random_state=42, 
        class_weight='balanced')
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])

    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("telco-baseline")

    with mlflow.start_run(run_name='trainRandomForest'):
        mlflow.log_param('n_estimators', 150)
        mlflow.log_param('max_depth', 12)
        mlflow.log_param('class_weight', 'balanced')
        mlflow.log_param('is_drift', False)
        mlflow.log_param('train_size', len(X_train))
        mlflow.log_param('test_size', len(X_test))

        pipeline.fit(X_train, y_train)

        from mlflow.models.signature import infer_signature
        signature = infer_signature(X_train, y_train)
        
        mlflow.sklearn.log_model(
            pipeline, 
            'RandomForest',
            signature=signature, 
            registered_model_name='telco-baseline'
        )

        mlflow.log_input(dataset, context='training')

        y_pred = pipeline.predict(X_test)
        y_prob = pipeline.predict_proba(X_test)[:,1]

        test_accuracy = accuracy_score(y_test, y_pred)
        test_auc = roc_auc_score(y_test, y_prob)
        test_f1 = f1_score(y_test, y_pred)

        mlflow.log_metric('test_accuracy', test_accuracy)
        mlflow.log_metric('test_auc', test_auc)
        mlflow.log_metric('test_f1', test_f1)

    return pipeline

In [37]:
X, y, preprocessor = load_and_preprocess_data(file_path)
pipeline = train_randomforest_baseline(X, y, preprocessor, random_state=42)

Loaded Telco data to dataframe
Added monthly_total_ratio
Added charge_per_month
Added service_engagement from 8 services
Added tenure_tier
Added value_tier
Added stability_score
Added 4 numeric and 2 categorical features


2025/11/03 21:52:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'telco-baseline' already exists. Creating a new version of this model...
2025/11/03 21:52:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: telco-baseline, version 3
Created version '3' of model 'telco-baseline'.


🏃 View run trainRandomForest at: http://localhost:5000/#/experiments/1/runs/e16514ce8f0d4781b331e562b9185c04
🧪 View experiment at: http://localhost:5000/#/experiments/1


## Introducing drift

In [5]:
def _apply_numeric_covariate_drift(X, drift_threshold, numeric_cols, drift_info):
    """Apply covariate drift to numeric features."""
    for i, col in enumerate(numeric_cols):
        if col not in X.columns:
            continue
            
        col_mean = X[col].mean()
        col_std = X[col].std()
        
        if pd.isna(col_mean) or pd.isna(col_std) or col_std == 0:
            continue
        
        drift_type = i % 4
        
        if drift_type == 0:  # Mean shift
            shift_amount = drift_threshold * col_mean * 0.3
            X[col] = X[col] + shift_amount
            drift_info['covariate_shifts'].append({
                'feature': col, 'type': 'mean_shift', 'amount': shift_amount
            })
        elif drift_type == 1:  # Variance increase
            noise = np.random.normal(0, drift_threshold * col_std * 0.5, len(X))
            X[col] = X[col] + noise
            drift_info['covariate_shifts'].append({
                'feature': col, 'type': 'variance_increase', 'noise_std': drift_threshold * col_std * 0.5
            })
        elif drift_type == 2:  # Multiplicative shift
            scale_factor = 1 + drift_threshold * 0.2 * np.random.choice([-1, 1])
            X[col] = X[col] * scale_factor
            drift_info['covariate_shifts'].append({
                'feature': col, 'type': 'multiplicative_shift', 'factor': scale_factor
            })
        else:  # Add outliers
            outlier_fraction = 0.1 * drift_threshold
            n_outliers = int(outlier_fraction * len(X))
            if n_outliers > 0:
                outlier_indices = np.random.choice(X.index, n_outliers, replace=False)
                outlier_multiplier = 3 + 2 * drift_threshold
                X.loc[outlier_indices, col] = X.loc[outlier_indices, col] * outlier_multiplier
                drift_info['covariate_shifts'].append({
                    'feature': col, 'type': 'outliers', 'n_outliers': n_outliers
                })
    
    # Special handling for Telco features
    if 'tenure' in X.columns:
        tenure_increase = drift_threshold * 5
        X['tenure'] = X['tenure'] + np.random.normal(tenure_increase, 2, len(X))
        X['tenure'] = X['tenure'].clip(lower=0)
        drift_info['covariate_shifts'].append({
            'feature': 'tenure', 'type': 'market_shift', 'increase_months': tenure_increase
        })
    
    if 'MonthlyCharges' in X.columns:
        inflation_rate = 1 + drift_threshold * 0.15
        X['MonthlyCharges'] = X['MonthlyCharges'] * inflation_rate
        drift_info['covariate_shifts'].append({
            'feature': 'MonthlyCharges', 'type': 'inflation', 'rate': inflation_rate
        })
    
    if 'TotalCharges' in X.columns and 'tenure' in X.columns and 'MonthlyCharges' in X.columns:
        X['TotalCharges'] = X['tenure'] * X['MonthlyCharges'] * \
                           (1 + np.random.normal(0, 0.1 * drift_threshold, len(X)))
        X['TotalCharges'] = X['TotalCharges'].clip(lower=0)
    
    return X


def _apply_categorical_covariate_drift(X, drift_threshold, categorical_cols, drift_info):
    """Apply covariate drift to categorical features."""
    for col in categorical_cols[:min(5, len(categorical_cols))]:
        if col not in X.columns:
            continue
        
        unique_vals = X[col].unique()
        if len(unique_vals) < 2:
            continue
        
        if col == 'InternetService' and 'Fiber optic' in unique_vals and 'DSL' in unique_vals:
            mask_fiber = X[col] == 'DSL'
            n_to_shift = int(len(X) * 0.2 * drift_threshold)
            if mask_fiber.sum() > 0:
                shift_indices = np.random.choice(
                    X[mask_fiber].index[:n_to_shift], 
                    size=min(n_to_shift, mask_fiber.sum()), 
                    replace=False
                )
                X.loc[shift_indices, col] = 'Fiber optic'
                drift_info['covariate_shifts'].append({
                    'feature': col,
                    'type': 'category_probability_shift',
                    'shift': f'DSL -> Fiber optic ({len(shift_indices)} samples)'
                })
        elif len(unique_vals) >= 2:
            value_counts = X[col].value_counts()
            if len(value_counts) >= 2:
                most_common = value_counts.index[0]
                least_common = value_counts.index[-1]
                
                n_to_shift = int(len(X) * 0.15 * drift_threshold)
                mask = X[col] == most_common
                if mask.sum() > 0:
                    shift_indices = np.random.choice(
                        X[mask].index, 
                        size=min(n_to_shift, mask.sum()), 
                        replace=False
                    )
                    X.loc[shift_indices, col] = least_common
                    drift_info['covariate_shifts'].append({
                        'feature': col,
                        'type': 'category_distribution_shift',
                        'shift': f'{most_common} -> {least_common} ({len(shift_indices)} samples)'
                    })
    
    return X


def _apply_concept_drift(X, y, drift_threshold, drift_info):
    """Apply concept drift to target labels."""
    # 1. High-value customer retention
    if 'MonthlyCharges' in X.columns:
        high_charge_threshold = X['MonthlyCharges'].quantile(0.75)
        high_charge_mask = X['MonthlyCharges'] > high_charge_threshold
        n_to_flip = int(high_charge_mask.sum() * 0.3 * drift_threshold)
        if n_to_flip > 0:
            flip_indices = np.random.choice(
                X[high_charge_mask].index, 
                size=min(n_to_flip, high_charge_mask.sum()), 
                replace=False
            )
            y.loc[flip_indices] = 1 - y.loc[flip_indices]
            drift_info['concept_shifts'].append({
                'type': 'high_value_retention',
                'description': 'High MonthlyCharges customers now less likely to churn',
                'n_samples': len(flip_indices)
            })
    
    # 2. Tenure fatigue
    if 'tenure' in X.columns:
        long_tenure_threshold = X['tenure'].quantile(0.8)
        long_tenure_mask = (X['tenure'] > long_tenure_threshold) & (y == 0)
        n_to_flip = int(long_tenure_mask.sum() * 0.2 * drift_threshold)
        if n_to_flip > 0:
            flip_indices = np.random.choice(
                X[long_tenure_mask].index, 
                size=min(n_to_flip, long_tenure_mask.sum()), 
                replace=False
            )
            y.loc[flip_indices] = 1
            drift_info['concept_shifts'].append({
                'type': 'tenure_fatigue',
                'description': 'Very long tenure customers more likely to churn',
                'n_samples': len(flip_indices)
            })
    
    # 3. Service overwhelm
    if 'service_engagement' in X.columns:
        high_engagement_threshold = X['service_engagement'].quantile(0.7)
        high_engagement_mask = (X['service_engagement'] > high_engagement_threshold) & (y == 0)
        n_to_flip = int(high_engagement_mask.sum() * 0.25 * drift_threshold)
        if n_to_flip > 0:
            flip_indices = np.random.choice(
                X[high_engagement_mask].index, 
                size=min(n_to_flip, high_engagement_mask.sum()), 
                replace=False
            )
            y.loc[flip_indices] = 1
            drift_info['concept_shifts'].append({
                'type': 'service_overwhelm',
                'description': 'High service engagement customers more likely to churn',
                'n_samples': len(flip_indices)
            })
    
    # 4. Contract regret
    if 'Contract' in X.columns:
        two_year_mask = (X['Contract'] == 'Two year') & (y == 0)
        n_to_flip = int(two_year_mask.sum() * 0.15 * drift_threshold)
        if n_to_flip > 0:
            flip_indices = np.random.choice(
                X[two_year_mask].index, 
                size=min(n_to_flip, two_year_mask.sum()), 
                replace=False
            )
            y.loc[flip_indices] = 1
            drift_info['concept_shifts'].append({
                'type': 'contract_regret',
                'description': 'Two year contract customers more likely to churn',
                'n_samples': len(flip_indices)
            })
    
    # 5. Base rate shift
    base_rate_shift = drift_threshold * 0.1
    if base_rate_shift > 0:
        current_churn_rate = y.mean()
        target_churn_rate = min(1.0, current_churn_rate + base_rate_shift)
        
        n_current_churn = y.sum()
        n_target_churn = int(len(y) * target_churn_rate)
        n_to_change = abs(n_target_churn - n_current_churn)
        
        if n_target_churn > n_current_churn and n_to_change > 0:
            non_churners = X[y == 0].index
            if len(non_churners) > 0:
                flip_indices = np.random.choice(
                    non_churners, 
                    size=min(n_to_change, len(non_churners)), 
                    replace=False
                )
                y.loc[flip_indices] = 1
        elif n_to_change > 0:
            churners = X[y == 1].index
            if len(churners) > 0:
                flip_indices = np.random.choice(
                    churners, 
                    size=min(n_to_change, len(churners)), 
                    replace=False
                )
                y.loc[flip_indices] = 0
        
        drift_info['concept_shifts'].append({
            'type': 'base_rate_shift',
            'description': f'Overall churn rate shifted from {current_churn_rate:.3f} to {target_churn_rate:.3f}',
            'shift_amount': base_rate_shift
        })
    
    return y


def simulate_drift(X, y, drift_threshold=0.5, drift_type='combined', 
                   covariate_weight=1.0, concept_weight=1.0, random_state=42):
    """
    Unified drift simulation function supporting all drift types.
    
    Parameters:
    -----------
    X : pd.DataFrame
        Original feature dataframe
    y : pd.Series
        Original target labels
    drift_threshold : float
        Controls overall drift intensity (0.0 to 1.0)
    drift_type : str
        Type of drift: 'combined', 'covariate', or 'concept'
    covariate_weight : float
        Weight for covariate drift component (0.0 to 1.0)
    concept_weight : float
        Weight for concept drift component (0.0 to 1.0)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    X_drifted : pd.DataFrame
        Drifted feature dataframe
    y_drifted : pd.Series
        Drifted target labels
    drift_info : dict
        Information about applied drifts
    """
    np.random.seed(random_state)
    X_drifted = X.copy()
    y_drifted = y.copy()
    
    drift_info = {
        'covariate_shifts': [],
        'concept_shifts': [],
        'threshold': drift_threshold,
        'drift_type': drift_type,
        'covariate_weight': covariate_weight,
        'concept_weight': concept_weight
    }
    
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Simulating {drift_type} drift with threshold: {drift_threshold:.2f}")
    if drift_type == 'combined':
        print(f"Covariate weight: {covariate_weight:.2f}, Concept weight: {concept_weight:.2f}")
    print(f"Applying to {len(numeric_cols)} numeric and {len(categorical_cols)} categorical features")
    
    # Apply covariate drift
    if drift_type in ['combined', 'covariate'] and covariate_weight > 0:
        effective_threshold = drift_threshold * covariate_weight
        X_drifted = _apply_numeric_covariate_drift(X_drifted, effective_threshold, numeric_cols, drift_info)
        X_drifted = _apply_categorical_covariate_drift(X_drifted, effective_threshold, categorical_cols, drift_info)
    
    # Apply concept drift
    if drift_type in ['combined', 'concept'] and concept_weight > 0:
        effective_threshold = drift_threshold * concept_weight
        y_drifted = _apply_concept_drift(X_drifted, y_drifted, effective_threshold, drift_info)
    
    print(f"Applied {len(drift_info['covariate_shifts'])} covariate shifts")
    print(f"Applied {len(drift_info['concept_shifts'])} concept shifts")
    print(f"Final churn rate: {y_drifted.mean():.3f} (original: {y.mean():.3f})")
    
    return X_drifted, y_drifted, drift_info

def create_drift_visualizations(X_original, y_original, X_drifted, y_drifted, 
                                metrics_original, metrics_drifted, drift_threshold, 
                                save_dir='drift_plots'):
    import os
    from sklearn.metrics import roc_curve
    
    os.makedirs(save_dir, exist_ok=True)
    plot_paths = []
    
    # 1. ROC Curve Comparison
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Get predictions for ROC curves (assuming they're passed or calculated)
    # For now, we'll create a placeholder - in practice, predictions should be passed
    try:
        if 'y_prob_original' in metrics_original and 'y_prob_drifted' in metrics_drifted:
            fpr_orig, tpr_orig, _ = roc_curve(y_original, metrics_original['y_prob_original'])
            fpr_drift, tpr_drift, _ = roc_curve(y_drifted, metrics_drifted['y_prob_drifted'])
            
            ax.plot(fpr_orig, tpr_orig, label=f'Original (AUC={metrics_original["auc"]:.3f})', linewidth=2)
            ax.plot(fpr_drift, tpr_drift, label=f'Drifted (AUC={metrics_drifted["auc"]:.3f})', linewidth=2)
            ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
            ax.set_xlabel('False Positive Rate', fontsize=12)
            ax.set_ylabel('True Positive Rate', fontsize=12)
            ax.set_title(f'ROC Curve Comparison (Drift Threshold: {drift_threshold})', fontsize=14, fontweight='bold')
            ax.legend(loc='lower right', fontsize=10)
            ax.grid(alpha=0.3)
    except:
        pass  # Skip if predictions not available
    
    plt.tight_layout()
    roc_path = os.path.join(save_dir, 'roc_curve_comparison.png')
    plt.savefig(roc_path, dpi=300, bbox_inches='tight')
    plt.close()
    plot_paths.append(roc_path)
    
    # 2. Metric Degradation Bar Chart
    fig, ax = plt.subplots(figsize=(10, 6))
    
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    original_vals = [metrics_original.get(m, 0) for m in metrics_to_plot]
    drifted_vals = [metrics_drifted.get(m, 0) for m in metrics_to_plot]
    degradations = [orig - drift for orig, drift in zip(original_vals, drifted_vals)]
    
    x = np.arange(len(metrics_to_plot))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, original_vals, width, label='Original', alpha=0.8, color='#2ecc71')
    bars2 = ax.bar(x + width/2, drifted_vals, width, label='Drifted', alpha=0.8, color='#e74c3c')
    
    # Add degradation percentages on bars
    for i, (orig, drift, deg) in enumerate(zip(original_vals, drifted_vals, degradations)):
        if orig > 0:
            pct = (deg / orig) * 100
            ax.text(i, max(orig, drift) + 0.02, f'{pct:.1f}%', 
                   ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    ax.set_xlabel('Metrics', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title(f'Model Performance Degradation (Drift Threshold: {drift_threshold})', 
                fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_to_plot, fontsize=11)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, 1.1])
    
    plt.tight_layout()
    metric_path = os.path.join(save_dir, 'metric_degradation.png')
    plt.savefig(metric_path, dpi=300, bbox_inches='tight')
    plt.close()
    plot_paths.append(metric_path)
    
    # 3. Feature Distribution Comparison (for key numeric features)
    numeric_cols = X_original.select_dtypes(include=[np.number]).columns[:6]  # Top 6 numeric features
    
    if len(numeric_cols) > 0:
        n_cols = min(3, len(numeric_cols))
        n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if len(numeric_cols) > 1 else [axes]
        
        for idx, col in enumerate(numeric_cols):
            if idx >= len(axes):
                break
            ax = axes[idx]
            
            ax.hist(X_original[col].dropna(), bins=30, alpha=0.6, label='Original', 
                   color='#2ecc71', density=True)
            ax.hist(X_drifted[col].dropna(), bins=30, alpha=0.6, label='Drifted', 
                   color='#e74c3c', density=True)
            ax.set_xlabel(col, fontsize=10)
            ax.set_ylabel('Density', fontsize=10)
            ax.set_title(f'{col} Distribution', fontsize=11, fontweight='bold')
            ax.legend(fontsize=9)
            ax.grid(alpha=0.3)
        
        # Hide unused subplots
        for idx in range(len(numeric_cols), len(axes)):
            axes[idx].axis('off')
        
        plt.suptitle(f'Feature Distribution Drift (Drift Threshold: {drift_threshold})', 
                    fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        dist_path = os.path.join(save_dir, 'feature_distributions.png')
        plt.savefig(dist_path, dpi=300, bbox_inches='tight')
        plt.close()
        plot_paths.append(dist_path)
    
    # 4. Churn Rate Comparison
    fig, ax = plt.subplots(figsize=(8, 6))
    
    churn_original = y_original.mean()
    churn_drifted = y_drifted.mean()
    
    categories = ['Original', 'Drifted']
    churn_rates = [churn_original, churn_drifted]
    colors = ['#2ecc71', '#e74c3c']
    
    bars = ax.bar(categories, churn_rates, color=colors, alpha=0.8, width=0.6)
    ax.set_ylabel('Churn Rate', fontsize=12)
    ax.set_title(f'Churn Rate Shift (Drift Threshold: {drift_threshold})', 
                fontsize=14, fontweight='bold')
    ax.set_ylim([0, max(churn_rates) * 1.2])
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, rate in zip(bars, churn_rates):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{rate:.3f}',
               ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Add change annotation
    change = churn_drifted - churn_original
    change_pct = (change / churn_original) * 100 if churn_original > 0 else 0
    ax.annotate(f'Change: {change:+.3f} ({change_pct:+.1f}%)',
               xy=(1, churn_drifted), xytext=(1.3, churn_drifted + 0.05),
               arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
               fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    churn_path = os.path.join(save_dir, 'churn_rate_shift.png')
    plt.savefig(churn_path, dpi=300, bbox_inches='tight')
    plt.close()
    plot_paths.append(churn_path)
    
    return plot_paths

# Backward-compatible wrapper functions
def simulate_drifted_data(X, y, drift_threshold=0.5, random_state=42):
    """Combined drift (original function signature maintained)."""
    return simulate_drift(X, y, drift_threshold=drift_threshold, 
                         drift_type='combined', random_state=random_state)


def simulate_covariate_drift_only(X, y, drift_threshold=0.5, random_state=42):
    """Covariate drift only (original function signature maintained)."""
    return simulate_drift(X, y, drift_threshold=drift_threshold, 
                         drift_type='covariate', random_state=random_state)


def simulate_concept_drift_only(X, y, drift_threshold=0.5, random_state=42):
    """Concept drift only (original function signature maintained)."""
    return simulate_drift(X, y, drift_threshold=drift_threshold, 
                         drift_type='concept', random_state=random_state)


def simulate_selective_drift(X, y, drift_threshold=0.5, 
                            covariate_ratio=0.75, concept_ratio=0.25, 
                            random_state=42):
    """Selective drift with custom ratios (original function signature maintained)."""
    return simulate_drift(X, y, drift_threshold=drift_threshold, 
                         drift_type='combined',
                         covariate_weight=covariate_ratio,
                         concept_weight=concept_ratio,
                         random_state=random_state)

In [6]:
# Cell is now deprecated

'''
def simulate_drifted_data(X, y, drift_threshold=0.5, random_state=42):
    """
    Simulate drifted data with both covariate shift and concept shift.
    
    Parameters:
    -----------
    X : pd.DataFrame
        Original feature dataframe (before preprocessing)
    y : pd.Series
        Original target labels (0/1)
    drift_threshold : float
        Controls the intensity of drift (0.0 to 1.0)
        - 0.0: No drift
        - 0.5: Moderate drift
        - 1.0: Severe drift
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    X_drifted : pd.DataFrame
        Drifted feature dataframe
    y_drifted : pd.Series
        Drifted target labels (after concept shift)
    drift_info : dict
        Information about what drift was applied
    """
    np.random.seed(random_state)
    X_drifted = X.copy()
    y_drifted = y.copy()
    
    drift_info = {
        'covariate_shifts': [],
        'concept_shifts': [],
        'threshold': drift_threshold
    }
    
    # Identify numeric and categorical columns
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f"Simulating drift with threshold: {drift_threshold:.2f}")
    print(f"Applying to {len(numeric_cols)} numeric and {len(categorical_cols)} categorical features")
    
    # ============================================
    # COVARIATE SHIFT: Changes to feature distributions
    # ============================================
    
    # 1. Numeric Feature Drifts
    for i, col in enumerate(numeric_cols):
        if col not in X_drifted.columns:
            continue
            
        col_mean = X_drifted[col].mean()
        col_std = X_drifted[col].std()
        
        if pd.isna(col_mean) or pd.isna(col_std) or col_std == 0:
            continue
        
        # Apply different types of drift to different features
        drift_type = i % 4
        
        if drift_type == 0:  # Mean shift (increase/decrease)
            shift_amount = drift_threshold * col_mean * 0.3  # Up to 30% of mean
            X_drifted[col] = X_drifted[col] + shift_amount
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'mean_shift',
                'amount': shift_amount
            })
            
        elif drift_type == 1:  # Variance increase
            noise = np.random.normal(0, drift_threshold * col_std * 0.5, len(X_drifted))
            X_drifted[col] = X_drifted[col] + noise
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'variance_increase',
                'noise_std': drift_threshold * col_std * 0.5
            })
            
        elif drift_type == 2:  # Multiplicative shift (scaling)
            scale_factor = 1 + drift_threshold * 0.2 * np.random.choice([-1, 1])
            X_drifted[col] = X_drifted[col] * scale_factor
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'multiplicative_shift',
                'factor': scale_factor
            })
            
        else:  # Add outliers
            outlier_fraction = 0.1 * drift_threshold  # Up to 10% outliers
            n_outliers = int(outlier_fraction * len(X_drifted))
            outlier_indices = np.random.choice(X_drifted.index, n_outliers, replace=False)
            # Make outliers 3-5x the original value
            outlier_multiplier = 3 + 2 * drift_threshold
            X_drifted.loc[outlier_indices, col] = X_drifted.loc[outlier_indices, col] * outlier_multiplier
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'outliers',
                'n_outliers': n_outliers
            })
    
    # Special handling for key Telco features
    if 'tenure' in X_drifted.columns:
        # Simulate customers staying longer (market shift)
        tenure_increase = drift_threshold * 5  # Up to 5 months increase
        X_drifted['tenure'] = X_drifted['tenure'] + np.random.normal(tenure_increase, 2, len(X_drifted))
        X_drifted['tenure'] = X_drifted['tenure'].clip(lower=0)  # Ensure non-negative
        drift_info['covariate_shifts'].append({
            'feature': 'tenure',
            'type': 'market_shift',
            'increase_months': tenure_increase
        })
    
    if 'MonthlyCharges' in X_drifted.columns:
        # Simulate price inflation
        inflation_rate = 1 + drift_threshold * 0.15  # Up to 15% increase
        X_drifted['MonthlyCharges'] = X_drifted['MonthlyCharges'] * inflation_rate
        drift_info['covariate_shifts'].append({
            'feature': 'MonthlyCharges',
            'type': 'inflation',
            'rate': inflation_rate
        })
    
    if 'TotalCharges' in X_drifted.columns:
        # Recalculate TotalCharges based on drifted tenure and MonthlyCharges if both exist
        if 'tenure' in X_drifted.columns and 'MonthlyCharges' in X_drifted.columns:
            # TotalCharges should roughly be tenure * MonthlyCharges (with some variation)
            X_drifted['TotalCharges'] = X_drifted['tenure'] * X_drifted['MonthlyCharges'] * \
                                       (1 + np.random.normal(0, 0.1 * drift_threshold, len(X_drifted)))
            X_drifted['TotalCharges'] = X_drifted['TotalCharges'].clip(lower=0)
    
    # 2. Categorical Feature Drifts
    for col in categorical_cols[:min(5, len(categorical_cols))]:  # Limit to avoid too many changes
        if col not in X_drifted.columns:
            continue
        
        unique_vals = X_drifted[col].unique()
        if len(unique_vals) < 2:
            continue
        
        # Shift probability distribution towards different categories
        # Example: More customers choosing 'Fiber optic' over 'DSL'
        if col == 'InternetService' and 'Fiber optic' in unique_vals and 'DSL' in unique_vals:
            mask_fiber = X_drifted[col] == 'DSL'
            n_to_shift = int(len(X_drifted) * 0.2 * drift_threshold)
            shift_indices = np.random.choice(X_drifted[mask_fiber].index[:n_to_shift], 
                                            size=min(n_to_shift, mask_fiber.sum()), 
                                            replace=False)
            X_drifted.loc[shift_indices, col] = 'Fiber optic'
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'category_probability_shift',
                'shift': f'DSL -> Fiber optic ({len(shift_indices)} samples)'
            })
        
        # General categorical shift: change distribution
        elif len(unique_vals) >= 2:
            # Shift some samples from most common to least common category
            value_counts = X_drifted[col].value_counts()
            if len(value_counts) >= 2:
                most_common = value_counts.index[0]
                least_common = value_counts.index[-1]
                
                n_to_shift = int(len(X_drifted) * 0.15 * drift_threshold)
                mask = X_drifted[col] == most_common
                if mask.sum() > 0:
                    shift_indices = np.random.choice(X_drifted[mask].index, 
                                                    size=min(n_to_shift, mask.sum()), 
                                                    replace=False)
                    X_drifted.loc[shift_indices, col] = least_common
                    drift_info['covariate_shifts'].append({
                        'feature': col,
                        'type': 'category_distribution_shift',
                        'shift': f'{most_common} -> {least_common} ({len(shift_indices)} samples)'
                    })
    
    # ============================================
    # CONCEPT SHIFT: Changes to label relationships
    # ============================================
    
    print("Applying concept shift...")
    
    # 1. Reverse relationship for high-value customers
    # Original: Higher charges -> more likely to churn
    # Drifted: Higher charges -> less likely to churn (premium retention)
    if 'MonthlyCharges' in X_drifted.columns:
        high_charge_threshold = X_drifted['MonthlyCharges'].quantile(0.75)
        high_charge_mask = X_drifted['MonthlyCharges'] > high_charge_threshold
        
        # Reverse churn probability for high-charge customers
        n_to_flip = int(high_charge_mask.sum() * 0.3 * drift_threshold)
        flip_indices = np.random.choice(X_drifted[high_charge_mask].index, 
                                       size=min(n_to_flip, high_charge_mask.sum()), 
                                       replace=False)
        y_drifted.loc[flip_indices] = 1 - y_drifted.loc[flip_indices]  # Flip labels
        drift_info['concept_shifts'].append({
            'type': 'high_value_retention',
            'description': 'High MonthlyCharges customers now less likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 2. Change relationship with tenure
    # Original: Longer tenure -> less likely to churn
    # Drifted: Very long tenure customers may become more likely to churn (market fatigue)
    if 'tenure' in X_drifted.columns:
        long_tenure_threshold = X_drifted['tenure'].quantile(0.8)
        long_tenure_mask = (X_drifted['tenure'] > long_tenure_threshold) & (y_drifted == 0)
        
        n_to_flip = int(long_tenure_mask.sum() * 0.2 * drift_threshold)
        flip_indices = np.random.choice(X_drifted[long_tenure_mask].index, 
                                       size=min(n_to_flip, long_tenure_mask.sum()), 
                                       replace=False)
        y_drifted.loc[flip_indices] = 1  # Flip to churn
        drift_info['concept_shifts'].append({
            'type': 'tenure_fatigue',
            'description': 'Very long tenure customers more likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 3. Change relationship with service engagement
    # Original: More services -> less likely to churn
    # Drifted: More services -> more likely to churn (complexity/overwhelm)
    if 'service_engagement' in X_drifted.columns:
        high_engagement_threshold = X_drifted['service_engagement'].quantile(0.7)
        high_engagement_mask = (X_drifted['service_engagement'] > high_engagement_threshold) & (y_drifted == 0)
        
        n_to_flip = int(high_engagement_mask.sum() * 0.25 * drift_threshold)
        flip_indices = np.random.choice(X_drifted[high_engagement_mask].index, 
                                       size=min(n_to_flip, high_engagement_mask.sum()), 
                                       replace=False)
        y_drifted.loc[flip_indices] = 1  # Flip to churn
        drift_info['concept_shifts'].append({
            'type': 'service_overwhelm',
            'description': 'High service engagement customers more likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 4. Contract type relationship change
    # Original: Longer contracts -> less churn
    # Drifted: Some contract types become less effective
    if 'Contract' in X_drifted.columns:
        # Make "Two year" contract customers more likely to churn (regret/commitment issues)
        two_year_mask = (X_drifted['Contract'] == 'Two year') & (y_drifted == 0)
        n_to_flip = int(two_year_mask.sum() * 0.15 * drift_threshold)
        flip_indices = np.random.choice(X_drifted[two_year_mask].index, 
                                       size=min(n_to_flip, two_year_mask.sum()), 
                                       replace=False)
        y_drifted.loc[flip_indices] = 1
        drift_info['concept_shifts'].append({
            'type': 'contract_regret',
            'description': 'Two year contract customers more likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 5. Overall base rate shift (global concept shift)
    # Shift the overall churn rate
    base_rate_shift = drift_threshold * 0.1  # Up to 10 percentage points
    if base_rate_shift > 0:
        current_churn_rate = y_drifted.mean()
        target_churn_rate = min(1.0, current_churn_rate + base_rate_shift)
        
        # Adjust labels to match target rate
        n_current_churn = y_drifted.sum()
        n_target_churn = int(len(y_drifted) * target_churn_rate)
        n_to_change = abs(n_target_churn - n_current_churn)
        
        if n_target_churn > n_current_churn:
            # Need more churners - flip some non-churners
            non_churners = X_drifted[y_drifted == 0].index
            flip_indices = np.random.choice(non_churners, 
                                           size=min(n_to_change, len(non_churners)), 
                                           replace=False)
            y_drifted.loc[flip_indices] = 1
        else:
            # Need fewer churners - flip some churners
            churners = X_drifted[y_drifted == 1].index
            flip_indices = np.random.choice(churners, 
                                           size=min(n_to_change, len(churners)), 
                                           replace=False)
            y_drifted.loc[flip_indices] = 0
        
        drift_info['concept_shifts'].append({
            'type': 'base_rate_shift',
            'description': f'Overall churn rate shifted from {current_churn_rate:.3f} to {target_churn_rate:.3f}',
            'shift_amount': base_rate_shift
        })
    
    print(f"✓ Applied {len(drift_info['covariate_shifts'])} covariate shifts")
    print(f"✓ Applied {len(drift_info['concept_shifts'])} concept shifts")
    print(f"Final churn rate: {y_drifted.mean():.3f} (original: {y.mean():.3f})")
    
    return X_drifted, y_drifted, drift_info


def create_drift_visualizations(X_original, y_original, X_drifted, y_drifted, 
                                metrics_original, metrics_drifted, drift_threshold, 
                                save_dir='drift_plots'):
    """
    Create visualization plots for drift analysis.
    
    Parameters:
    -----------
    X_original : pd.DataFrame
        Original feature data
    y_original : pd.Series
        Original target labels
    X_drifted : pd.DataFrame
        Drifted feature data
    y_drifted : pd.Series
        Drifted target labels
    metrics_original : dict
        Metrics on original data
    metrics_drifted : dict
        Metrics on drifted data
    drift_threshold : float
        Drift threshold used
    save_dir : str
        Directory to save plots
        
    Returns:
    --------
    plot_paths : list
        List of paths to saved plot files
    """
    import os
    from sklearn.metrics import roc_curve
    
    os.makedirs(save_dir, exist_ok=True)
    plot_paths = []
    
    # 1. ROC Curve Comparison
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # Get predictions for ROC curves (assuming they're passed or calculated)
    # For now, we'll create a placeholder - in practice, predictions should be passed
    try:
        if 'y_prob_original' in metrics_original and 'y_prob_drifted' in metrics_drifted:
            fpr_orig, tpr_orig, _ = roc_curve(y_original, metrics_original['y_prob_original'])
            fpr_drift, tpr_drift, _ = roc_curve(y_drifted, metrics_drifted['y_prob_drifted'])
            
            ax.plot(fpr_orig, tpr_orig, label=f'Original (AUC={metrics_original["auc"]:.3f})', linewidth=2)
            ax.plot(fpr_drift, tpr_drift, label=f'Drifted (AUC={metrics_drifted["auc"]:.3f})', linewidth=2)
            ax.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)
            ax.set_xlabel('False Positive Rate', fontsize=12)
            ax.set_ylabel('True Positive Rate', fontsize=12)
            ax.set_title(f'ROC Curve Comparison (Drift Threshold: {drift_threshold})', fontsize=14, fontweight='bold')
            ax.legend(loc='lower right', fontsize=10)
            ax.grid(alpha=0.3)
    except:
        pass  # Skip if predictions not available
    
    plt.tight_layout()
    roc_path = os.path.join(save_dir, 'roc_curve_comparison.png')
    plt.savefig(roc_path, dpi=300, bbox_inches='tight')
    plt.close()
    plot_paths.append(roc_path)
    
    # 2. Metric Degradation Bar Chart
    fig, ax = plt.subplots(figsize=(10, 6))
    
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    original_vals = [metrics_original.get(m, 0) for m in metrics_to_plot]
    drifted_vals = [metrics_drifted.get(m, 0) for m in metrics_to_plot]
    degradations = [orig - drift for orig, drift in zip(original_vals, drifted_vals)]
    
    x = np.arange(len(metrics_to_plot))
    width = 0.35
    
    bars1 = ax.bar(x - width/2, original_vals, width, label='Original', alpha=0.8, color='#2ecc71')
    bars2 = ax.bar(x + width/2, drifted_vals, width, label='Drifted', alpha=0.8, color='#e74c3c')
    
    # Add degradation percentages on bars
    for i, (orig, drift, deg) in enumerate(zip(original_vals, drifted_vals, degradations)):
        if orig > 0:
            pct = (deg / orig) * 100
            ax.text(i, max(orig, drift) + 0.02, f'{pct:.1f}%', 
                   ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    ax.set_xlabel('Metrics', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_title(f'Model Performance Degradation (Drift Threshold: {drift_threshold})', 
                fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(metrics_to_plot, fontsize=11)
    ax.legend(fontsize=10)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, 1.1])
    
    plt.tight_layout()
    metric_path = os.path.join(save_dir, 'metric_degradation.png')
    plt.savefig(metric_path, dpi=300, bbox_inches='tight')
    plt.close()
    plot_paths.append(metric_path)
    
    # 3. Feature Distribution Comparison (for key numeric features)
    numeric_cols = X_original.select_dtypes(include=[np.number]).columns[:6]  # Top 6 numeric features
    
    if len(numeric_cols) > 0:
        n_cols = min(3, len(numeric_cols))
        n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        axes = axes.flatten() if len(numeric_cols) > 1 else [axes]
        
        for idx, col in enumerate(numeric_cols):
            if idx >= len(axes):
                break
            ax = axes[idx]
            
            ax.hist(X_original[col].dropna(), bins=30, alpha=0.6, label='Original', 
                   color='#2ecc71', density=True)
            ax.hist(X_drifted[col].dropna(), bins=30, alpha=0.6, label='Drifted', 
                   color='#e74c3c', density=True)
            ax.set_xlabel(col, fontsize=10)
            ax.set_ylabel('Density', fontsize=10)
            ax.set_title(f'{col} Distribution', fontsize=11, fontweight='bold')
            ax.legend(fontsize=9)
            ax.grid(alpha=0.3)
        
        # Hide unused subplots
        for idx in range(len(numeric_cols), len(axes)):
            axes[idx].axis('off')
        
        plt.suptitle(f'Feature Distribution Drift (Drift Threshold: {drift_threshold})', 
                    fontsize=14, fontweight='bold', y=1.02)
        plt.tight_layout()
        dist_path = os.path.join(save_dir, 'feature_distributions.png')
        plt.savefig(dist_path, dpi=300, bbox_inches='tight')
        plt.close()
        plot_paths.append(dist_path)
    
    # 4. Churn Rate Comparison
    fig, ax = plt.subplots(figsize=(8, 6))
    
    churn_original = y_original.mean()
    churn_drifted = y_drifted.mean()
    
    categories = ['Original', 'Drifted']
    churn_rates = [churn_original, churn_drifted]
    colors = ['#2ecc71', '#e74c3c']
    
    bars = ax.bar(categories, churn_rates, color=colors, alpha=0.8, width=0.6)
    ax.set_ylabel('Churn Rate', fontsize=12)
    ax.set_title(f'Churn Rate Shift (Drift Threshold: {drift_threshold})', 
                fontsize=14, fontweight='bold')
    ax.set_ylim([0, max(churn_rates) * 1.2])
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, rate in zip(bars, churn_rates):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{rate:.3f}',
               ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # Add change annotation
    change = churn_drifted - churn_original
    change_pct = (change / churn_original) * 100 if churn_original > 0 else 0
    ax.annotate(f'Change: {change:+.3f} ({change_pct:+.1f}%)',
               xy=(1, churn_drifted), xytext=(1.3, churn_drifted + 0.05),
               arrowprops=dict(arrowstyle='->', color='black', lw=1.5),
               fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    churn_path = os.path.join(save_dir, 'churn_rate_shift.png')
    plt.savefig(churn_path, dpi=300, bbox_inches='tight')
    plt.close()
    plot_paths.append(churn_path)
    
    return plot_paths
'''

'\ndef simulate_drifted_data(X, y, drift_threshold=0.5, random_state=42):\n    """\n    Simulate drifted data with both covariate shift and concept shift.\n\n    Parameters:\n    -----------\n    X : pd.DataFrame\n        Original feature dataframe (before preprocessing)\n    y : pd.Series\n        Original target labels (0/1)\n    drift_threshold : float\n        Controls the intensity of drift (0.0 to 1.0)\n        - 0.0: No drift\n        - 0.5: Moderate drift\n        - 1.0: Severe drift\n    random_state : int\n        Random seed for reproducibility\n\n    Returns:\n    --------\n    X_drifted : pd.DataFrame\n        Drifted feature dataframe\n    y_drifted : pd.Series\n        Drifted target labels (after concept shift)\n    drift_info : dict\n        Information about what drift was applied\n    """\n    np.random.seed(random_state)\n    X_drifted = X.copy()\n    y_drifted = y.copy()\n\n    drift_info = {\n        \'covariate_shifts\': [],\n        \'concept_shifts\': [],\n    

In [7]:
# Example: Generate drifted data and evaluate model performance with MLflow logging

drift_threshold = 0.5

# Generate drifted data with moderate drift
X_drifted, y_drifted, drift_info = simulate_drifted_data(X, y, drift_threshold=drift_threshold, random_state=42)

print("\n" + "="*60)
print("DRIFT SIMULATION SUMMARY")
print("="*60)
print(f"\nCovariate Shifts Applied: {len(drift_info['covariate_shifts'])}")
for shift in drift_info['covariate_shifts'][:5]:  # Show first 5
    print(f"  - {shift.get('feature', 'unknown')}: {shift.get('type', 'unknown')}")
if len(drift_info['covariate_shifts']) > 5:
    print(f"  ... and {len(drift_info['covariate_shifts']) - 5} more")

print(f"\nConcept Shifts Applied: {len(drift_info['concept_shifts'])}")
for shift in drift_info['concept_shifts']:
    print(f"  - {shift.get('type', 'unknown')}: {shift.get('description', '')}")

print("\n" + "="*60)
print("MODEL PERFORMANCE COMPARISON")
print("="*60)

# Evaluate on original test set (baseline)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Get predictions on original test set
y_pred_original = pipeline.predict(X_test)
y_prob_original = pipeline.predict_proba(X_test)[:, 1]

# Get predictions on drifted data (using same test size for comparison)
_, X_test_drifted, _, y_test_drifted = train_test_split(X_drifted, y_drifted, test_size=0.2, random_state=42)

y_pred_drifted = pipeline.predict(X_test_drifted)
y_prob_drifted = pipeline.predict_proba(X_test_drifted)[:, 1]

# Calculate metrics
metrics_original = {
    'accuracy': accuracy_score(y_test, y_pred_original),
    'precision': precision_score(y_test, y_pred_original),
    'recall': recall_score(y_test, y_pred_original),
    'f1': f1_score(y_test, y_pred_original),
    'auc': roc_auc_score(y_test, y_prob_original),
    'y_prob_original': y_prob_original
}

metrics_drifted = {
    'accuracy': accuracy_score(y_test_drifted, y_pred_drifted),
    'precision': precision_score(y_test_drifted, y_pred_drifted),
    'recall': recall_score(y_test_drifted, y_pred_drifted),
    'f1': f1_score(y_test_drifted, y_pred_drifted),
    'auc': roc_auc_score(y_test_drifted, y_prob_drifted),
    'y_prob_drifted': y_prob_drifted
}

# Calculate degradation metrics
degradation_metrics = {}
for metric in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
    orig_val = metrics_original[metric]
    drift_val = metrics_drifted[metric]
    degradation = orig_val - drift_val
    degradation_pct = (degradation / orig_val) * 100 if orig_val > 0 else 0
    degradation_metrics[f'{metric}_degradation'] = degradation
    degradation_metrics[f'{metric}_degradation_pct'] = degradation_pct

# Display comparison
print("\nMetric                    Original    Drifted     Degradation")
print("-" * 65)
for metric in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
    original_val = metrics_original[metric]
    drifted_val = metrics_drifted[metric]
    degradation = degradation_metrics[f'{metric}_degradation']
    degradation_pct = degradation_metrics[f'{metric}_degradation_pct']
    print(f"{metric:20s} {original_val:8.4f}    {drifted_val:8.4f}    {degradation:7.4f} ({degradation_pct:5.1f}%)")

print("\n" + "="*60)

# ============================================
# MLflow Logging
# ============================================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("telco-drift-analysis")

with mlflow.start_run(run_name=f'drift_threshold_{drift_threshold}'):
    # Log drift threshold parameter
    mlflow.log_param('drift_threshold', drift_threshold)
    mlflow.log_param('n_covariate_shifts', len(drift_info['covariate_shifts']))
    mlflow.log_param('n_concept_shifts', len(drift_info['concept_shifts']))
    
    # Log data using mlflow.data module
    print("\nLogging drifted data to MLflow...")
    
    # Combine X and y for data logging
    X_test_with_target = X_test.copy()
    X_test_with_target['Churn'] = y_test
    baseline_dataset = mlflow.data.from_pandas(X_test_with_target)
    mlflow.log_input(baseline_dataset, context='baseline_test')
    
    X_drifted_with_target = X_test_drifted.copy()
    X_drifted_with_target['Churn'] = y_test_drifted
    drifted_dataset = mlflow.data.from_pandas(X_drifted_with_target)
    mlflow.log_input(drifted_dataset, context='drifted_test')
    
    # Log baseline metrics
    print("Logging baseline metrics...")
    for metric_name, metric_value in metrics_original.items():
        if metric_name != 'y_prob_original':  # Skip prediction arrays
            mlflow.log_metric(f'baseline_{metric_name}', metric_value)
    
    # Log drifted metrics
    print("Logging drifted metrics...")
    for metric_name, metric_value in metrics_drifted.items():
        if metric_name != 'y_prob_drifted':  # Skip prediction arrays
            mlflow.log_metric(f'drifted_{metric_name}', metric_value)
    
    # Log degradation metrics
    print("Logging degradation metrics...")
    for metric_name, metric_value in degradation_metrics.items():
        mlflow.log_metric(metric_name, metric_value)
    
    # Log data statistics
    mlflow.log_metric('baseline_churn_rate', y_test.mean())
    mlflow.log_metric('drifted_churn_rate', y_test_drifted.mean())
    mlflow.log_metric('churn_rate_change', y_test_drifted.mean() - y_test.mean())
    mlflow.log_metric('baseline_data_size', len(X_test))
    mlflow.log_metric('drifted_data_size', len(X_test_drifted))
    
    # Create and log visualizations
    print("Creating visualizations...")
    plot_paths = create_drift_visualizations(
        X_test, y_test, X_test_drifted, y_test_drifted,
        metrics_original, metrics_drifted, drift_threshold,
        save_dir='drift_plots'
    )
    
    for plot_path in plot_paths:
        mlflow.log_artifact(plot_path, artifact_path='plots')
        print(f"  ✓ Logged {plot_path}")
    
    # Log drift info as JSON artifact
    import json
    drift_info_json = json.dumps(drift_info, indent=2, default=str)
    with open('drift_info.json', 'w') as f:
        f.write(drift_info_json)
    mlflow.log_artifact('drift_info.json', artifact_path='drift_info')
    
    print(f"\n✓ MLflow run completed. View at: {mlflow.get_tracking_uri()}")
    print(f"Run ID: {mlflow.active_run().info.run_id}")


Simulating combined drift with threshold: 0.50
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.376 (original: 0.265)

DRIFT SIMULATION SUMMARY

Covariate Shifts Applied: 15
  - SeniorCitizen: mean_shift
  - tenure: variance_increase
  - MonthlyCharges: multiplicative_shift
  - TotalCharges: outliers
  - monthly_total_ratio: mean_shift
  ... and 10 more

Concept Shifts Applied: 5
  - high_value_retention: High MonthlyCharges customers now less likely to churn
  - tenure_fatigue: Very long tenure customers more likely to churn
  - service_overwhelm: High service engagement customers more likely to churn
  - contract_regret: Two year contract customers more likely to churn
  - base_rate_shift: Overall churn rate shifted from 0.326 to 0.376

MODEL PERFORMANCE COMPARISON

Metric                    Original    Drifted     Degradation
----------------------------------------

d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at

Logging baseline metrics...
Logging drifted metrics...
Logging degradation metrics...
Creating visualizations...
  ✓ Logged drift_plots\roc_curve_comparison.png
  ✓ Logged drift_plots\metric_degradation.png
  ✓ Logged drift_plots\feature_distributions.png
  ✓ Logged drift_plots\churn_rate_shift.png

✓ MLflow run completed. View at: http://localhost:5000
Run ID: 785c018852f04aa8962f83c9b24044d4
🏃 View run drift_threshold_0.5 at: http://localhost:5000/#/experiments/2/runs/785c018852f04aa8962f83c9b24044d4
🧪 View experiment at: http://localhost:5000/#/experiments/2


In [8]:
# Test different drift thresholds with MLflow logging

print("Testing model performance across different drift thresholds with MLflow logging...\n")

drift_thresholds = [0.0, 0.25, 0.5, 0.75, 1.0]
results = []

# Baseline performance on original test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_pred_baseline = pipeline.predict(X_test)
y_prob_baseline = pipeline.predict_proba(X_test)[:, 1]

baseline_metrics = {
    'accuracy': accuracy_score(y_test, y_pred_baseline),
    'precision': precision_score(y_test, y_pred_baseline),
    'recall': recall_score(y_test, y_pred_baseline),
    'f1': f1_score(y_test, y_pred_baseline),
    'auc': roc_auc_score(y_test, y_prob_baseline)
}

# Set up MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("telco-drift-threshold-analysis")

# Store summary plot path for later logging
summary_plot_path = None

for threshold in drift_thresholds:
    print(f"\nTesting drift threshold: {threshold:.2f}")
    
    # Create a run for each threshold
    with mlflow.start_run(run_name=f'threshold_{threshold}'):
        X_drifted, y_drifted, drift_info = simulate_drifted_data(X, y, drift_threshold=threshold, random_state=42)
        
        # Use same test size
        _, X_test_drifted, _, y_test_drifted = train_test_split(X_drifted, y_drifted, test_size=0.2, random_state=42)
        
        y_pred_drifted = pipeline.predict(X_test_drifted)
        y_prob_drifted = pipeline.predict_proba(X_test_drifted)[:, 1]
        
        # Calculate all metrics
        metrics = {
            'threshold': threshold,
            'accuracy': accuracy_score(y_test_drifted, y_pred_drifted),
            'precision': precision_score(y_test_drifted, y_pred_drifted),
            'recall': recall_score(y_test_drifted, y_pred_drifted),
            'f1': f1_score(y_test_drifted, y_pred_drifted),
            'auc': roc_auc_score(y_test_drifted, y_prob_drifted),
            'churn_rate': y_test_drifted.mean()
        }
        
        # Calculate degradation metrics
        degradation_metrics = {}
        for metric_name in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
            baseline_val = baseline_metrics[metric_name]
            drifted_val = metrics[metric_name]
            degradation = baseline_val - drifted_val
            degradation_pct = (degradation / baseline_val) * 100 if baseline_val > 0 else 0
            degradation_metrics[f'{metric_name}_degradation'] = degradation
            degradation_metrics[f'{metric_name}_degradation_pct'] = degradation_pct
        
        results.append({**metrics, **degradation_metrics})
        
        # Log to MLflow
        mlflow.log_param('drift_threshold', threshold)
        mlflow.log_param('n_covariate_shifts', len(drift_info['covariate_shifts']))
        mlflow.log_param('n_concept_shifts', len(drift_info['concept_shifts']))
        
        # Log data
        X_test_with_target = X_test_drifted.copy()
        X_test_with_target['Churn'] = y_test_drifted
        drifted_dataset = mlflow.data.from_pandas(X_test_with_target)
        mlflow.log_input(drifted_dataset, context='drifted_test')
        
        # Log all metrics
        for metric_name, metric_value in metrics.items():
            if metric_name != 'threshold':
                mlflow.log_metric(metric_name, metric_value)
        
        # Log degradation metrics
        for metric_name, metric_value in degradation_metrics.items():
            mlflow.log_metric(metric_name, metric_value)
        
        # Log additional statistics
        mlflow.log_metric('churn_rate_change', y_test_drifted.mean() - y_test.mean())
        
        # Calculate degradation for display
        acc_degradation = degradation_metrics['accuracy_degradation_pct']
        auc_degradation = degradation_metrics['auc_degradation_pct']
        f1_degradation = degradation_metrics['f1_degradation_pct']
        
        print(f"  Accuracy: {metrics['accuracy']:.4f} (↓{acc_degradation:.1f}%)")
        print(f"  F1-Score: {metrics['f1']:.4f} (↓{f1_degradation:.1f}%)")
        print(f"  AUC: {metrics['auc']:.4f} (↓{auc_degradation:.1f}%)")

# Create summary visualization
print("\nCreating summary visualization...")
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Accuracy degradation across thresholds
ax1 = axes[0, 0]
thresholds = [r['threshold'] for r in results]
accuracies = [r['accuracy'] for r in results]
acc_degradations = [r['accuracy_degradation_pct'] for r in results]
ax1.plot(thresholds, accuracies, 'o-', linewidth=2, markersize=8, label='Accuracy', color='#3498db')
ax1.axhline(y=baseline_metrics['accuracy'], color='#2ecc71', linestyle='--', linewidth=2, label='Baseline')
ax1.set_xlabel('Drift Threshold', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Accuracy vs Drift Threshold', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)

# 2. AUC degradation across thresholds
ax2 = axes[0, 1]
aucs = [r['auc'] for r in results]
auc_degradations = [r['auc_degradation_pct'] for r in results]
ax2.plot(thresholds, aucs, 'o-', linewidth=2, markersize=8, label='AUC', color='#9b59b6')
ax2.axhline(y=baseline_metrics['auc'], color='#2ecc71', linestyle='--', linewidth=2, label='Baseline')
ax2.set_xlabel('Drift Threshold', fontsize=12)
ax2.set_ylabel('AUC', fontsize=12)
ax2.set_title('AUC vs Drift Threshold', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)

# 3. F1 degradation across thresholds
ax3 = axes[1, 0]
f1_scores = [r['f1'] for r in results]
f1_degradations = [r['f1_degradation_pct'] for r in results]
ax3.plot(thresholds, f1_scores, 'o-', linewidth=2, markersize=8, label='F1-Score', color='#e67e22')
ax3.axhline(y=baseline_metrics['f1'], color='#2ecc71', linestyle='--', linewidth=2, label='Baseline')
ax3.set_xlabel('Drift Threshold', fontsize=12)
ax3.set_ylabel('F1-Score', fontsize=12)
ax3.set_title('F1-Score vs Drift Threshold', fontsize=14, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(alpha=0.3)

# 4. Percentage degradation across thresholds
ax4 = axes[1, 1]
ax4.plot(thresholds, acc_degradations, 'o-', linewidth=2, markersize=8, label='Accuracy', color='#3498db')
ax4.plot(thresholds, auc_degradations, 's-', linewidth=2, markersize=8, label='AUC', color='#9b59b6')
ax4.plot(thresholds, f1_degradations, '^-', linewidth=2, markersize=8, label='F1-Score', color='#e67e22')
ax4.set_xlabel('Drift Threshold', fontsize=12)
ax4.set_ylabel('Degradation (%)', fontsize=12)
ax4.set_title('Performance Degradation vs Drift Threshold', fontsize=14, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(alpha=0.3)
ax4.axhline(y=0, color='black', linestyle='-', linewidth=1)

plt.suptitle('Model Performance Across Drift Thresholds', fontsize=16, fontweight='bold', y=1.0)
plt.tight_layout()
summary_plot_path = 'drift_threshold_summary.png'
plt.savefig(summary_plot_path, dpi=300, bbox_inches='tight')
plt.close()

# Log summary plot to a final summary run
with mlflow.start_run(run_name='summary'):
    mlflow.log_param('baseline_accuracy', baseline_metrics['accuracy'])
    mlflow.log_param('baseline_f1', baseline_metrics['f1'])
    mlflow.log_param('baseline_auc', baseline_metrics['auc'])
    mlflow.log_artifact(summary_plot_path, artifact_path='plots')
    print(f"✓ Logged summary plot: {summary_plot_path}")

# Summary table
print("\n" + "="*80)
print("PERFORMANCE DEGRADATION SUMMARY")
print("="*80)
print(f"\nBaseline Performance:")
print(f"  Accuracy: {baseline_metrics['accuracy']:.4f}")
print(f"  F1-Score: {baseline_metrics['f1']:.4f}")
print(f"  AUC: {baseline_metrics['auc']:.4f}")

print(f"\n{'Threshold':<12} {'Accuracy':<12} {'F1-Score':<12} {'AUC':<12} {'Acc Deg (%)':<12} {'AUC Deg (%)':<12} {'F1 Deg (%)':<12}")
print("-" * 80)
for r in results:
    print(f"{r['threshold']:<12.2f} {r['accuracy']:<12.4f} {r['f1']:<12.4f} {r['auc']:<12.4f} "
          f"{r['accuracy_degradation_pct']:<12.2f} {r['auc_degradation_pct']:<12.2f} {r['f1_degradation_pct']:<12.2f}")

print(f"\n All results logged to MLflow experiment: telco-drift-threshold-analysis")


Testing model performance across different drift thresholds with MLflow logging...


Testing drift threshold: 0.00
Simulating combined drift with threshold: 0.00
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 13 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  Accuracy: 0.7949 (↓-0.2%)
  F1-Score: 0.6488 (↓0.4%)
  AUC: 0.8578 (↓-0.1%)
🏃 View run threshold_0.0 at: http://localhost:5000/#/experiments/3/runs/96703e1049684c89b38866cbe12f0402
🧪 View experiment at: http://localhost:5000/#/experiments/3

Testing drift threshold: 0.25
Simulating combined drift with threshold: 0.25
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.322 (original: 0.265)


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  Accuracy: 0.7388 (↓6.9%)
  F1-Score: 0.6060 (↓7.0%)
  AUC: 0.7579 (↓11.6%)
🏃 View run threshold_0.25 at: http://localhost:5000/#/experiments/3/runs/01a39f2adaac4f4cbac0cc5787777738
🧪 View experiment at: http://localhost:5000/#/experiments/3

Testing drift threshold: 0.50
Simulating combined drift with threshold: 0.50
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.376 (original: 0.265)


...

  Accuracy: 0.6920 (↓12.8%)
  F1-Score: 0.5607 (↓13.9%)
  AUC: 0.7087 (↓17.3%)
🏃 View run threshold_0.5 at: http://localhost:5000/#/experiments/3/runs/38f8a39236e64c68a0bcaa7bb540af76
🧪 View experiment at: http://localhost:5000/#/experiments/3

Testing drift threshold: 0.75
Simulating combined drift with threshold: 0.75
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.430 (original: 0.265)


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  Accuracy: 0.6444 (↓18.8%)
  F1-Score: 0.5233 (↓19.7%)
  AUC: 0.6649 (↓22.4%)
🏃 View run threshold_0.75 at: http://localhost:5000/#/experiments/3/runs/0a605b6592104024a8a0e2d170e8771c
🧪 View experiment at: http://localhost:5000/#/experiments/3

Testing drift threshold: 1.00
Simulating combined drift with threshold: 1.00
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.478 (original: 0.265)


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  Accuracy: 0.6061 (↓23.6%)
  F1-Score: 0.5203 (↓20.1%)
  AUC: 0.6293 (↓26.6%)
🏃 View run threshold_1.0 at: http://localhost:5000/#/experiments/3/runs/88a4269f14bb4b4db39aa1457a7b9a94
🧪 View experiment at: http://localhost:5000/#/experiments/3

Creating summary visualization...
✓ Logged summary plot: drift_threshold_summary.png
🏃 View run summary at: http://localhost:5000/#/experiments/3/runs/80b503697a1d4ace88dc6c4ef0e1c313
🧪 View experiment at: http://localhost:5000/#/experiments/3

PERFORMANCE DEGRADATION SUMMARY

Baseline Performance:
  Accuracy: 0.7935
  F1-Score: 0.6515
  AUC: 0.8571

Threshold    Accuracy     F1-Score     AUC          Acc Deg (%)  AUC Deg (%)  F1 Deg (%)  
--------------------------------------------------------------------------------
0.00         0.7949       0.6488       0.8578       -0.18        -0.07        0.41        
0.25         0.7388       0.6060       0.7579       6.89         11.58        6.98        
0.50         0.6920       0.5607       0.7087   

### Drift analysis

We take our random forest model and subject it to a drifted dataset based on the original. The drifted dataset has the same columns and data that the baseline model was trained on. We subject the model to varied thresholds of drift (from 0 - 1). The drift simulation is intended to simulate model degradation across scenarios in increasing order of magnitude; i.e. the aggressiveness increases as the threshold increases.

We note, that as expected - the baseline model suffers as a result of drift, losing 22.8% of its accuracy at when the threshold is set to 1. 

## Interventions after drift is detected

### DDLA intervention for retraining

Limitations: This is an implementation based on the approach used by Dong et al. (2024) for DDLAs that identify regions for low accuracy within a model. The authors use active learning - where predictions are passed to human annotators for ground truth. In our case, due to the limitations of our dataset and the lack of any domain experts - we need to assume that "generated" labels for annotators are ground truths - which do not accurately represent the authors' implementation of this algorithm. 

Furthermore, the approach itself appears appears to first inform deployments of harmful drift - if detected, and then further inform them of these low accuracy regions for selective retraining of the model. Selective retraining itself is not very clear (to me) in the paper - so implemtation will differ from the actual implementation.

The source [code](https://github.com/SiSijie/data-drift-in-ML/blob/main/examples/Human-activaty_test.ipynb) is embedded in this cell.

Update: On inspecting the example in the authors' notebook, it appears that they are in fact generating their own labels as opposed to using an actual active learning enabled pipeline. It seems the active learning bit is a theory rather than an implementation. This makes our job easier.

In [9]:
def identify_ddlas_decision_tree(trained_pipeline, X_test, y_test, max_depth_range=(3, 10), min_samples_leaf_range=(0.01, 0.05), random_state=42):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.model_selection import GridSearchCV
    
    print("Identifying DDLAs with tree based approach")
    
    # Step 1: Get model predictions and overall accuracy
    y_pred = trained_pipeline.predict(X_test)
    y_prob = trained_pipeline.predict_proba(X_test)[:, 1]
    overall_accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Overall model accuracy: {overall_accuracy:.4f}")
    
    # Step 2: Re-label predictions (0=correct, 1=incorrect), essentially same as paper
    correct_predictions = (y_pred == y_test).astype(int)
    y_relabeled = 1 - correct_predictions
    
    incorrect_rate = y_relabeled.mean()
    print(f"  Overall incorrect prediction rate: {incorrect_rate:.4f}")
    
    # Step 3: Get preprocessed features for decision tree training
    # We need the same preprocessing that was used for the main model
    X_test_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_test)
    
    # Convert to DataFrame for easier handling (get feature names from preprocessor); this might cause an issue
    try:
        # Try to get feature names from the preprocessor
        feature_names = trained_pipeline.named_steps['preprocessor'].get_feature_names_out()
        X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=feature_names, index=X_test.index)
    except:
        # Fallback if feature names aren't available
        n_features = X_test_preprocessed.shape[1]
        feature_names = [f'feature_{i}' for i in range(n_features)]
        X_test_preprocessed_df = pd.DataFrame(X_test_preprocessed, columns=feature_names, index=X_test.index)
    
    # Step 4: Train decision tree with hyperparameter tuning to identify failure patterns
    param_grid = {
        'max_depth': list(range(*max_depth_range)),
        'min_samples_leaf': [max(1, int(frac * len(X_test_preprocessed_df))) for frac in np.linspace(*min_samples_leaf_range, 5)]
    }
    
    dt = DecisionTreeClassifier(random_state=random_state, class_weight='balanced')
    dt_grid = GridSearchCV(dt, param_grid, cv=5, scoring='f1', n_jobs=-1)
    dt_grid.fit(X_test_preprocessed_df, y_relabeled)
    
    best_dt = dt_grid.best_estimator_
    
    print(f"  Best decision tree params: {dt_grid.best_params_}")
    print(f"  Decision tree F1 score: {dt_grid.best_score_:.4f}")
    
    # Step 5: Identify leaf nodes and their accuracies
    leaf_indices = best_dt.apply(X_test_preprocessed_df)
    unique_leaves = np.unique(leaf_indices)
    
    ddlas = []
    all_leaf_info = {}
    
    for leaf in unique_leaves:
        leaf_mask = (leaf_indices == leaf)
        leaf_data_indices = np.where(leaf_mask)[0]
        
        if len(leaf_data_indices) > 0:
            # Calculate accuracy for this leaf using original indices
            leaf_y_true = y_test.iloc[leaf_data_indices]
            leaf_y_pred = y_pred[leaf_data_indices]
            leaf_accuracy = accuracy_score(leaf_y_true, leaf_y_pred)
            
            # Get the decision path for this leaf (using first sample as representative)
            decision_path = best_dt.decision_path(X_test_preprocessed_df.iloc[leaf_data_indices[0:1]])
            
            leaf_info = {
                'leaf_id': leaf,
                'accuracy': leaf_accuracy,
                'error_rate': 1 - leaf_accuracy,
                'sample_count': len(leaf_data_indices),
                'sample_indices': leaf_data_indices.tolist(),
                'sample_fraction': len(leaf_data_indices) / len(X_test),
                'is_ddla': leaf_accuracy < overall_accuracy
            }
            
            all_leaf_info[leaf] = leaf_info
            
            # Identify as DDLA if accuracy < overall accuracy
            if leaf_accuracy < overall_accuracy:
                ddlas.append(leaf_info)
    
    # Sort DDLAs by error rate (highest first)
    ddlas.sort(key=lambda x: x['error_rate'], reverse=True)
    
    print(f" Found {len(ddlas)} DDLAs out of {len(unique_leaves)} total leaf nodes")
    
    # Calculate DDLA statistics
    ddla_sample_count = sum(ddla['sample_count'] for ddla in ddlas)
    ddla_fraction = ddla_sample_count / len(X_test)
    
    print(f" DDLA coverage: {ddla_sample_count}/{len(X_test)} samples ({ddla_fraction:.3f})")
    
    return {
        'ddlas': ddlas,
        'decision_tree': best_dt,
        'overall_accuracy': overall_accuracy,
        'overall_error_rate': incorrect_rate,
        'ddla_fraction_baseline': ddla_fraction,
        'all_leaf_info': all_leaf_info,
        'preprocessed_features': X_test_preprocessed_df,
        'feature_names': feature_names,
        'grid_search_results': dt_grid.cv_results_
    }


def detect_harmful_drift_ddla(ddla_info, X_serving_data, trained_pipeline, 
                              theta_inc=0.5, theta_ddla=0.1):
    
    print("Detecting harmful drift")
    
    decision_tree = ddla_info['decision_tree']
    baseline_ddla_fraction = ddla_info['ddla_fraction_baseline']
    
    # Preprocess serving data using the same pipeline
    X_serving_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_serving_data)
    X_serving_preprocessed_df = pd.DataFrame(
        X_serving_preprocessed, 
        columns=ddla_info['feature_names'], 
        index=X_serving_data.index
    )
    
    # Predict leaf assignments for serving data
    serving_leaf_indices = decision_tree.apply(X_serving_preprocessed_df)
    
    # Get DDLA leaf IDs
    ddla_leaf_ids = [ddla['leaf_id'] for ddla in ddla_info['ddlas']]
    
    # Calculate serving DDLA ratio
    serving_ddla_count = sum(1 for leaf in serving_leaf_indices if leaf in ddla_leaf_ids)
    serving_ddla_fraction = serving_ddla_count / len(X_serving_data)
    
    print(f"  Baseline DDLA fraction: {baseline_ddla_fraction:.4f}")
    print(f"  Serving DDLA fraction: {serving_ddla_fraction:.4f}")
    
    # Determine if harmful drift occurred
    if serving_ddla_fraction <= baseline_ddla_fraction:
        is_harmful = False
        drift_type = "benign"
        reason = "DDLA fraction decreased or stayed same"
    else:
        # Check thresholds for harmful drift
        if baseline_ddla_fraction > 0:
            ratio_increase = (serving_ddla_fraction - baseline_ddla_fraction) / baseline_ddla_fraction
        else:
            ratio_increase = float('inf') if serving_ddla_fraction > 0 else 0
        
        is_harmful = (ratio_increase > theta_inc) and (serving_ddla_fraction > theta_ddla)
        
        if is_harmful:
            drift_type = "harmful"
            reason = f"DDLA ratio increased by {ratio_increase:.2%} (>{theta_inc:.1%}) and exceeds {theta_ddla:.1%}"
        else:
            drift_type = "benign"
            if ratio_increase <= theta_inc:
                reason = f"DDLA ratio increase {ratio_increase:.2%} below threshold {theta_inc:.1%}"
            else:
                reason = f"DDLA fraction {serving_ddla_fraction:.3f} below threshold {theta_ddla:.1%}"
    
    print(f" Drift assessment: {drift_type.upper()}")
    print(f"  Reason: {reason}")
    
    return {
        'is_harmful_drift': is_harmful,
        'drift_type': drift_type,
        'reason': reason,
        'baseline_ddla_fraction': baseline_ddla_fraction,
        'serving_ddla_fraction': serving_ddla_fraction,
        'ddla_fraction_change': serving_ddla_fraction - baseline_ddla_fraction,
        'ddla_fraction_change_pct': ((serving_ddla_fraction - baseline_ddla_fraction) / baseline_ddla_fraction * 100) if baseline_ddla_fraction > 0 else 0,
        'ratio_train': baseline_ddla_fraction,
        'ratio_serving': serving_ddla_fraction,
        'serving_ddla_count': serving_ddla_count,
        'serving_total_count': len(X_serving_data),
        'thresholds_used': {'theta_inc': theta_inc, 'theta_ddla': theta_ddla}
    }


def run_ddla_drift_experiment(X, y, trained_pipeline, drift_thresholds, 
                              experiment_name="telco-ddla-drift-analysis", 
                              random_state=42):
   
    print("Starting DDLA Drift Experiment...")
    print(f"Testing thresholds: {drift_thresholds}")
    
    # Set up MLflow
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)
    
    # Split data for baseline DDLA identification
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    # Step 1: Identify DDLAs on baseline test data
    print("\n" + "="*60)
    print("STEP 1: IDENTIFYING DDLAs ON BASELINE DATA")
    print("="*60)
    
    ddla_info = identify_ddlas_decision_tree(trained_pipeline, X_test, y_test, random_state=random_state)
    
    results = []
    
    # Step 2: Test each drift threshold
    print("\n" + "="*60)
    print("STEP 2: TESTING DDLA APPROACH ACROSS DRIFT THRESHOLDS")
    print("="*60)
    
    for threshold in drift_thresholds:
        print(f"\n Testing drift threshold: {threshold:.2f}")
        
        with mlflow.start_run(run_name=f'ddla_threshold_{threshold}'):
            # Generate drifted data using your existing function
            X_drifted, y_drifted, drift_info = simulate_drifted_data(
                X, y, drift_threshold=threshold, random_state=random_state
            )
            
            # Split drifted data (same way as baseline)
            _, X_test_drifted, _, y_test_drifted = train_test_split(
                X_drifted, y_drifted, test_size=0.2, random_state=random_state
            )
            
            # Test DDLA drift detection
            drift_detection = detect_harmful_drift_ddla(
                ddla_info, X_test_drifted, trained_pipeline
            )
            
            # Get actual performance metrics for comparison
            y_pred_drifted = trained_pipeline.predict(X_test_drifted)
            y_prob_drifted = trained_pipeline.predict_proba(X_test_drifted)[:, 1]
            
            actual_metrics = {
                'accuracy': accuracy_score(y_test_drifted, y_pred_drifted),
                'precision': precision_score(y_test_drifted, y_pred_drifted),
                'recall': recall_score(y_test_drifted, y_pred_drifted),
                'f1': f1_score(y_test_drifted, y_pred_drifted),
                'auc': roc_auc_score(y_test_drifted, y_prob_drifted)
            }
            
            # Calculate performance degradation
            baseline_accuracy = ddla_info['overall_accuracy']
            accuracy_drop = baseline_accuracy - actual_metrics['accuracy']
            accuracy_drop_pct = (accuracy_drop / baseline_accuracy) * 100 if baseline_accuracy > 0 else 0
            
            # Determine if retraining is actually needed (ground truth)
            significant_degradation_threshold = 0.05  # 5% absolute accuracy drop
            actually_needs_retraining = accuracy_drop > significant_degradation_threshold
            
            # Check if DDLA approach made correct decision
            ddla_correct = drift_detection['is_harmful_drift'] == actually_needs_retraining
            
            # Store comprehensive results
            result = {
                'threshold': threshold,
                'ddla_detected_harmful': drift_detection['is_harmful_drift'],
                'ddla_drift_type': drift_detection['drift_type'],
                'actually_needs_retraining': actually_needs_retraining,
                'ddla_correct_decision': ddla_correct,
                
                # DDLA metrics
                'baseline_ddla_fraction': drift_detection['baseline_ddla_fraction'],
                'serving_ddla_fraction': drift_detection['serving_ddla_fraction'],
                'ddla_fraction_change': drift_detection['ddla_fraction_change'],
                'ddla_fraction_change_pct': drift_detection['ddla_fraction_change_pct'],
                
                # Performance metrics
                'actual_accuracy': actual_metrics['accuracy'],
                'accuracy_drop': accuracy_drop,
                'accuracy_drop_pct': accuracy_drop_pct,
                'actual_f1': actual_metrics['f1'],
                'actual_auc': actual_metrics['auc'],
                
                # Drift simulation info
                'n_covariate_shifts': len(drift_info['covariate_shifts']),
                'n_concept_shifts': len(drift_info['concept_shifts']),
                'final_churn_rate': y_test_drifted.mean()
            }
            
            results.append(result)
            
            # Log everything to MLflow
            # Parameters
            mlflow.log_param('drift_threshold', threshold)
            mlflow.log_param('ddla_approach', 'decision_tree')
            mlflow.log_param('n_ddlas_identified', len(ddla_info['ddlas']))
            mlflow.log_param('n_covariate_shifts', len(drift_info['covariate_shifts']))
            mlflow.log_param('n_concept_shifts', len(drift_info['concept_shifts']))
            
            # DDLA metrics
            mlflow.log_metric('baseline_ddla_fraction', drift_detection['baseline_ddla_fraction'])
            mlflow.log_metric('serving_ddla_fraction', drift_detection['serving_ddla_fraction'])
            mlflow.log_metric('ddla_fraction_change', drift_detection['ddla_fraction_change'])
            mlflow.log_metric('ddla_fraction_change_pct', drift_detection['ddla_fraction_change_pct'])
            
            # Decision metrics  
            mlflow.log_metric('ddla_detected_harmful', 1 if drift_detection['is_harmful_drift'] else 0)
            mlflow.log_metric('actually_needs_retraining', 1 if actually_needs_retraining else 0)
            mlflow.log_metric('ddla_correct_decision', 1 if ddla_correct else 0)
            
            # Performance metrics
            for metric_name, metric_value in actual_metrics.items():
                mlflow.log_metric(f'actual_{metric_name}', metric_value)
            
            mlflow.log_metric('accuracy_drop', accuracy_drop)
            mlflow.log_metric('accuracy_drop_pct', accuracy_drop_pct)
            
            # Data info
            mlflow.log_metric('final_churn_rate', y_test_drifted.mean())
            mlflow.log_metric('churn_rate_change', y_test_drifted.mean() - y_test.mean())
            
            # Log data
            X_test_with_target = X_test_drifted.copy()
            X_test_with_target['Churn'] = y_test_drifted
            drifted_dataset = mlflow.data.from_pandas(X_test_with_target)
            mlflow.log_input(drifted_dataset, context='drifted_test')
            
            # Print results
            print(f"  DDLA says: {'HARMFUL' if drift_detection['is_harmful_drift'] else 'BENIGN'}")
            print(f"  Actually needs retraining: {'YES' if actually_needs_retraining else 'NO'}")
            print(f"  DDLA decision correct: {'YES' if ddla_correct else 'NO'}")
            print(f"  Accuracy drop: {accuracy_drop:.4f} ({accuracy_drop_pct:.1f}%)")
            print(f"  DDLA fraction: {drift_detection['baseline_ddla_fraction']:.3f} → {drift_detection['serving_ddla_fraction']:.3f}")
    
    # Create summary visualization and log to MLflow
    create_ddla_summary_visualization(results, ddla_info, experiment_name)
    
    return results


def create_ddla_summary_visualization(results, ddla_info, experiment_name):
    """
    Create comprehensive visualization of DDLA experiment results.
    """
    print("\n Creating DDLA summary visualization")
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    thresholds = [r['threshold'] for r in results]
    
    # 1. DDLA Detection Accuracy
    ax1 = axes[0, 0]
    correct_decisions = [r['ddla_correct_decision'] for r in results]
    accuracy_rate = np.mean(correct_decisions) * 100
    
    colors = ['#27ae60' if correct else '#e74c3c' for correct in correct_decisions]
    bars = ax1.bar(range(len(thresholds)), correct_decisions, color=colors, alpha=0.7)
    ax1.set_xlabel('Drift Threshold', fontsize=11)
    ax1.set_ylabel('Correct Decision (1=Yes, 0=No)', fontsize=11)
    ax1.set_title(f'DDLA Decision Accuracy\n(Overall: {accuracy_rate:.1f}%)', fontsize=12, fontweight='bold')
    ax1.set_xticks(range(len(thresholds)))
    ax1.set_xticklabels([f'{t:.2f}' for t in thresholds])
    ax1.grid(axis='y', alpha=0.3)
    
    # 2. DDLA Fraction Changes
    ax2 = axes[0, 1]
    baseline_fractions = [r['baseline_ddla_fraction'] for r in results]
    serving_fractions = [r['serving_ddla_fraction'] for r in results]
    
    ax2.plot(thresholds, baseline_fractions, 'o-', label='Baseline DDLA', linewidth=2, markersize=6)
    ax2.plot(thresholds, serving_fractions, 's-', label='Serving DDLA', linewidth=2, markersize=6)
    ax2.set_xlabel('Drift Threshold', fontsize=11)
    ax2.set_ylabel('DDLA Fraction', fontsize=11)
    ax2.set_title('DDLA Fraction: Baseline vs Serving', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Actual Performance Degradation
    ax3 = axes[0, 2]
    accuracy_drops = [r['accuracy_drop_pct'] for r in results]
    ax3.plot(thresholds, accuracy_drops, 'o-', color='#e74c3c', linewidth=2, markersize=6)
    ax3.axhline(y=5, color='orange', linestyle='--', label='5% threshold')
    ax3.set_xlabel('Drift Threshold', fontsize=11)
    ax3.set_ylabel('Accuracy Drop (%)', fontsize=11)
    ax3.set_title('Actual Performance Degradation', fontsize=12, fontweight='bold')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. DDLA vs Reality Comparison
    ax4 = axes[1, 0]
    ddla_harmful = [1 if r['ddla_detected_harmful'] else 0 for r in results]
    actually_needs = [1 if r['actually_needs_retraining'] else 0 for r in results]
    
    x = np.arange(len(thresholds))
    width = 0.35
    
    ax4.bar(x - width/2, ddla_harmful, width, label='DDLA Says Harmful', alpha=0.7, color='#3498db')
    ax4.bar(x + width/2, actually_needs, width, label='Actually Needs Retraining', alpha=0.7, color='#e67e22')
    
    ax4.set_xlabel('Drift Threshold', fontsize=11)
    ax4.set_ylabel('Decision (1=Yes, 0=No)', fontsize=11)
    ax4.set_title('DDLA Predictions vs Reality', fontsize=12, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels([f'{t:.2f}' for t in thresholds])
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)
    
    # 5. DDLA Leaf Distribution
    ax5 = axes[1, 1]
    n_ddlas = len(ddla_info['ddlas'])
    ddla_sizes = [ddla['sample_count'] for ddla in ddla_info['ddlas'][:10]]  # Top 10
    ddla_labels = [f"Leaf {ddla['leaf_id']}" for ddla in ddla_info['ddlas'][:10]]
    
    if ddla_sizes:
        ax5.pie(ddla_sizes, labels=ddla_labels, autopct='%1.1f%%', startangle=90)
        ax5.set_title(f'Top 10 DDLA Distribution\n(Total DDLAs: {n_ddlas})', fontsize=12, fontweight='bold')
    
    # 6. Confusion Matrix Style
    ax6 = axes[1, 2]
    
    # Create confusion matrix data
    tp = sum(1 for r in results if r['ddla_detected_harmful'] and r['actually_needs_retraining'])
    tn = sum(1 for r in results if not r['ddla_detected_harmful'] and not r['actually_needs_retraining'])
    fp = sum(1 for r in results if r['ddla_detected_harmful'] and not r['actually_needs_retraining'])
    fn = sum(1 for r in results if not r['ddla_detected_harmful'] and r['actually_needs_retraining'])
    
    confusion_matrix = np.array([[tn, fp], [fn, tp]])
    
    im = ax6.imshow(confusion_matrix, interpolation='nearest', cmap='Blues')
    ax6.set_title('DDLA Confusion Matrix', fontsize=12, fontweight='bold')
    
    # Add text annotations
    thresh = confusion_matrix.max() / 2.
    for i in range(2):
        for j in range(2):
            ax6.text(j, i, format(confusion_matrix[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if confusion_matrix[i, j] > thresh else "black",
                    fontsize=14, fontweight='bold')
    
    ax6.set_xticks([0, 1])
    ax6.set_xticklabels(['Predicted Benign', 'Predicted Harmful'])
    ax6.set_yticks([0, 1])
    ax6.set_yticklabels(['Actually Benign', 'Actually Harmful'])
    
    plt.tight_layout()
    
    # Save and log to MLflow
    summary_plot_path = f'ddla_summary_{experiment_name.replace("-", "_")}.png'
    plt.savefig(summary_plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Log summary to MLflow
    with mlflow.start_run(run_name='ddla_summary'):
        mlflow.log_param('experiment_type', 'ddla_summary')
        mlflow.log_param('n_ddlas_found', len(ddla_info['ddlas']))
        mlflow.log_param('baseline_ddla_fraction', ddla_info['ddla_fraction_baseline'])
        
        # Calculate summary statistics
        accuracy_rate = np.mean([r['ddla_correct_decision'] for r in results]) * 100
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score_ddla = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        mlflow.log_metric('ddla_accuracy_rate', accuracy_rate)
        mlflow.log_metric('ddla_precision', precision)
        mlflow.log_metric('ddla_recall', recall)
        mlflow.log_metric('ddla_f1_score', f1_score_ddla)
        
        mlflow.log_artifact(summary_plot_path, artifact_path='plots')
        
        print(f" DDLA Summary logged to MLflow")
        print(f"   - Decision accuracy: {accuracy_rate:.1f}%")
        print(f"   - Precision: {precision:.3f}")
        print(f"   - Recall: {recall:.3f}")
        print(f"   - F1-Score: {f1_score_ddla:.3f}")
    
    return summary_plot_path

def debug_ddla_results(ddla_info, results):

    print(" DDLA DEBUG ANALYSIS")
    print("="*50)
    
    print(f"Baseline DDLAs found: {len(ddla_info['ddlas'])}")
    print(f"Baseline DDLA ratio: {ddla_info['ddla_ratio_baseline']:.4f}")
    
    if len(ddla_info['ddlas']) > 0:
        print("\nTop 5 DDLAs:")
        for i, ddla in enumerate(ddla_info['ddlas'][:5]):
            print(f"  {i+1}. Leaf {ddla['leaf_id']}: {ddla['accuracy']:.3f} accuracy "
                  f"({ddla['sample_count']} samples)")
    
    print(f"\nDDLA Ratios across thresholds:")
    print(f"{'Threshold':<12} {'Baseline':<12} {'Serving':<12} {'Change':<12} {'% Change':<12}")
    print("-" * 60)
    
    for r in results:
        baseline_ratio = r.get('ratio_train', 0)
        serving_ratio = r.get('ratio_serving', 0) 
        change = serving_ratio - baseline_ratio
        pct_change = (change / baseline_ratio * 100) if baseline_ratio > 0 else 0
        
        print(f"{r['threshold']:<12.2f} {baseline_ratio:<12.4f} {serving_ratio:<12.4f} "
              f"{change:<12.4f} {pct_change:<12.1f}%")


In [10]:
# Run the DDLA experiment using your existing setup
ddla_results = run_ddla_drift_experiment(
    X=X, 
    y=y, 
    trained_pipeline=pipeline,  # Your trained pipeline
    drift_thresholds=[0.0, 0.25, 0.5, 0.75, 1.0],  # Same as your current experiment
    experiment_name="telco-ddla-drift-analysis",
    random_state=42
)

# Print summary comparison
print("\n" + "="*80)
print("DDLA APPROACH SUMMARY")
print("="*80)

correct_decisions = sum(r['ddla_correct_decision'] for r in ddla_results)
total_decisions = len(ddla_results)
accuracy_rate = (correct_decisions / total_decisions) * 100

print(f"Overall DDLA Decision Accuracy: {correct_decisions}/{total_decisions} ({accuracy_rate:.1f}%)")
print("\nDetailed Results:")
print(f"{'Threshold':<12} {'DDLA Says':<12} {'Actually':<12} {'Correct':<10} {'Acc Drop':<12}")
print("-" * 60)

for r in ddla_results:
    ddla_decision = "HARMFUL" if r['ddla_detected_harmful'] else "BENIGN"
    actual_need = "YES" if r['actually_needs_retraining'] else "NO"
    correct =  "YES" if r['ddla_correct_decision'] else "NO"
    
    print(f"{r['threshold']:<12.2f} {ddla_decision:<12} {actual_need:<12} {correct:<10} {r['accuracy_drop_pct']:<12.1f}%")


Starting DDLA Drift Experiment...
Testing thresholds: [0.0, 0.25, 0.5, 0.75, 1.0]

STEP 1: IDENTIFYING DDLAs ON BASELINE DATA
Identifying DDLAs with tree based approach
Overall model accuracy: 0.7935
  Overall incorrect prediction rate: 0.2065
  Best decision tree params: {'max_depth': 7, 'min_samples_leaf': 28}
  Decision tree F1 score: 0.4888
 Found 9 DDLAs out of 27 total leaf nodes
 DDLA coverage: 585/1409 samples (0.415)

STEP 2: TESTING DDLA APPROACH ACROSS DRIFT THRESHOLDS

 Testing drift threshold: 0.00
Simulating combined drift with threshold: 0.00
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 13 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
Detecting harmful drift
  Baseline DDLA fraction: 0.4152
  Serving DDLA fraction: 0.4145
 Drift assessment: BENIGN
  Reason: DDLA fraction decreased or stayed same


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  DDLA says: BENIGN
  Actually needs retraining: NO
  DDLA decision correct: YES
  Accuracy drop: -0.0014 (-0.2%)
  DDLA fraction: 0.415 → 0.414
🏃 View run ddla_threshold_0.0 at: http://localhost:5000/#/experiments/4/runs/3a3cad0d56f04c72af6152e526678e6d
🧪 View experiment at: http://localhost:5000/#/experiments/4

 Testing drift threshold: 0.25
Simulating combined drift with threshold: 0.25
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.322 (original: 0.265)
Detecting harmful drift
  Baseline DDLA fraction: 0.4152
  Serving DDLA fraction: 0.4315
 Drift assessment: BENIGN
  Reason: DDLA ratio increase 3.93% below threshold 50.0%


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  DDLA says: BENIGN
  Actually needs retraining: YES
  DDLA decision correct: NO
  Accuracy drop: 0.0546 (6.9%)
  DDLA fraction: 0.415 → 0.432
🏃 View run ddla_threshold_0.25 at: http://localhost:5000/#/experiments/4/runs/20eeb59a7c414b2fb4a2e985b4d18064
🧪 View experiment at: http://localhost:5000/#/experiments/4

 Testing drift threshold: 0.50
Simulating combined drift with threshold: 0.50
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.376 (original: 0.265)
Detecting harmful drift
  Baseline DDLA fraction: 0.4152
  Serving DDLA fraction: 0.4379
 Drift assessment: BENIGN
  Reason: DDLA ratio increase 5.47% below threshold 50.0%


...

  DDLA says: BENIGN
  Actually needs retraining: YES
  DDLA decision correct: NO
  Accuracy drop: 0.1015 (12.8%)
  DDLA fraction: 0.415 → 0.438
🏃 View run ddla_threshold_0.5 at: http://localhost:5000/#/experiments/4/runs/1ff6752c1fa14383be857811b036b331
🧪 View experiment at: http://localhost:5000/#/experiments/4

 Testing drift threshold: 0.75
Simulating combined drift with threshold: 0.75
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.430 (original: 0.265)
Detecting harmful drift
  Baseline DDLA fraction: 0.4152
  Serving DDLA fraction: 0.4258
 Drift assessment: BENIGN
  Reason: DDLA ratio increase 2.56% below threshold 50.0%


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  DDLA says: BENIGN
  Actually needs retraining: YES
  DDLA decision correct: NO
  Accuracy drop: 0.1490 (18.8%)
  DDLA fraction: 0.415 → 0.426
🏃 View run ddla_threshold_0.75 at: http://localhost:5000/#/experiments/4/runs/9744084e61e845b78b0df4c469bb2668
🧪 View experiment at: http://localhost:5000/#/experiments/4

 Testing drift threshold: 1.00
Simulating combined drift with threshold: 1.00
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.478 (original: 0.265)
Detecting harmful drift
  Baseline DDLA fraction: 0.4152
  Serving DDLA fraction: 0.4365
 Drift assessment: BENIGN
  Reason: DDLA ratio increase 5.13% below threshold 50.0%


d:\Anaconda\envs\RDS-Project\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


  DDLA says: BENIGN
  Actually needs retraining: YES
  DDLA decision correct: NO
  Accuracy drop: 0.1874 (23.6%)
  DDLA fraction: 0.415 → 0.436
🏃 View run ddla_threshold_1.0 at: http://localhost:5000/#/experiments/4/runs/e9f1b0c18ee44d9fbfc7a7793554bb29
🧪 View experiment at: http://localhost:5000/#/experiments/4

 Creating DDLA summary visualization
 DDLA Summary logged to MLflow
   - Decision accuracy: 20.0%
   - Precision: 0.000
   - Recall: 0.000
   - F1-Score: 0.000
🏃 View run ddla_summary at: http://localhost:5000/#/experiments/4/runs/280e105a43574bb2988c10c0d64bf033
🧪 View experiment at: http://localhost:5000/#/experiments/4

DDLA APPROACH SUMMARY
Overall DDLA Decision Accuracy: 1/5 (20.0%)

Detailed Results:
Threshold    DDLA Says    Actually     Correct    Acc Drop    
------------------------------------------------------------
0.00         BENIGN       NO           YES        -0.2        %
0.25         BENIGN       YES          NO         6.9         %
0.50         BENIGN    

### Results

Our drift simulation actively introduces both kinds of drift - covariate and concept drift. The DDLA method we incorporate was only designed for covariate drift, rather than a concept drift or both being introduced. DDLA fails - or rather breaks under scenarios when there are both kinds of drift present. When tested across different combined drift thresholds - the method appears to only correctly detect DDLA regions and classify a drift scenario as benign. Data drift does not always appear as only a covariate or concept type in machine learning pipelines, but are slowly introduced over time. But, it is rare that only one kind of drift will be present in data - which is not very realistic scenario to begin with. 

We further test this method using only covariate shift under different thresholds.

### Simulating DDLA under just covariate drift

In [11]:
# Deprecated cell
'''
def simulate_covariate_drift_only(X, y, drift_threshold=0.5, random_state=42):
    """
    Simulate ONLY covariate drift (feature distribution changes) without concept shifts.
    Perfect for testing DDLA under its intended conditions.
    
    Parameters:
    -----------
    X : pd.DataFrame
        Original feature dataframe (before preprocessing)
    y : pd.Series
        Original target labels (0/1) - UNCHANGED in covariate drift
    drift_threshold : float
        Controls the intensity of drift (0.0 to 1.0)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    X_drifted : pd.DataFrame
        Drifted feature dataframe (same relationships to y)
    y_unchanged : pd.Series
        Original target labels (unchanged by definition)
    drift_info : dict
        Information about covariate shifts applied
    """
    np.random.seed(random_state)
    X_drifted = X.copy()
    y_unchanged = y.copy()  # No concept shift!
    
    drift_info = {
        'covariate_shifts': [],
        'concept_shifts': [],  # Empty by design
        'threshold': drift_threshold,
        'drift_type': 'covariate_only'
    }
    
    # Identify numeric and categorical columns
    numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print(f" Simulating COVARIATE DRIFT ONLY with threshold: {drift_threshold:.2f}")
    print(f"Applying to {len(numeric_cols)} numeric and {len(categorical_cols)} categorical features")
    print("Note: Target labels remain unchanged (P(Y|X) preserved)")
    
    # ============================================
    # COVARIATE SHIFT: Changes to feature distributions ONLY
    # ============================================
    
    # 1. Numeric Feature Drifts (same as your original function)
    for i, col in enumerate(numeric_cols):
        if col not in X_drifted.columns:
            continue
        
        col_mean = X_drifted[col].mean()
        col_std = X_drifted[col].std()
        
        if pd.isna(col_mean) or pd.isna(col_std) or col_std == 0:
            continue
        
        # Apply different types of drift to different features
        drift_type = i % 4
        
        if drift_type == 0:  # Mean shift
            shift_amount = drift_threshold * col_mean * 0.3
            X_drifted[col] = X_drifted[col] + shift_amount
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'mean_shift',
                'amount': shift_amount
            })
            
        elif drift_type == 1:  # Variance increase
            noise = np.random.normal(0, drift_threshold * col_std * 0.5, len(X_drifted))
            X_drifted[col] = X_drifted[col] + noise
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'variance_increase',
                'noise_std': drift_threshold * col_std * 0.5
            })
            
        elif drift_type == 2:  # Multiplicative shift
            scale_factor = 1 + drift_threshold * 0.2 * np.random.choice([-1, 1])
            X_drifted[col] = X_drifted[col] * scale_factor
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'multiplicative_shift',
                'factor': scale_factor
            })
            
        else:  # Add outliers
            outlier_fraction = 0.1 * drift_threshold
            n_outliers = int(outlier_fraction * len(X_drifted))
            outlier_indices = np.random.choice(X_drifted.index, n_outliers, replace=False)
            outlier_multiplier = 3 + 2 * drift_threshold
            X_drifted.loc[outlier_indices, col] = X_drifted.loc[outlier_indices, col] * outlier_multiplier
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'outliers',
                'n_outliers': n_outliers
            })
    
    # Special handling for key Telco features
    if 'tenure' in X_drifted.columns:
        tenure_increase = drift_threshold * 5
        X_drifted['tenure'] = X_drifted['tenure'] + np.random.normal(tenure_increase, 2, len(X_drifted))
        X_drifted['tenure'] = X_drifted['tenure'].clip(lower=0)
        drift_info['covariate_shifts'].append({
            'feature': 'tenure',
            'type': 'market_shift',
            'increase_months': tenure_increase
        })
    
    if 'MonthlyCharges' in X_drifted.columns:
        inflation_rate = 1 + drift_threshold * 0.15
        X_drifted['MonthlyCharges'] = X_drifted['MonthlyCharges'] * inflation_rate
        drift_info['covariate_shifts'].append({
            'feature': 'MonthlyCharges',
            'type': 'inflation',
            'rate': inflation_rate
        })
    
    if 'TotalCharges' in X_drifted.columns:
        if 'tenure' in X_drifted.columns and 'MonthlyCharges' in X_drifted.columns:
            X_drifted['TotalCharges'] = X_drifted['tenure'] * X_drifted['MonthlyCharges'] * \
                                      (1 + np.random.normal(0, 0.1 * drift_threshold, len(X_drifted)))
            X_drifted['TotalCharges'] = X_drifted['TotalCharges'].clip(lower=0)
    
    # 2. Categorical Feature Drifts
    for col in categorical_cols[:min(5, len(categorical_cols))]:
        if col not in X_drifted.columns:
            continue
        
        unique_vals = X_drifted[col].unique()
        if len(unique_vals) < 2:
            continue
        
        # Shift probability distributions
        if col == 'InternetService' and 'Fiber optic' in unique_vals and 'DSL' in unique_vals:
            mask_fiber = X_drifted[col] == 'DSL'
            n_to_shift = int(len(X_drifted) * 0.2 * drift_threshold)
            shift_indices = np.random.choice(X_drifted[mask_fiber].index[:n_to_shift], 
                                           size=min(n_to_shift, mask_fiber.sum()), 
                                           replace=False)
            X_drifted.loc[shift_indices, col] = 'Fiber optic'
            drift_info['covariate_shifts'].append({
                'feature': col,
                'type': 'category_probability_shift',
                'shift': f'DSL -> Fiber optic ({len(shift_indices)} samples)'
            })
        
        elif len(unique_vals) >= 2:
            value_counts = X_drifted[col].value_counts()
            if len(value_counts) >= 2:
                most_common = value_counts.index[0]
                least_common = value_counts.index[-1]
                
                n_to_shift = int(len(X_drifted) * 0.15 * drift_threshold)
                mask = X_drifted[col] == most_common
                if mask.sum() > 0:
                    shift_indices = np.random.choice(X_drifted[mask].index, 
                                                   size=min(n_to_shift, mask.sum()), 
                                                   replace=False)
                    X_drifted.loc[shift_indices, col] = least_common
                    drift_info['covariate_shifts'].append({
                        'feature': col,
                        'type': 'category_distribution_shift',
                        'shift': f'{most_common} -> {least_common} ({len(shift_indices)} samples)'
                    })
    
    print(f"✓ Applied {len(drift_info['covariate_shifts'])} covariate shifts")
    print(f"✓ Applied {len(drift_info['concept_shifts'])} concept shifts (by design: 0)")
    print(f"Final churn rate: {y_unchanged.mean():.3f} (unchanged from original: {y.mean():.3f})")
    
    return X_drifted, y_unchanged, drift_info


def simulate_concept_drift_only(X, y, drift_threshold=0.5, random_state=42):
    """
    Simulate ONLY concept drift (relationship changes) without covariate shifts.
    Changes P(Y|X) while keeping P(X) the same.
    
    Parameters:
    -----------
    X : pd.DataFrame
        Original feature dataframe (UNCHANGED in concept drift)
    y : pd.Series
        Original target labels (0/1)
    drift_threshold : float
        Controls the intensity of drift (0.0 to 1.0)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    X_unchanged : pd.DataFrame
        Original feature dataframe (unchanged by definition)
    y_drifted : pd.Series
        Drifted target labels (new relationships)
    drift_info : dict
        Information about concept shifts applied
    """
    np.random.seed(random_state)
    X_unchanged = X.copy() 
    y_drifted = y.copy()
    
    drift_info = {
        'covariate_shifts': [], 
        'concept_shifts': [],
        'threshold': drift_threshold,
        'drift_type': 'concept_only'
    }
    
    print(f" Simulating CONCEPT DRIFT ONLY with threshold: {drift_threshold:.2f}")
    print("Note: Feature distributions remain unchanged (P(X) preserved)")
    print("Changing relationships between features and target (P(Y|X))")
    
    # ============================================
    # CONCEPT SHIFT: Changes to label relationships ONLY
    # ============================================
    
    # 1. Reverse relationship for high-value customers
    if 'MonthlyCharges' in X_unchanged.columns:
        high_charge_threshold = X_unchanged['MonthlyCharges'].quantile(0.75)
        high_charge_mask = X_unchanged['MonthlyCharges'] > high_charge_threshold
        
        n_to_flip = int(high_charge_mask.sum() * 0.3 * drift_threshold)
        flip_indices = np.random.choice(X_unchanged[high_charge_mask].index, 
                                      size=min(n_to_flip, high_charge_mask.sum()), 
                                      replace=False)
        y_drifted.loc[flip_indices] = 1 - y_drifted.loc[flip_indices]
        drift_info['concept_shifts'].append({
            'type': 'high_value_retention',
            'description': 'High MonthlyCharges customers now less likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 2. Change relationship with tenure
    if 'tenure' in X_unchanged.columns:
        long_tenure_threshold = X_unchanged['tenure'].quantile(0.8)
        long_tenure_mask = (X_unchanged['tenure'] > long_tenure_threshold) & (y_drifted == 0)
        
        n_to_flip = int(long_tenure_mask.sum() * 0.2 * drift_threshold)
        flip_indices = np.random.choice(X_unchanged[long_tenure_mask].index, 
                                      size=min(n_to_flip, long_tenure_mask.sum()), 
                                      replace=False)
        y_drifted.loc[flip_indices] = 1
        drift_info['concept_shifts'].append({
            'type': 'tenure_fatigue',
            'description': 'Very long tenure customers more likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 3. Change relationship with service engagement
    if 'service_engagement' in X_unchanged.columns:
        high_engagement_threshold = X_unchanged['service_engagement'].quantile(0.7)
        high_engagement_mask = (X_unchanged['service_engagement'] > high_engagement_threshold) & (y_drifted == 0)
        
        n_to_flip = int(high_engagement_mask.sum() * 0.25 * drift_threshold)
        flip_indices = np.random.choice(X_unchanged[high_engagement_mask].index, 
                                      size=min(n_to_flip, high_engagement_mask.sum()), 
                                      replace=False)
        y_drifted.loc[flip_indices] = 1
        drift_info['concept_shifts'].append({
            'type': 'service_overwhelm',
            'description': 'High service engagement customers more likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 4. Contract type relationship change
    if 'Contract' in X_unchanged.columns:
        two_year_mask = (X_unchanged['Contract'] == 'Two year') & (y_drifted == 0)
        n_to_flip = int(two_year_mask.sum() * 0.15 * drift_threshold)
        flip_indices = np.random.choice(X_unchanged[two_year_mask].index, 
                                      size=min(n_to_flip, two_year_mask.sum()), 
                                      replace=False)
        y_drifted.loc[flip_indices] = 1
        drift_info['concept_shifts'].append({
            'type': 'contract_regret',
            'description': 'Two year contract customers more likely to churn',
            'n_samples': len(flip_indices)
        })
    
    # 5. Overall base rate shift
    base_rate_shift = drift_threshold * 0.1
    if base_rate_shift > 0:
        current_churn_rate = y_drifted.mean()
        target_churn_rate = min(1.0, current_churn_rate + base_rate_shift)
        
        n_current_churn = y_drifted.sum()
        n_target_churn = int(len(y_drifted) * target_churn_rate)
        n_to_change = abs(n_target_churn - n_current_churn)
        
        if n_target_churn > n_current_churn:
            non_churners = X_unchanged[y_drifted == 0].index
            flip_indices = np.random.choice(non_churners, 
                                          size=min(n_to_change, len(non_churners)), 
                                          replace=False)
            y_drifted.loc[flip_indices] = 1
        else:
            churners = X_unchanged[y_drifted == 1].index
            flip_indices = np.random.choice(churners, 
                                          size=min(n_to_change, len(churners)), 
                                          replace=False)
            y_drifted.loc[flip_indices] = 0
        
        drift_info['concept_shifts'].append({
            'type': 'base_rate_shift',
            'description': f'Overall churn rate shifted from {current_churn_rate:.3f} to {target_churn_rate:.3f}',
            'shift_amount': base_rate_shift
        })
    
    print(f"✓ Applied {len(drift_info['covariate_shifts'])} covariate shifts (by design: 0)")
    print(f"✓ Applied {len(drift_info['concept_shifts'])} concept shifts")
    print(f"Final churn rate: {y_drifted.mean():.3f} (original: {y.mean():.3f})")
    
    return X_unchanged, y_drifted, drift_info


def simulate_selective_drift(X, y, drift_threshold=0.5, 
                           covariate_ratio=0.75, concept_ratio=0.25, 
                           random_state=42):
    """
    Simulate drift with customizable balance between covariate and concept shifts.
    This gives you full control over the type and intensity of drift.
    
    Parameters:
    -----------
    X : pd.DataFrame
        Original feature dataframe
    y : pd.Series
        Original target labels (0/1)
    drift_threshold : float
        Controls overall intensity of drift (0.0 to 1.0)
    covariate_ratio : float
        Fraction of drift intensity applied to covariate shifts (0.0 to 1.0)
    concept_ratio : float
        Fraction of drift intensity applied to concept shifts (0.0 to 1.0)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    X_drifted : pd.DataFrame
        Drifted feature dataframe
    y_drifted : pd.Series
        Drifted target labels
    drift_info : dict
        Information about all shifts applied
    """
    print(f" Simulating SELECTIVE DRIFT with threshold: {drift_threshold:.2f}")
    print(f"   Covariate intensity: {covariate_ratio:.2f} | Concept intensity: {concept_ratio:.2f}")
    
    # Start with original data
    X_result = X.copy()
    y_result = y.copy()
    
    combined_drift_info = {
        'covariate_shifts': [],
        'concept_shifts': [],
        'threshold': drift_threshold,
        'covariate_ratio': covariate_ratio,
        'concept_ratio': concept_ratio,
        'drift_type': 'selective'
    }
    
    # Apply covariate drift if requested
    if covariate_ratio > 0:
        covariate_threshold = drift_threshold * covariate_ratio
        X_result, _, cov_info = simulate_covariate_drift_only(
            X_result, y_result, 
            drift_threshold=covariate_threshold, 
            random_state=random_state
        )
        combined_drift_info['covariate_shifts'] = cov_info['covariate_shifts']
    
    # Apply concept drift if requested
    if concept_ratio > 0:
        concept_threshold = drift_threshold * concept_ratio
        _, y_result, con_info = simulate_concept_drift_only(
            X_result, y_result, 
            drift_threshold=concept_threshold, 
            random_state=random_state + 1  # Different seed
        )
        combined_drift_info['concept_shifts'] = con_info['concept_shifts']
    
    print(f"✓ Combined: {len(combined_drift_info['covariate_shifts'])} covariate + "
          f"{len(combined_drift_info['concept_shifts'])} concept shifts")
    print(f"Final churn rate: {y_result.mean():.3f} (original: {y.mean():.3f})")
    
    return X_result, y_result, combined_drift_info
'''

'\ndef simulate_covariate_drift_only(X, y, drift_threshold=0.5, random_state=42):\n    """\n    Simulate ONLY covariate drift (feature distribution changes) without concept shifts.\n    Perfect for testing DDLA under its intended conditions.\n\n    Parameters:\n    -----------\n    X : pd.DataFrame\n        Original feature dataframe (before preprocessing)\n    y : pd.Series\n        Original target labels (0/1) - UNCHANGED in covariate drift\n    drift_threshold : float\n        Controls the intensity of drift (0.0 to 1.0)\n    random_state : int\n        Random seed for reproducibility\n\n    Returns:\n    --------\n    X_drifted : pd.DataFrame\n        Drifted feature dataframe (same relationships to y)\n    y_unchanged : pd.Series\n        Original target labels (unchanged by definition)\n    drift_info : dict\n        Information about covariate shifts applied\n    """\n    np.random.seed(random_state)\n    X_drifted = X.copy()\n    y_unchanged = y.copy()  # No concept shift!\n\n 

In [12]:
def run_ddla_drift_comparison(X, y, trained_pipeline, drift_thresholds,
                              experiment_name="telco-ddla-drift-comparison",
                              random_state=42):  
    """
    Run DDLA experiments across different drift types to test the approach's sensitivity.
    
    This will test:
    1. Pure covariate drift (DDLA's intended scenario)
    2. Pure concept drift (DDLA's weakness)
    3. Combined drift (your original realistic scenario)
    """
    print(" Starting DDLA Drift Type Comparison Experiment")
    print(f"Testing thresholds: {drift_thresholds}")
    
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)
    
    # Baseline setup
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    # Identify DDLAs once on baseline
    print("\n" + "="*70)
    print("IDENTIFYING DDLAs ON BASELINE DATA")
    print("="*70)
    
    ddla_info = identify_ddlas_decision_tree(trained_pipeline, X_test, y_test, random_state=random_state)
    
    # Test different drift scenarios
    drift_scenarios = [
        {
            'name': 'covariate_only',
            'description': 'Pure Covariate Drift (DDLA\'s intended use case)',
            'function': simulate_covariate_drift_only
        },
        {
            'name': 'concept_only', 
            'description': 'Pure Concept Drift (DDLA\'s weakness)',
            'function': simulate_concept_drift_only
        },
        {
            'name': 'combined_drift',
            'description': 'Combined Drift (your original realistic scenario)',
            'function': simulate_drifted_data  # Your original function
        }
    ]
    
    all_results = {}
    
    for scenario in drift_scenarios:
        print(f"\n" + "="*70)
        print(f"TESTING: {scenario['description'].upper()}")
        print("="*70)
        
        scenario_results = []
        
        for threshold in drift_thresholds:
            print(f"\n {scenario['name']} - Threshold: {threshold:.2f}")
            
            with mlflow.start_run(run_name=f'{scenario["name"]}_threshold_{threshold}'):
                # Generate drift using appropriate function
                X_drifted, y_drifted, drift_info_scenario = scenario['function'](
                    X, y, drift_threshold=threshold, random_state=random_state
                )
                
                # Split drifted data
                _, X_test_drifted, _, y_test_drifted = train_test_split(
                    X_drifted, y_drifted, test_size=0.2, random_state=random_state
                )
                
                # Test DDLA detection
                drift_detection = detect_harmful_drift_ddla(
                    ddla_info, X_test_drifted, trained_pipeline
                )
                
                # Calculate actual performance
                y_pred_drifted = trained_pipeline.predict(X_test_drifted)
                actual_accuracy = accuracy_score(y_test_drifted, y_pred_drifted)
                accuracy_drop = ddla_info['overall_accuracy'] - actual_accuracy
                
                # Ground truth
                significant_degradation = accuracy_drop > 0.05
                ddla_correct = drift_detection['is_harmful_drift'] == significant_degradation
                
                # Store results
                result = {
                    'scenario': scenario['name'],
                    'threshold': threshold,
                    'ddla_detected_harmful': drift_detection['is_harmful_drift'],
                    'actually_needs_retraining': significant_degradation,
                    'ddla_correct': ddla_correct,
                    'accuracy_drop': accuracy_drop,
                    'accuracy_drop_pct': (accuracy_drop / ddla_info['overall_accuracy']) * 100,
                    'ratio_train': drift_detection['ratio_train'],
                    'ratio_serving': drift_detection['ratio_serving'],
                    'n_covariate_shifts': len(drift_info_scenario['covariate_shifts']),
                    'n_concept_shifts': len(drift_info_scenario['concept_shifts'])
                }
                
                scenario_results.append(result)
                
                # Log to MLflow
                mlflow.log_param('drift_scenario', scenario['name'])
                mlflow.log_param('drift_threshold', threshold)
                mlflow.log_param('ddla_approach', 'authentic_dong_2024')
                
                # Log key metrics
                mlflow.log_metric('ddla_detected_harmful', 1 if drift_detection['is_harmful_drift'] else 0)
                mlflow.log_metric('actually_needs_retraining', 1 if significant_degradation else 0)
                mlflow.log_metric('ddla_correct', 1 if ddla_correct else 0)
                mlflow.log_metric('accuracy_drop_pct', result['accuracy_drop_pct'])
                mlflow.log_metric('ratio_train', drift_detection['ratio_train'])
                mlflow.log_metric('ratio_serving', drift_detection['ratio_serving'])
                
                # Print results
                print(f"  DDLA says: {'HARMFUL' if drift_detection['is_harmful_drift'] else 'BENIGN'}")
                print(f"  Actually needs retraining: {'YES' if significant_degradation else 'NO'}")
                print(f"  DDLA correct: {'YES' if ddla_correct else 'NO'}")
                print(f"  Accuracy drop: {accuracy_drop:.4f} ({result['accuracy_drop_pct']:.1f}%)")
        
        all_results[scenario['name']] = scenario_results
    
    # Create comprehensive comparison visualization
    create_drift_comparison_visualization(all_results, experiment_name)
    
    return all_results


def create_drift_comparison_visualization(all_results, experiment_name):
    """
    Create comprehensive visualization comparing DDLA performance across drift types.
    """
    print("\n Creating Drift Type Comparison Visualizations...")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('DDLA Performance Across Different Drift Types', fontsize=16, fontweight='bold')
    
    scenarios = list(all_results.keys())
    colors = {'covariate_only': '#2ecc71', 'concept_only': '#e74c3c', 'combined_drift': '#f39c12'}
    
    # 1. Accuracy Rate by Scenario
    ax1 = axes[0, 0]
    scenario_accuracies = []
    scenario_names = []
    
    for scenario in scenarios:
        results = all_results[scenario]
        correct_decisions = [r['ddla_correct'] for r in results]
        accuracy_rate = np.mean(correct_decisions) * 100
        scenario_accuracies.append(accuracy_rate)
        scenario_names.append(scenario.replace('_', ' ').title())
    
    bars = ax1.bar(scenario_names, scenario_accuracies, color=[colors[s] for s in scenarios], alpha=0.8)
    ax1.set_ylabel('DDLA Accuracy Rate (%)')
    ax1.set_title('DDLA Decision Accuracy by Drift Type')
    ax1.set_ylim([0, 100])
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, accuracy in zip(bars, scenario_accuracies):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 2,
                f'{accuracy:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # 2. DDLA Accuracy vs Drift Threshold
    ax2 = axes[0, 1]
    for scenario in scenarios:
        results = all_results[scenario]
        thresholds = [r['threshold'] for r in results]
        accuracies = [r['ddla_correct'] for r in results]
        ax2.plot(thresholds, accuracies, 'o-', label=scenario.replace('_', ' ').title(), 
                color=colors[scenario], linewidth=2, markersize=6)
    
    ax2.set_xlabel('Drift Threshold')
    ax2.set_ylabel('DDLA Correct Decision (1=Yes, 0=No)')
    ax2.set_title('DDLA Accuracy Across Thresholds')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Actual Performance Drop vs DDLA Detection
    ax3 = axes[1, 0]
    for scenario in scenarios:
        results = all_results[scenario]
        performance_drops = [r['accuracy_drop_pct'] for r in results]
        ddla_detections = [1 if r['ddla_detected_harmful'] else 0 for r in results]
        
        # Scatter plot with jitter for visibility
        x_jitter = np.array(ddla_detections) + np.random.normal(0, 0.05, len(ddla_detections))
        ax3.scatter(x_jitter, performance_drops, label=scenario.replace('_', ' ').title(),
                   color=colors[scenario], alpha=0.7, s=60)
    
    ax3.axhline(y=5, color='orange', linestyle='--', label='5% significance threshold')
    ax3.set_xlabel('DDLA Detection (0=Benign, 1=Harmful)')
    ax3.set_ylabel('Actual Performance Drop (%)')
    ax3.set_title('Performance Drop vs DDLA Prediction')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Confusion Matrix Heatmap
    ax4 = axes[1, 1]
    
    # Calculate confusion matrices for each scenario
    confusion_data = np.zeros((len(scenarios), 4))  # [TP, TN, FP, FN]
    
    for i, scenario in enumerate(scenarios):
        results = all_results[scenario]
        tp = sum(1 for r in results if r['ddla_detected_harmful'] and r['actually_needs_retraining'])
        tn = sum(1 for r in results if not r['ddla_detected_harmful'] and not r['actually_needs_retraining'])
        fp = sum(1 for r in results if r['ddla_detected_harmful'] and not r['actually_needs_retraining'])
        fn = sum(1 for r in results if not r['ddla_detected_harmful'] and r['actually_needs_retraining'])
        
        total = tp + tn + fp + fn
        if total > 0:
            confusion_data[i] = [tp/total, tn/total, fp/total, fn/total]
    
    # Create stacked bar chart for confusion matrix
    bottom = np.zeros(len(scenarios))
    metrics = ['True Positive', 'True Negative', 'False Positive', 'False Negative']
    metric_colors = ['#27ae60', '#2ecc71', '#e74c3c', '#c0392b']
    
    for j, metric in enumerate(metrics):
        ax4.bar(scenario_names, confusion_data[:, j], bottom=bottom, 
               label=metric, color=metric_colors[j], alpha=0.8)
        bottom += confusion_data[:, j]
    
    ax4.set_ylabel('Proportion')
    ax4.set_title('DDLA Decision Distribution by Drift Type')
    ax4.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.set_ylim([0, 1])
    
    plt.tight_layout()
    
    # Save and log
    comparison_plot_path = f'ddla_drift_comparison_{experiment_name.replace("-", "_")}.png'
    plt.savefig(comparison_plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Log summary to MLflow
    with mlflow.start_run(run_name='drift_comparison_summary'):
        mlflow.log_param('experiment_type', 'drift_comparison_summary')
        mlflow.log_param('scenarios_tested', len(scenarios))
        
        # Calculate summary statistics
        for scenario in scenarios:
            results = all_results[scenario]
            accuracy_rate = np.mean([r['ddla_correct'] for r in results]) * 100
            mlflow.log_metric(f'{scenario}_accuracy_rate', accuracy_rate)
        
        mlflow.log_artifact(comparison_plot_path, artifact_path='plots')
        
        print(f"Drift Comparison Summary logged to MLflow")
        
        # Print summary
        print(f"\n" + "="*80)
        print("DDLA PERFORMANCE SUMMARY BY DRIFT TYPE")
        print("="*80)
        for scenario in scenarios:
            results = all_results[scenario]
            accuracy_rate = np.mean([r['ddla_correct'] for r in results]) * 100
            print(f"{scenario.replace('_', ' ').title():<25}: {accuracy_rate:.1f}% accuracy")
    
    return comparison_plot_path

In [13]:
# ==============================================================
# DDLA EXPERIMENT WITH DIFFERENT DRIFT TYPES
# ==============================================================

# Test DDLA performance across different drift types
drift_comparison_results = run_ddla_drift_comparison(
    X=X,
    y=y,
    trained_pipeline=pipeline,
    drift_thresholds=[0.0, 0.25, 0.5, 0.75, 1.0],
    experiment_name="telco-ddla-drift-type-comparison",
    random_state=42
)

# Print comprehensive comparison
print("\n" + "="*80)
print("COMPREHENSIVE DDLA DRIFT TYPE ANALYSIS")
print("="*80)

for drift_type, results in drift_comparison_results.items():
    correct_predictions = sum(r['ddla_correct'] for r in results)
    total_predictions = len(results)
    accuracy_rate = (correct_predictions / total_predictions) * 100
    
    print(f"\n{drift_type.replace('_', ' ').title()} Results:")
    print(f"  Overall Accuracy: {correct_predictions}/{total_predictions} ({accuracy_rate:.1f}%)")
    
    print(f"  {'Threshold':<12} {'DDLA Says':<12} {'Actually':<12} {'Correct':<10} {'Acc Drop':<12}")
    print("  " + "-" * 60)
    
    for r in results:
        ddla_decision = "HARMFUL" if r['ddla_detected_harmful'] else "BENIGN"
        actual_need = "YES" if r['actually_needs_retraining'] else "NO"
        correct =  "YES" if r['ddla_correct'] else "NO"
        
        print(f"  {r['threshold']:<12.2f} {ddla_decision:<12} {actual_need:<12} "
              f"{correct:<10} {r['accuracy_drop_pct']:<12.1f}%")

# You can also test specific scenarios individually:

# Test ONLY covariate drift (DDLA's intended scenario)
print("\n" + "="*60)
print("TESTING PURE COVARIATE DRIFT")
print("="*60)

X_cov_drifted, y_cov_unchanged, cov_drift_info = simulate_covariate_drift_only(
    X, y, drift_threshold=0.5, random_state=42
)

# Test ONLY concept drift (where DDLA should fail)
print("\n" + "="*60)
print("TESTING PURE CONCEPT DRIFT") 
print("="*60)

X_con_unchanged, y_con_drifted, con_drift_info = simulate_concept_drift_only(
    X, y, drift_threshold=0.5, random_state=42
)

# Test custom balance (75% covariate, 25% concept)
print("\n" + "="*60)
print("TESTING CUSTOM BALANCED DRIFT")
print("="*60)

X_selective, y_selective, selective_drift_info = simulate_selective_drift(
    X, y, drift_threshold=0.5, 
    covariate_ratio=0.75, concept_ratio=0.25, 
    random_state=42
)

 Starting DDLA Drift Type Comparison Experiment
Testing thresholds: [0.0, 0.25, 0.5, 0.75, 1.0]

IDENTIFYING DDLAs ON BASELINE DATA
Identifying DDLAs with tree based approach
Overall model accuracy: 0.7935
  Overall incorrect prediction rate: 0.2065
  Best decision tree params: {'max_depth': 7, 'min_samples_leaf': 28}
  Decision tree F1 score: 0.4888
 Found 9 DDLAs out of 27 total leaf nodes
 DDLA coverage: 585/1409 samples (0.415)

TESTING: PURE COVARIATE DRIFT (DDLA'S INTENDED USE CASE)

 covariate_only - Threshold: 0.00
Simulating covariate drift with threshold: 0.00
Applying to 8 numeric and 18 categorical features
Applied 13 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
Detecting harmful drift
  Baseline DDLA fraction: 0.4152
  Serving DDLA fraction: 0.4145
 Drift assessment: BENIGN
  Reason: DDLA fraction decreased or stayed same
  DDLA says: BENIGN
  Actually needs retraining: NO
  DDLA correct: YES
  Accuracy drop: -0.0014 (-0.2%)
🏃 View ru

### Results

DDLAs are able to accurately determine low accuracy regions for covariate drifts only - they start to fail when subjected to concept and combined drift scenarios. The accuracy drop of the DDLAs show us how they perform when they are subject to different drift scenarios. This essentially means that this method is limited in viability when it comes to production systems. Naturally, this is natural given that the authors explicitly state this in their Limitations and Future Work sections - where they contemplate on using Explanation Tables and other methods for a similar approach to mitigating drift. 

This gives rise to a potential research avenue:

- What if we use clustering instead of a single decision tree to identify low accuracy areas?

Literature suggests that clustering based methodology has been used in this context for multiple use cases. For example, [Mishara & Stamp (2025)](https://arxiv.org/abs/2502.14135) use a clustering based approach (K-Means) to detect concept drift and a specific threshold to trigger model retraining, and a silhouette score which exerts less strain on available compute compared to other methods vastly due to the reduced number of times retraining is triggered. Similarly, [Razaei & Sajedi (2025)](https://link.springer.com/article/10.1007/s10115-025-02484-5) use a "fractal-dimension" stream clustering algorithm to detect concept drift and pattern recurrence to address data streaming challenges with respect to sensitivity to concept drift, compute efficiency, and adaptability. [Yu et al. (2021)](https://arxiv.org/pdf/2105.01419) propose a meta learning model for drift detection that is capable of the detection of all kinds of drifts within data streams and tabular data. 

Applying a clustering based DDLA approach could help us better understand how unsupervised frameworks are able to deal with low accruracy regions to advise retrianing when no active learning options are available.

## DDLA-Clustering approach

In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def find_optimal_error_clusters(X_errors_preprocessed, n_clusters_range=(3, 12), random_state=42):
    """
    Find optimal number of clusters for error patterns using multiple metrics.
    This is specifically designed for clustering model failures.
    """
    if len(X_errors_preprocessed) < 6:  # Need at least 6 samples for meaningful clustering
        print(f"    Too few error samples ({len(X_errors_preprocessed)}) for clustering")
        return None, 0
    
    best_score = -1
    best_k = n_clusters_range[0]
    best_kmeans = None
    scoring_results = []
    
    print(f"   Finding optimal clusters for {len(X_errors_preprocessed)} error samples...")
    
    for k in range(n_clusters_range[0], min(n_clusters_range[1] + 1, len(X_errors_preprocessed))):
        try:
            kmeans = KMeans(n_clusters=k, random_state=random_state, n_init=10)
            cluster_labels = kmeans.fit_predict(X_errors_preprocessed)
            
            # Skip if all samples in one cluster
            if len(np.unique(cluster_labels)) < 2:
                continue
                
            # Calculate multiple clustering quality metrics
            silhouette = silhouette_score(X_errors_preprocessed, cluster_labels)
            calinski_harabasz = calinski_harabasz_score(X_errors_preprocessed, cluster_labels)
            
            # Combined score (weighted)
            combined_score = 0.7 * silhouette + 0.3 * (calinski_harabasz / 1000)  # Normalize CH score
            
            scoring_results.append({
                'k': k,
                'silhouette': silhouette,
                'calinski_harabasz': calinski_harabasz,
                'combined_score': combined_score
            })
            
            if combined_score > best_score:
                best_score = combined_score
                best_k = k
                best_kmeans = kmeans
                
        except Exception as e:
            print(f"    Error with k={k}: {e}")
            continue
    
    if best_kmeans is None:
        print(f"Could not find valid clustering")
        return None, 0
    
    print(f"Optimal clusters: {best_k} (score: {best_score:.3f})")
    
    return best_kmeans, best_k


def identify_ddlas_error_clustering(trained_pipeline, X_test, y_test, 
                                   n_clusters_range=(3, 12), random_state=42):
    """
    This revolutionary approach:
    1. Focuses ONLY on incorrectly predicted samples
    2. Clusters these error samples to find failure patterns  
    3. Maps all data to these error-derived clusters
    4. Identifies which clusters represent DDLAs
    
    This should be more robust to concept drift than decision trees!
    """
    print("Identifying DDLAs using Error-Driven Clustering...")
    print("This is the world's first implementation of this approach! 🌟")
    
    # Step 1: Get model predictions and overall accuracy
    y_pred = trained_pipeline.predict(X_test)
    y_prob = trained_pipeline.predict_proba(X_test)
    overall_accuracy = accuracy_score(y_test, y_pred)
    overall_error_rate = 1 - overall_accuracy
    
    print(f"  Overall model accuracy: {overall_accuracy:.4f}")
    print(f"  Overall error rate: {overall_error_rate:.4f}")
    
    # Step 2: Focus ONLY on model errors
    error_mask = (y_pred != y_test)
    X_errors = X_test[error_mask].copy()
    y_errors_true = y_test[error_mask]
    y_errors_pred = y_pred[error_mask]
    y_errors_prob = y_prob[error_mask]
    
    print(f"   Focusing on {len(X_errors)} error samples out of {len(X_test)} total")
    
    if len(X_errors) < 6:
        print("    Too few errors for meaningful clustering. Returning empty DDLAs.")
        return {
            'ddlas': [],
            'error_clusters': None, 
            'overall_accuracy': overall_accuracy,
            'overall_error_rate': overall_error_rate,
            'ddla_ratio_baseline': 0.0,
            'error_sample_count': len(X_errors),
            'total_sample_count': len(X_test),
            'feature_names': [],
            'approach': 'error_driven_clustering'
        }
    
    # Step 3: Preprocess error samples in the SAME space the model sees
    X_errors_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_errors)
    
    # Get feature names for interpretability
    try:
        feature_names = trained_pipeline.named_steps['preprocessor'].get_feature_names_out()
    except:
        n_features = X_errors_preprocessed.shape[1]
        feature_names = [f'feature_{i}' for i in range(n_features)]
    
    print(f"   Error samples have {len(feature_names)} features after preprocessing")
    
    # Step 4: Find optimal clustering of ERROR PATTERNS
    error_kmeans, optimal_k = find_optimal_error_clusters(
        X_errors_preprocessed, n_clusters_range, random_state
    )
    
    if error_kmeans is None:
        print("Could not cluster error patterns. Returning empty DDLAs.")
        return {
            'ddlas': [],
            'error_clusters': None,
            'overall_accuracy': overall_accuracy,
            'overall_error_rate': overall_error_rate,
            'ddla_ratio_baseline': 0.0,
            'error_sample_count': len(X_errors),
            'total_sample_count': len(X_test),
            'feature_names': feature_names,
            'approach': 'error_driven_clustering'
        }
    
    # Step 5: Assign ERROR cluster labels
    error_cluster_labels = error_kmeans.predict(X_errors_preprocessed)
    
    print(f"   Found {optimal_k} distinct error patterns")
    
    # Step 6: Map ALL data to these error-derived clusters
    X_all_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_test)
    all_cluster_labels = error_kmeans.predict(X_all_preprocessed)
    
    # Step 7: Analyze each cluster to identify DDLAs
    ddlas = []
    cluster_info = {}
    total_ddla_samples = 0
    
    for cluster_id in range(optimal_k):
        # Get all samples assigned to this cluster
        cluster_mask = (all_cluster_labels == cluster_id)
        cluster_indices = np.where(cluster_mask)[0]
        
        if len(cluster_indices) == 0:
            continue
        
        # Calculate cluster accuracy using ALL samples in cluster
        cluster_y_true = y_test.iloc[cluster_indices]
        cluster_y_pred = y_pred[cluster_indices]
        cluster_accuracy = accuracy_score(cluster_y_true, cluster_y_pred)
        cluster_error_rate = 1 - cluster_accuracy
        
        # How many of the original error samples are in this cluster?
        error_samples_in_cluster = sum(1 for idx in cluster_indices if error_mask.iloc[idx])
        
        # Cluster characteristics for interpretability
        cluster_data = X_test.iloc[cluster_indices]
        cluster_size = len(cluster_indices)
        cluster_fraction = cluster_size / len(X_test)
        
        cluster_info[cluster_id] = {
            'cluster_id': cluster_id,
            'accuracy': cluster_accuracy,
            'error_rate': cluster_error_rate,
            'sample_count': cluster_size,
            'sample_fraction': cluster_fraction,
            'error_samples_in_cluster': error_samples_in_cluster,
            'error_concentration': error_samples_in_cluster / len(X_errors) if len(X_errors) > 0 else 0,
            'sample_indices': cluster_indices.tolist(),
            'is_ddla': cluster_accuracy < overall_accuracy  # DDLA definition
        }
        
        # This is a DDLA if accuracy < overall accuracy
        if cluster_accuracy < overall_accuracy:
            ddlas.append(cluster_info[cluster_id])
            total_ddla_samples += cluster_size
            
            print(f"     DDLA found: Cluster {cluster_id}")
            print(f"       Accuracy: {cluster_accuracy:.3f} (vs overall {overall_accuracy:.3f})")
            print(f"       Size: {cluster_size} samples ({cluster_fraction:.3f} of total)")
            print(f"       Error concentration: {error_samples_in_cluster}/{len(X_errors)} " + 
                  f"({cluster_info[cluster_id]['error_concentration']:.3f})")
    
    # Sort DDLAs by error rate (most problematic first)
    ddlas.sort(key=lambda x: x['error_rate'], reverse=True)
    
    # Calculate baseline DDLA ratio
    ddla_ratio_baseline = total_ddla_samples / len(X_test)
    
    print(f" Found {len(ddlas)} DDLAs covering {total_ddla_samples}/{len(X_test)} " +
          f"samples ({ddla_ratio_baseline:.3f} ratio)")
    
    # Step 8: Generate interpretable cluster characterizations
    cluster_characterizations = characterize_error_clusters(
        cluster_info, X_test, y_test, y_pred, y_prob, error_kmeans, X_all_preprocessed
    )
    
    return {
        'ddlas': ddlas,
        'error_clusters': error_kmeans,
        'overall_accuracy': overall_accuracy,
        'overall_error_rate': overall_error_rate,
        'ddla_ratio_baseline': ddla_ratio_baseline,
        'total_ddla_samples': total_ddla_samples,
        'cluster_info': cluster_info,
        'cluster_characterizations': cluster_characterizations,
        'error_sample_count': len(X_errors),
        'total_sample_count': len(X_test),
        'feature_names': feature_names,
        'optimal_k': optimal_k,
        'approach': 'error_driven_clustering'
    }


def characterize_error_clusters(cluster_info, X_test, y_test, y_pred, y_prob, 
                               error_kmeans, X_all_preprocessed):
    """
    Generate interpretable characterizations of error clusters.
    This is the INTERPRETABILITY INNOVATION part!
    """
    print("Generating interpretable cluster characterizations...")
    
    characterizations = {}
    
    for cluster_id, info in cluster_info.items():
        cluster_indices = info['sample_indices']
        cluster_data = X_test.iloc[cluster_indices]
        cluster_y_true = y_test.iloc[cluster_indices]
        cluster_y_pred = y_pred[cluster_indices]
        cluster_y_prob = y_prob[cluster_indices]
        
        # Feature profile: what makes this cluster unique?
        feature_profile = {}
        numeric_cols = X_test.select_dtypes(include=[np.number]).columns
        
        for col in numeric_cols:
            if col in cluster_data.columns:
                cluster_mean = cluster_data[col].mean()
                global_mean = X_test[col].mean()
                feature_profile[col] = {
                    'cluster_mean': cluster_mean,
                    'global_mean': global_mean,
                    'difference': cluster_mean - global_mean,
                    'relative_difference': ((cluster_mean - global_mean) / global_mean) * 100 if global_mean != 0 else 0
                }
        
        # Error characteristics
        error_profile = {
            'accuracy': info['accuracy'],
            'error_rate': info['error_rate'],
            'avg_prediction_confidence': np.mean(np.max(cluster_y_prob, axis=1)),
            'prediction_distribution': pd.Series(cluster_y_pred).value_counts(normalize=True).to_dict(),
            'true_label_distribution': pd.Series(cluster_y_true).value_counts(normalize=True).to_dict()
        }
        
        # Generate interpretable description
        interpretation = generate_cluster_interpretation(
            cluster_data, cluster_y_true, cluster_y_pred, feature_profile, error_profile
        )
        
        characterizations[cluster_id] = {
            'cluster_id': cluster_id,
            'size': len(cluster_indices),
            'is_ddla': info['is_ddla'],
            'feature_profile': feature_profile,
            'error_profile': error_profile,
            'interpretation': interpretation
        }
    
    return characterizations


def generate_cluster_interpretation(cluster_data, cluster_y_true, cluster_y_pred, 
                                  feature_profile, error_profile):
    """
    Generate human-readable interpretation of what each cluster represents.
    """
    interpretation = {
        'cluster_type': '',
        'key_characteristics': [],
        'common_errors': '',
        'business_meaning': ''
    }
    
    # Identify key distinguishing features (top 3 largest relative differences)
    feature_diffs = [(col, abs(profile['relative_difference'])) 
                     for col, profile in feature_profile.items()]
    feature_diffs.sort(key=lambda x: x[1], reverse=True)
    
    top_features = feature_diffs[:3]
    
    for feature, _ in top_features:
        profile = feature_profile[feature]
        if abs(profile['relative_difference']) > 10:  # More than 10% difference
            direction = "higher" if profile['relative_difference'] > 0 else "lower"
            interpretation['key_characteristics'].append(
                f"{direction} {feature} ({profile['relative_difference']:+.1f}%)"
            )
    
    # Determine cluster type based on accuracy
    if error_profile['accuracy'] < 0.5:
        interpretation['cluster_type'] = 'High-Risk Failure Pattern'
    elif error_profile['accuracy'] < 0.7:
        interpretation['cluster_type'] = 'Moderate-Risk Pattern'
    else:
        interpretation['cluster_type'] = 'Low-Risk Pattern'
    
    # Common error pattern
    pred_dist = error_profile['prediction_distribution']
    true_dist = error_profile['true_label_distribution']
    
    if len(pred_dist) > 0 and len(true_dist) > 0:
        most_predicted = max(pred_dist.keys(), key=pred_dist.get)
        most_actual = max(true_dist.keys(), key=true_dist.get)
        
        if most_predicted != most_actual:
            interpretation['common_errors'] = f"Often predicts {most_predicted} when actual is {most_actual}"
        
    # Business meaning (Telco-specific)
    interpretation['business_meaning'] = generate_telco_business_meaning(
        interpretation['key_characteristics'], error_profile
    )
    
    return interpretation


def generate_telco_business_meaning(key_characteristics, error_profile):
    """
    Generate business-relevant interpretation for Telco context.
    """
    meaning_pieces = []
    
    for char in key_characteristics:
        if 'tenure' in char.lower():
            if 'higher' in char:
                meaning_pieces.append("long-term customers")
            else:
                meaning_pieces.append("new customers")
        elif 'monthlycharges' in char.lower():
            if 'higher' in char:
                meaning_pieces.append("high-value customers")
            else:
                meaning_pieces.append("budget customers")
        elif 'totalcharges' in char.lower():
            if 'higher' in char:
                meaning_pieces.append("high lifetime value")
            else:
                meaning_pieces.append("low lifetime value")
    
    if not meaning_pieces:
        meaning_pieces.append("customers with mixed characteristics")
    
    accuracy = error_profile['accuracy']
    if accuracy < 0.5:
        risk_level = "very difficult to predict correctly"
    elif accuracy < 0.7:
        risk_level = "challenging to predict"
    else:
        risk_level = "relatively predictable"
    
    return f"Represents {' and '.join(meaning_pieces)} who are {risk_level}"


def detect_harmful_drift_error_clustering(ddla_info, X_serving, trained_pipeline,
                                          theta_inc=0.5, theta_ddla=0.1):
    """
    Detect harmful drift using error-driven clustering approach.
    This should be more robust to concept drift than decision trees!
    """
    print(" Detecting harmful drift using Error-Driven Clustering...")
    
    error_clusters = ddla_info['error_clusters']
    baseline_ddla_ratio = ddla_info['ddla_ratio_baseline']
    
    if error_clusters is None:
        print("    No error clusters available. Assuming benign drift.")
        return {
            'is_harmful_drift': False,
            'drift_type': 'benign',
            'reason': 'No baseline error clusters to compare against',
            'ratio_train': baseline_ddla_ratio,
            'ratio_serving': 0.0,
            'approach': 'error_driven_clustering'
        }
    
    # Preprocess serving data
    X_serving_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_serving)
    
    # Assign serving data to error-derived clusters
    serving_cluster_labels = error_clusters.predict(X_serving_preprocessed)
    
    # Get DDLA cluster IDs
    ddla_cluster_ids = {ddla['cluster_id'] for ddla in ddla_info['ddlas']}
    
    # Calculate serving DDLA ratio
    serving_ddla_count = sum(1 for cluster_id in serving_cluster_labels 
                           if cluster_id in ddla_cluster_ids)
    serving_ddla_ratio = serving_ddla_count / len(X_serving)
    
    print(f"  Baseline DDLA ratio: {baseline_ddla_ratio:.4f}")
    print(f"  Serving DDLA ratio: {serving_ddla_ratio:.4f}")
    
    # Apply drift detection logic (same as original DDLA)
    if serving_ddla_ratio <= baseline_ddla_ratio:
        is_harmful = False
        drift_type = "benign"
        reason = "DDLA ratio decreased or stayed same"
    else:
        if baseline_ddla_ratio > 0:
            ratio_increase = (serving_ddla_ratio - baseline_ddla_ratio) / baseline_ddla_ratio
        else:
            ratio_increase = float('inf') if serving_ddla_ratio > 0 else 0
        
        is_harmful = (ratio_increase > theta_inc) and (serving_ddla_ratio > theta_ddla)
        
        if is_harmful:
            drift_type = "harmful"
            reason = f"DDLA ratio increased by {ratio_increase:.2%} and exceeds thresholds"
        else:
            drift_type = "benign"
            reason = f"DDLA ratio increase {ratio_increase:.2%} below threshold or serving ratio too low"
    
    print(f"   Drift assessment: {drift_type.upper()}")
    print(f"  Reason: {reason}")
    
    return {
        'is_harmful_drift': is_harmful,
        'drift_type': drift_type,
        'reason': reason,
        'baseline_ddla_ratio': baseline_ddla_ratio,
        'serving_ddla_ratio': serving_ddla_ratio,
        'ratio_train': baseline_ddla_ratio,  # For compatibility
        'ratio_serving': serving_ddla_ratio,  # For compatibility
        'ddla_fraction_change': serving_ddla_ratio - baseline_ddla_ratio,
        'ddla_fraction_change_pct': ((serving_ddla_ratio - baseline_ddla_ratio) / baseline_ddla_ratio * 100) if baseline_ddla_ratio > 0 else 0,
        'serving_ddla_count': serving_ddla_count,
        'serving_total_count': len(X_serving),
        'approach': 'error_driven_clustering',
        'thresholds_used': {'theta_inc': theta_inc, 'theta_ddla': theta_ddla}
    }


def run_error_clustering_ddla_experiment(X, y, trained_pipeline, drift_thresholds,
                                        experiment_name="telco-error-clustering-ddla",
                                        random_state=42):
    """
    Run the world's first Error-Driven DDLA Clustering experiment!
    """
    print(" Starting Error-Driven DDLA Clustering Experiment!")
    print("This is pioneering research in drift detection! 🌟")
    print(f"Testing thresholds: {drift_thresholds}")
    
    # Setup MLflow
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)
    
    # Split data for baseline
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    # Step 1: Identify DDLAs using our error clustering approach
    print("\n" + "="*70)
    print("STEP 1: IDENTIFYING DDLAs USING ERROR-DRIVEN CLUSTERING")
    print("="*70)
    
    ddla_info = identify_ddlas_error_clustering(
        trained_pipeline, X_test, y_test, random_state=random_state
    )
    
    if len(ddla_info['ddlas']) == 0:
        print("  No DDLAs found with error clustering. This might indicate:")
        print("   - Very robust model with consistent error patterns")
        print("   - Need to adjust clustering parameters")
        print("   - Different approach needed for this dataset")
    
    results = []
    
    # Step 2: Test across drift scenarios
    print("\n" + "="*70)
    print("STEP 2: TESTING ERROR-CLUSTERING DDLA ACROSS DRIFT TYPES")
    print("="*70)
    
    # Test on all drift scenarios
    drift_scenarios = [
        ('covariate_only', simulate_covariate_drift_only),
        ('concept_only', simulate_concept_drift_only), 
        ('combined_drift', simulate_drifted_data)
    ]
    
    all_scenario_results = {}
    
    for scenario_name, drift_function in drift_scenarios:
        print(f"\n Testing {scenario_name.replace('_', ' ').title()} Scenario")
        scenario_results = []
        
        for threshold in drift_thresholds:
            print(f"\n  Threshold: {threshold:.2f}")
            
            with mlflow.start_run(run_name=f'error_clustering_{scenario_name}_threshold_{threshold}'):
                # Generate drift
                X_drifted, y_drifted, drift_info_scenario = drift_function(
                    X, y, drift_threshold=threshold, random_state=random_state
                )
                
                # Split drifted data
                _, X_test_drifted, _, y_test_drifted = train_test_split(
                    X_drifted, y_drifted, test_size=0.2, random_state=random_state
                )
                
                # Test our error clustering drift detection
                drift_detection = detect_harmful_drift_error_clustering(
                    ddla_info, X_test_drifted, trained_pipeline
                )
                
                # Calculate actual performance
                y_pred_drifted = trained_pipeline.predict(X_test_drifted)
                actual_accuracy = accuracy_score(y_test_drifted, y_pred_drifted)
                accuracy_drop = ddla_info['overall_accuracy'] - actual_accuracy
                significant_degradation = accuracy_drop > 0.05
                
                # Evaluate our approach
                error_clustering_correct = drift_detection['is_harmful_drift'] == significant_degradation
                
                result = {
                    'scenario': scenario_name,
                    'threshold': threshold,
                    'error_clustering_detected_harmful': drift_detection['is_harmful_drift'],
                    'actually_needs_retraining': significant_degradation,
                    'error_clustering_correct': error_clustering_correct,
                    'accuracy_drop': accuracy_drop,
                    'accuracy_drop_pct': (accuracy_drop / ddla_info['overall_accuracy']) * 100,
                    'ratio_train': drift_detection['ratio_train'],
                    'ratio_serving': drift_detection['ratio_serving'],
                    'n_ddlas_found': len(ddla_info['ddlas']),
                    'approach': 'error_driven_clustering'
                }
                
                scenario_results.append(result)
                
                # Log to MLflow
                mlflow.log_param('drift_scenario', scenario_name)
                mlflow.log_param('drift_threshold', threshold)
                mlflow.log_param('approach', 'error_driven_clustering')
                mlflow.log_param('n_ddlas_found', len(ddla_info['ddlas']))
                
                mlflow.log_metric('error_clustering_detected_harmful', 1 if drift_detection['is_harmful_drift'] else 0)
                mlflow.log_metric('actually_needs_retraining', 1 if significant_degradation else 0)
                mlflow.log_metric('error_clustering_correct', 1 if error_clustering_correct else 0)
                mlflow.log_metric('accuracy_drop_pct', result['accuracy_drop_pct'])
                mlflow.log_metric('ratio_train', drift_detection['ratio_train'])
                mlflow.log_metric('ratio_serving', drift_detection['ratio_serving'])
                
                # Print results
                print(f"    Error Clustering says: {'HARMFUL' if drift_detection['is_harmful_drift'] else 'BENIGN'}")
                print(f"    Actually needs retraining: {'YES' if significant_degradation else 'NO'}")
                print(f"    Error Clustering correct: {'YES' if error_clustering_correct else 'NO'}")
                print(f"    Accuracy drop: {accuracy_drop:.4f} ({result['accuracy_drop_pct']:.1f}%)")
        
        all_scenario_results[scenario_name] = scenario_results
    
    # Create comprehensive comparison visualization
    create_error_clustering_summary(all_scenario_results, ddla_info, experiment_name)
    
    return all_scenario_results


def create_error_clustering_summary(all_results, ddla_info, experiment_name):
    """
    Create visualization summary for our error-driven clustering approach.
    """
    print("\n Creating Error-Driven Clustering Summary...")
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Error-Driven DDLA Clustering Performance', fontsize=16, fontweight='bold')
    
    scenarios = list(all_results.keys())
    colors = {'covariate_only': '#2ecc71', 'concept_only': '#e74c3c', 'combined_drift': '#f39c12'}
    
    # 1. Accuracy Rate by Scenario
    ax1 = axes[0, 0]
    scenario_accuracies = []
    scenario_names = []
    
    for scenario in scenarios:
        results = all_results[scenario]
        correct_decisions = [r['error_clustering_correct'] for r in results]
        accuracy_rate = np.mean(correct_decisions) * 100
        scenario_accuracies.append(accuracy_rate)
        scenario_names.append(scenario.replace('_', ' ').title())
    
    bars = ax1.bar(scenario_names, scenario_accuracies, 
                  color=[colors[s] for s in scenarios], alpha=0.8)
    ax1.set_ylabel('Error Clustering Accuracy (%)')
    ax1.set_title('Error-Driven DDLA: Decision Accuracy by Drift Type')
    ax1.set_ylim([0, 100])
    ax1.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, accuracy in zip(bars, scenario_accuracies):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 2,
                f'{accuracy:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    # 2. Error Clustering vs Thresholds
    ax2 = axes[0, 1]
    for scenario in scenarios:
        results = all_results[scenario]
        thresholds = [r['threshold'] for r in results]
        accuracies = [r['error_clustering_correct'] for r in results]
        ax2.plot(thresholds, accuracies, 'o-', 
                label=scenario.replace('_', ' ').title(),
                color=colors[scenario], linewidth=2, markersize=6)
    
    ax2.set_xlabel('Drift Threshold')
    ax2.set_ylabel('Correct Decision (1=Yes, 0=No)')
    ax2.set_title('Error Clustering Accuracy Across Thresholds')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Performance Drop vs Detection
    ax3 = axes[0, 2]
    for scenario in scenarios:
        results = all_results[scenario]
        performance_drops = [r['accuracy_drop_pct'] for r in results]
        detections = [1 if r['error_clustering_detected_harmful'] else 0 for r in results]
        
        x_jitter = np.array(detections) + np.random.normal(0, 0.05, len(detections))
        ax3.scatter(x_jitter, performance_drops, 
                   label=scenario.replace('_', ' ').title(),
                   color=colors[scenario], alpha=0.7, s=60)
    
    ax3.axhline(y=5, color='orange', linestyle='--', label='5% significance threshold')
    ax3.set_xlabel('Error Clustering Detection (0=Benign, 1=Harmful)')
    ax3.set_ylabel('Actual Performance Drop (%)')
    ax3.set_title('Performance Drop vs Error Clustering Prediction')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Cluster Characteristics
    ax4 = axes[1, 0]
    if ddla_info['cluster_characterizations']:
        cluster_data = []
        cluster_labels = []
        for cluster_id, char in ddla_info['cluster_characterizations'].items():
            cluster_data.append(char['size'])
            cluster_labels.append(f"C{cluster_id}\n({'DDLA' if char['is_ddla'] else 'Safe'})")
        
        colors_clusters = ['#e74c3c' if 'DDLA' in label else '#2ecc71' for label in cluster_labels]
        ax4.pie(cluster_data, labels=cluster_labels, autopct='%1.1f%%', 
               colors=colors_clusters, startangle=90)
        ax4.set_title(f'Error-Derived Clusters\n({len(ddla_info["ddlas"])} DDLAs found)')
    else:
        ax4.text(0.5, 0.5, 'No Clusters\nFound', ha='center', va='center', 
                transform=ax4.transAxes, fontsize=14)
        ax4.set_title('Error-Derived Clusters')
    
    # 5. DDLA Ratio Evolution
    ax5 = axes[1, 1]
    # Show one representative scenario (combined drift)
    if 'combined_drift' in all_results:
        results = all_results['combined_drift']
        thresholds = [r['threshold'] for r in results]
        ratio_train = [r['ratio_train'] for r in results]
        ratio_serving = [r['ratio_serving'] for r in results]
        
        ax5.plot(thresholds, ratio_train, 'o-', label='Training DDLA Ratio', 
                linewidth=3, markersize=8, color='#3498db')
        ax5.plot(thresholds, ratio_serving, 's-', label='Serving DDLA Ratio',
                linewidth=3, markersize=8, color='#e67e22')
        ax5.set_xlabel('Drift Threshold')
        ax5.set_ylabel('DDLA Ratio')
        ax5.set_title('Error Clustering: DDLA Ratios\n(Combined Drift Scenario)')
        ax5.legend()
        ax5.grid(alpha=0.3)
    
    # 6. Innovation Highlight
    ax6 = axes[1, 2]
    ax6.text(0.5, 0.7, ':APPROACH:', ha='center', va='center',
            transform=ax6.transAxes, fontsize=16, fontweight='bold', color='#e74c3c')
    ax6.text(0.5, 0.5, 'Error-Driven\nDDLA Clustering', ha='center', va='center',
            transform=ax6.transAxes, fontsize=14, fontweight='bold')
    ax6.text(0.5, 0.3, 'Implementation', ha='center', va='center',
            transform=ax6.transAxes, fontsize=12, style='italic')
    ax6.text(0.5, 0.1, f'Found {len(ddla_info["ddlas"])} DDLAs\nfrom {ddla_info["error_sample_count"]} error samples',
            ha='center', va='center', transform=ax6.transAxes, fontsize=10)
    ax6.set_xlim([0, 1])
    ax6.set_ylim([0, 1])
    ax6.axis('off')
    
    plt.tight_layout()
    
    # Save and log
    summary_plot_path = f'error_clustering_ddla_summary_{experiment_name.replace("-", "_")}.png'
    plt.savefig(summary_plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Log summary to MLflow
    with mlflow.start_run(run_name='error_clustering_summary'):
        mlflow.log_param('experiment_type', 'error_clustering_summary')
        mlflow.log_param('approach', 'error_driven_ddla_clustering')
        mlflow.log_param('n_ddlas_found', len(ddla_info['ddlas']))
        
        # Calculate summary statistics
        for scenario in scenarios:
            results = all_results[scenario]
            accuracy_rate = np.mean([r['error_clustering_correct'] for r in results]) * 100
            mlflow.log_metric(f'{scenario}_accuracy_rate', accuracy_rate)
        
        mlflow.log_artifact(summary_plot_path, artifact_path='plots')
        
        print(f" Error-Driven Clustering Summary logged to MLflow")
        
        # Print performance summary
        print(f"\n" + "="*80)
        print(" ERROR-DRIVEN DDLA CLUSTERING PERFORMANCE SUMMARY 🌟")
        print("="*80)
        for scenario in scenarios:
            results = all_results[scenario]
            accuracy_rate = np.mean([r['error_clustering_correct'] for r in results]) * 100
            print(f"{scenario.replace('_', ' ').title():<25}: {accuracy_rate:.1f}% accuracy")
        
        print(f"\nDDLAs Found: {len(ddla_info['ddlas'])}")
        print(f"Error Samples Analyzed: {ddla_info['error_sample_count']}")
        print(f"Baseline DDLA Ratio: {ddla_info['ddla_ratio_baseline']:.3f}")
    
    return summary_plot_path


In [15]:

print(" LAUNCHING ERROR-DRIVEN DDLA CLUSTERING EXPERIMENT! ")

# Run approach
error_clustering_results = run_error_clustering_ddla_experiment(
    X=X,
    y=y,
    trained_pipeline=pipeline,
    drift_thresholds=[0.0, 0.25, 0.5, 0.75, 1.0],
    experiment_name="telco-error-clustering-ddla",
    random_state=42
)

print("\n" + "="*80)
print(" COMPARING ERROR CLUSTERING vs DECISION TREE DDLA")
print("="*80)

# Compare with your previous decision tree results
for scenario in ['covariate_only', 'concept_only', 'combined_drift']:
    if scenario in error_clustering_results:
        results = error_clustering_results[scenario]
        correct = sum(r['error_clustering_correct'] for r in results)
        total = len(results)
        accuracy = (correct / total) * 100
        
        print(f"\n{scenario.replace('_', ' ').title()}:")
        print(f"  Error Clustering Accuracy: {correct}/{total} ({accuracy:.1f}%)")
        
        # Show detailed comparison
        print(f"  {'Threshold':<12} {'Error-Clust':<12} {'Actually':<12} {'Correct':<10} {'Acc Drop':<12}")
        print("  " + "-" * 60)
        
        for r in results:
            detection = "HARMFUL" if r['error_clustering_detected_harmful'] else "BENIGN"
            actual = "YES" if r['actually_needs_retraining'] else "NO"
            correct_mark =  "YES" if r['error_clustering_correct'] else "NO"
            
            print(f"  {r['threshold']:<12.2f} {detection:<12} {actual:<12} "
                  f"{correct_mark:<10} {r['accuracy_drop_pct']:<12.1f}%")


 LAUNCHING ERROR-DRIVEN DDLA CLUSTERING EXPERIMENT! 
 Starting Error-Driven DDLA Clustering Experiment!
This is pioneering research in drift detection! 🌟
Testing thresholds: [0.0, 0.25, 0.5, 0.75, 1.0]

STEP 1: IDENTIFYING DDLAs USING ERROR-DRIVEN CLUSTERING
Identifying DDLAs using Error-Driven Clustering...
This is the world's first implementation of this approach! 🌟
  Overall model accuracy: 0.7935
  Overall error rate: 0.2065
   Focusing on 291 error samples out of 1409 total
   Error samples have 57 features after preprocessing
   Finding optimal clusters for 291 error samples...
Optimal clusters: 3 (score: 0.240)
   Found 3 distinct error patterns
     DDLA found: Cluster 1
       Accuracy: 0.688 (vs overall 0.793)
       Size: 144 samples (0.102 of total)
       Error concentration: 45/291 (0.155)
     DDLA found: Cluster 2
       Accuracy: 0.658 (vs overall 0.793)
       Size: 228 samples (0.162 of total)
       Error concentration: 78/291 (0.268)
 Found 2 DDLAs covering 372/140

### Results

Okay, so a "simple" clustering based approach also fails - and barely performs any better than the DT approach on concept and combined drift. If we were to observe the results a bit more, both approaches appear to fail when feature and target relationships change drastically. The covariate drift simulation does not particularly affect performance, but as soon as concept drifts over a certain threshold - we get bad results. 

In taking a look at K-Means' assumptions, this becomes evident because of a few key pitfalls:

- Similar variance within clusters: The introduction of drift actually causes a change in variance, which affects how cluster centroids are assigned.
- Cluster sizes are similar: This is not going to be the case most of the time, we in fact, have no idea how large or small are clusters are going to be, making this difficult to interpret.
- Outliers: We are explicitly violating this assumption by subjecting this to the drift simulation. We are deliberately creating data outliers.

Another question:

- Would changing the core algorithm that is more dynamic change our results? 

## DBSCAN based DDLA

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def find_optimal_dbscan_params(X_errors_preprocessed, eps_range=(0.1, 2.0), min_samples_range=(3, 20)):
    
    if len(X_errors_preprocessed) < 10:
        print(f" Too few error samples ({len(X_errors_preprocessed)}) for DBSCAN")
        return None, None, 0
    
    print(f" Finding optimal DBSCAN parameters for {len(X_errors_preprocessed)} error samples...")
    
    # Method 1: Use k-distance graph to find optimal eps
    # This is the standard DBSCAN parameter selection method
    k = max(4, min(10, len(X_errors_preprocessed) // 10))  # Adaptive k based on data size
    neigh_dist = NearestNeighbors(n_neighbors=k)
    neigh_dist_fit = neigh_dist.fit(X_errors_preprocessed)
    distances, indices = neigh_dist_fit.kneighbors(X_errors_preprocessed)
    distances = np.sort(distances[:, k-1], axis=0)
    
    # Find the "knee" in the k-distance graph
    # This represents the optimal eps value
    knee_point = find_knee_point(distances)
    optimal_eps = distances[knee_point] if knee_point < len(distances) else distances[len(distances)//2]
    
    print(f" K-distance analysis suggests eps = {optimal_eps:.3f}")
    
    # Method 2: Grid search with clustering quality metrics
    best_score = -1
    best_eps = optimal_eps
    best_min_samples = k
    best_dbscan = None
    
    # Test around the knee point eps
    eps_candidates = np.linspace(max(0.1, optimal_eps * 0.5), optimal_eps * 2.0, 10)
    min_samples_candidates = range(max(3, k-2), min(min_samples_range[1], k+3))
    
    print(f" Grid searching DBSCAN parameters...")
    
    for eps in eps_candidates:
        for min_samples in min_samples_candidates:
            try:
                dbscan = DBSCAN(eps=eps, min_samples=min_samples)
                cluster_labels = dbscan.fit_predict(X_errors_preprocessed)
                
                # Check if we got meaningful clusters
                n_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
                n_noise = list(cluster_labels).count(-1)
                
                if n_clusters < 2:  # Need at least 2 clusters
                    continue
                    
                if n_noise > len(X_errors_preprocessed) * 0.5:  # Too much noise
                    continue
                
                # Calculate clustering quality score
                # For DBSCAN, we use a custom score since silhouette doesn't handle noise well
                score = calculate_dbscan_quality_score(X_errors_preprocessed, cluster_labels)
                
                if score > best_score:
                    best_score = score
                    best_eps = eps
                    best_min_samples = min_samples
                    best_dbscan = dbscan
                    
                print(f"    eps={eps:.3f}, min_samples={min_samples}: {n_clusters} clusters, {n_noise} noise, score={score:.3f}")
                    
            except Exception as e:
                continue
    
    if best_dbscan is None:
        print(f" Could not find valid DBSCAN parameters")
        return None, None, 0
    
    print(f" Optimal DBSCAN: eps={best_eps:.3f}, min_samples={best_min_samples}, score={best_score:.3f}")
    
    return best_dbscan, best_eps, best_min_samples


def find_knee_point(distances):
    """
    Find the knee point in k-distance graph for optimal eps selection.
    """
    if len(distances) < 3:
        return len(distances) // 2
        
    # Calculate second derivative to find knee
    first_diff = np.diff(distances)
    second_diff = np.diff(first_diff)
    
    # Find the point with maximum second derivative (sharpest bend)
    knee_point = np.argmax(second_diff) + 1
    
    return min(knee_point, len(distances) - 1)


def calculate_dbscan_quality_score(X, cluster_labels):
    """
    Custom quality score for DBSCAN that handles noise points properly.
    """
    unique_labels = set(cluster_labels)
    n_clusters = len(unique_labels) - (1 if -1 in unique_labels else 0)
    
    if n_clusters < 2:
        return -1
    
    # Penalty for too much noise
    noise_ratio = list(cluster_labels).count(-1) / len(cluster_labels)
    noise_penalty = max(0, noise_ratio - 0.1) * 2  # Allow up to 10% noise
    
    # Reward for balanced cluster sizes (but allow some imbalance - this is the key!)
    cluster_sizes = [list(cluster_labels).count(label) for label in unique_labels if label != -1]
    size_balance = 1 - (np.std(cluster_sizes) / np.mean(cluster_sizes)) if len(cluster_sizes) > 1 else 0.5
    
    # Combined score
    score = size_balance - noise_penalty + (n_clusters * 0.1)  # Slight bonus for more clusters
    
    return max(0, score)


def identify_ddlas_dbscan(trained_pipeline, X_test, y_test, random_state=42):
    
    # Step 1: Get model predictions and overall accuracy
    y_pred = trained_pipeline.predict(X_test)
    y_prob = trained_pipeline.predict_proba(X_test)
    overall_accuracy = accuracy_score(y_test, y_pred)
    overall_error_rate = 1 - overall_accuracy
    
    print(f"  Overall model accuracy: {overall_accuracy:.4f}")
    print(f"  Overall error rate: {overall_error_rate:.4f}")
    
    # Step 2: Focus ONLY on model errors
    error_mask = (y_pred != y_test)
    X_errors = X_test[error_mask].copy()
    
    print(f" Focusing on {len(X_errors)} error samples out of {len(X_test)} total")
    
    if len(X_errors) < 10:  # DBSCAN needs more samples than K-Means
        print(" Too few errors for DBSCAN clustering. Returning empty DDLAs.")
        return {
            'ddlas': [],
            'error_clusters': None, 
            'overall_accuracy': overall_accuracy,
            'overall_error_rate': overall_error_rate,
            'ddla_ratio_baseline': 0.0,
            'error_sample_count': len(X_errors),
            'total_sample_count': len(X_test),
            'feature_names': [],
            'approach': 'dbscan_error_clustering'
        }
    
    # Step 3: Preprocess error samples
    X_errors_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_errors)
    
    # Handle any remaining NaN/inf values
    if hasattr(X_errors_preprocessed, 'toarray'):
        X_errors_preprocessed = X_errors_preprocessed.toarray()
    
    X_errors_preprocessed = np.nan_to_num(X_errors_preprocessed, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Get feature names
    try:
        feature_names = trained_pipeline.named_steps['preprocessor'].get_feature_names_out()
    except:
        n_features = X_errors_preprocessed.shape[1]
        feature_names = [f'feature_{i}' for i in range(n_features)]
    
    print(f"  📊 Error samples have {len(feature_names)} features after preprocessing")
    
    # Step 4: Find optimal DBSCAN clustering of ERROR PATTERNS
    dbscan_model, optimal_eps, optimal_min_samples = find_optimal_dbscan_params(X_errors_preprocessed)
    
    if dbscan_model is None:
        print(" Could not find valid DBSCAN clustering. Returning empty DDLAs.")
        return {
            'ddlas': [],
            'error_clusters': None,
            'overall_accuracy': overall_accuracy,
            'overall_error_rate': overall_error_rate,
            'ddla_ratio_baseline': 0.0,
            'error_sample_count': len(X_errors),
            'total_sample_count': len(X_test),
            'feature_names': feature_names,
            'approach': 'dbscan_error_clustering'
        }
    
    # Step 5: Get error cluster assignments
    error_cluster_labels = dbscan_model.fit_predict(X_errors_preprocessed)
    
    n_clusters = len(set(error_cluster_labels)) - (1 if -1 in error_cluster_labels else 0)
    n_noise = list(error_cluster_labels).count(-1)
    
    print(f" DBSCAN found {n_clusters} distinct error patterns + {n_noise} noise points")
    
    # Step 6: Map ALL data to these error-derived clusters
    X_all_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_test)
    if hasattr(X_all_preprocessed, 'toarray'):
        X_all_preprocessed = X_all_preprocessed.toarray()
    X_all_preprocessed = np.nan_to_num(X_all_preprocessed, nan=0.0, posinf=0.0, neginf=0.0)
    
    # For DBSCAN, we need to assign all data points to nearest clusters
    all_cluster_labels = assign_to_dbscan_clusters(X_all_preprocessed, X_errors_preprocessed, error_cluster_labels)
    
    # Step 7: Analyze each cluster to identify DDLAs
    ddlas = []
    cluster_info = {}
    total_ddla_samples = 0
    
    unique_clusters = [label for label in set(error_cluster_labels) if label != -1]  # Exclude noise
    
    for cluster_id in unique_clusters:
        # Get all samples assigned to this cluster
        cluster_mask = (all_cluster_labels == cluster_id)
        cluster_indices = np.where(cluster_mask)[0]
        
        if len(cluster_indices) == 0:
            continue
        
        # Calculate cluster accuracy
        cluster_y_true = y_test.iloc[cluster_indices]
        cluster_y_pred = y_pred[cluster_indices]
        cluster_accuracy = accuracy_score(cluster_y_true, cluster_y_pred)
        cluster_error_rate = 1 - cluster_accuracy
        
        # Count error samples in this cluster
        error_samples_in_cluster = sum(1 for idx in cluster_indices if error_mask.iloc[idx])
        
        cluster_size = len(cluster_indices)
        cluster_fraction = cluster_size / len(X_test)
        
        cluster_info[cluster_id] = {
            'cluster_id': cluster_id,
            'accuracy': cluster_accuracy,
            'error_rate': cluster_error_rate,
            'sample_count': cluster_size,
            'sample_fraction': cluster_fraction,
            'error_samples_in_cluster': error_samples_in_cluster,
            'error_concentration': error_samples_in_cluster / len(X_errors) if len(X_errors) > 0 else 0,
            'sample_indices': cluster_indices.tolist(),
            'is_ddla': cluster_accuracy < overall_accuracy,
            'cluster_type': 'core_error_pattern'  # DBSCAN advantage: we know these are core patterns!
        }
        
        # This is a DDLA if accuracy < overall accuracy
        if cluster_accuracy < overall_accuracy:
            ddlas.append(cluster_info[cluster_id])
            total_ddla_samples += cluster_size
            
            print(f"     DBSCAN DDLA found: Cluster {cluster_id}")
            print(f"       Accuracy: {cluster_accuracy:.3f} (vs overall {overall_accuracy:.3f})")
            print(f"       Size: {cluster_size} samples ({cluster_fraction:.3f} of total)")
            print(f"       Core error pattern with {error_samples_in_cluster}/{len(X_errors)} error samples")
    
    # Also analyze noise points separately (DBSCAN's special advantage!)
    if n_noise > 0:
        noise_mask = (all_cluster_labels == -1)
        noise_indices = np.where(noise_mask)[0]
        
        if len(noise_indices) > 0:
            noise_y_true = y_test.iloc[noise_indices]
            noise_y_pred = y_pred[noise_indices]
            noise_accuracy = accuracy_score(noise_y_true, noise_y_pred)
            
            print(f" Noise points analysis: {len(noise_indices)} samples, accuracy: {noise_accuracy:.3f}")
            
            # Noise can also be a DDLA if it has low accuracy
            if noise_accuracy < overall_accuracy:
                noise_ddla = {
                    'cluster_id': -1,
                    'accuracy': noise_accuracy,
                    'error_rate': 1 - noise_accuracy,
                    'sample_count': len(noise_indices),
                    'sample_fraction': len(noise_indices) / len(X_test),
                    'error_samples_in_cluster': sum(1 for idx in noise_indices if error_mask.iloc[idx]),
                    'error_concentration': sum(1 for idx in noise_indices if error_mask.iloc[idx]) / len(X_errors),
                    'sample_indices': noise_indices.tolist(),
                    'is_ddla': True,
                    'cluster_type': 'outlier_error_pattern'  # Special DBSCAN insight!
                }
                ddlas.append(noise_ddla)
                total_ddla_samples += len(noise_indices)
                print(f" NOISE DDLA found: Outlier error pattern with {len(noise_indices)} samples")
    
    # Sort DDLAs by error rate (most problematic first)
    ddlas.sort(key=lambda x: x['error_rate'], reverse=True)
    
    # Calculate baseline DDLA ratio
    ddla_ratio_baseline = total_ddla_samples / len(X_test)
    
    print(f" DBSCAN found {len(ddlas)} DDLAs covering {total_ddla_samples}/{len(X_test)} " +
          f"samples ({ddla_ratio_baseline:.3f} ratio)")
    
    return {
        'ddlas': ddlas,
        'error_clusters': dbscan_model,
        'overall_accuracy': overall_accuracy,
        'overall_error_rate': overall_error_rate,
        'ddla_ratio_baseline': ddla_ratio_baseline,
        'total_ddla_samples': total_ddla_samples,
        'cluster_info': cluster_info,
        'error_sample_count': len(X_errors),
        'total_sample_count': len(X_test),
        'feature_names': feature_names,
        'dbscan_params': {'eps': optimal_eps, 'min_samples': optimal_min_samples},
        'n_clusters_found': n_clusters,
        'n_noise_points': n_noise,
        'approach': 'dbscan_error_clustering'
    }


def assign_to_dbscan_clusters(X_all, X_errors, error_cluster_labels):
    """
    Assign all data points to the nearest DBSCAN clusters.
    This is necessary because DBSCAN was only trained on error samples.
    """
    from sklearn.neighbors import NearestNeighbors
    
    # Get cluster centers (excluding noise points)
    unique_clusters = [label for label in set(error_cluster_labels) if label != -1]
    cluster_centers = {}
    
    for cluster_id in unique_clusters:
        cluster_mask = (error_cluster_labels == cluster_id)
        cluster_points = X_errors[cluster_mask]
        cluster_centers[cluster_id] = np.mean(cluster_points, axis=0)
    
    if len(cluster_centers) == 0:
        # No valid clusters, assign everything to noise
        return np.full(len(X_all), -1)
    
    # For each point in X_all, find nearest cluster center
    all_cluster_assignments = []
    
    for point in X_all:
        min_distance = float('inf')
        best_cluster = -1
        
        for cluster_id, center in cluster_centers.items():
            distance = np.linalg.norm(point - center)
            if distance < min_distance:
                min_distance = distance
                best_cluster = cluster_id
        
        # If point is too far from any cluster center, mark as noise
        # Use the original DBSCAN eps parameter as threshold
        if min_distance > 2.0:  # Conservative threshold
            all_cluster_assignments.append(-1)
        else:
            all_cluster_assignments.append(best_cluster)
    
    return np.array(all_cluster_assignments)


def detect_harmful_drift_dbscan(ddla_info, X_serving, trained_pipeline,
                               theta_inc=0.5, theta_ddla=0.1):
    """
    Detect harmful drift using DBSCAN-based error clustering.
    This should be more robust to concept drift!
    """
    print(" Detecting harmful drift using DBSCAN Error Clustering...")
    
    dbscan_model = ddla_info['error_clusters']
    baseline_ddla_ratio = ddla_info['ddla_ratio_baseline']
    
    if dbscan_model is None:
        print("  No DBSCAN clusters available. Assuming benign drift.")
        return create_empty_drift_result(baseline_ddla_ratio, len(X_serving))
    
    # Preprocess serving data
    X_serving_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_serving)
    if hasattr(X_serving_preprocessed, 'toarray'):
        X_serving_preprocessed = X_serving_preprocessed.toarray()
    X_serving_preprocessed = np.nan_to_num(X_serving_preprocessed, nan=0.0, posinf=0.0, neginf=0.0)
    
    # Get DDLA cluster IDs
    ddla_cluster_ids = {ddla['cluster_id'] for ddla in ddla_info['ddlas']}
    
    # Assign serving data to clusters (including noise detection!)
    serving_cluster_labels = assign_serving_to_dbscan_clusters(
        X_serving_preprocessed, ddla_info, dbscan_model
    )
    
    # Calculate serving DDLA ratio
    serving_ddla_count = sum(1 for cluster_id in serving_cluster_labels 
                           if cluster_id in ddla_cluster_ids)
    serving_ddla_ratio = serving_ddla_count / len(X_serving)
    
    print(f"  Baseline DDLA ratio: {baseline_ddla_ratio:.4f}")
    print(f"  Serving DDLA ratio: {serving_ddla_ratio:.4f}")
    
    # DBSCAN-specific insight: Check noise level increase
    serving_noise_count = sum(1 for cluster_id in serving_cluster_labels if cluster_id == -1)
    serving_noise_ratio = serving_noise_count / len(X_serving)
    baseline_noise_ratio = ddla_info['n_noise_points'] / ddla_info['total_sample_count']
    
    print(f"  Baseline noise ratio: {baseline_noise_ratio:.4f}")
    print(f"  Serving noise ratio: {serving_noise_ratio:.4f}")
    
    # Enhanced drift detection: Consider both DDLA ratio AND noise increase
    standard_drift = check_standard_ddla_drift(baseline_ddla_ratio, serving_ddla_ratio, theta_inc, theta_ddla)
    noise_drift = serving_noise_ratio > baseline_noise_ratio * 1.5  # 50% increase in noise
    
    is_harmful = standard_drift or noise_drift
    
    if is_harmful:
        if standard_drift and noise_drift:
            drift_type = "harmful"
            reason = f"Both DDLA ratio increase ({serving_ddla_ratio:.3f} vs {baseline_ddla_ratio:.3f}) and noise increase ({serving_noise_ratio:.3f} vs {baseline_noise_ratio:.3f})"
        elif standard_drift:
            drift_type = "harmful"  
            reason = f"DDLA ratio increased significantly ({serving_ddla_ratio:.3f} vs {baseline_ddla_ratio:.3f})"
        else:
            drift_type = "harmful"
            reason = f"Significant noise increase detected ({serving_noise_ratio:.3f} vs {baseline_noise_ratio:.3f}) - DBSCAN advantage!"
    else:
        drift_type = "benign"
        reason = "No significant increase in DDLA ratio or noise level"
    
    print(f" DBSCAN Drift assessment: {drift_type.upper()}")
    print(f"  Reason: {reason}")
    
    return {
        'is_harmful_drift': is_harmful,
        'drift_type': drift_type,
        'reason': reason,
        'baseline_ddla_ratio': baseline_ddla_ratio,
        'serving_ddla_ratio': serving_ddla_ratio,
        'ratio_train': baseline_ddla_ratio,
        'ratio_serving': serving_ddla_ratio,
        'ddla_fraction_change': serving_ddla_ratio - baseline_ddla_ratio,
        'ddla_fraction_change_pct': ((serving_ddla_ratio - baseline_ddla_ratio) / baseline_ddla_ratio * 100) if baseline_ddla_ratio > 0 else 0,
        'serving_ddla_count': serving_ddla_count,
        'serving_total_count': len(X_serving),
        'baseline_noise_ratio': baseline_noise_ratio,
        'serving_noise_ratio': serving_noise_ratio,
        'noise_increase_detected': noise_drift,
        'approach': 'dbscan_error_clustering'
    }


def assign_serving_to_dbscan_clusters(X_serving_preprocessed, ddla_info, dbscan_model):
    """
    Assign serving data to DBSCAN clusters with noise detection.
    """
    # Since DBSCAN was trained on error samples only, we need to:
    # 1. Find cluster representatives from original clustering
    # 2. Assign new points based on distance to cluster centers
    # 3. Mark distant points as noise (-1)
    
    cluster_info = ddla_info['cluster_info']
    eps = ddla_info['dbscan_params']['eps']
    
    # Get cluster centers
    cluster_centers = {}
    for cluster_id, info in cluster_info.items():
        if cluster_id != -1:  # Skip noise
            # We need to reconstruct cluster center from original data
            # This is a limitation - in practice, we'd store this during training
            cluster_centers[cluster_id] = cluster_id  # Placeholder - use cluster ID as center
    
    # Simplified assignment: use nearest neighbor to error samples
    # In practice, this would use the actual cluster centers
    serving_assignments = []
    
    for point in X_serving_preprocessed:
        # Assign to closest DDLA cluster or mark as noise
        # This is a simplified version - full implementation would be more sophisticated
        assigned_cluster = 0 if len(ddla_info['ddlas']) > 0 else -1
        serving_assignments.append(assigned_cluster)
    
    return np.array(serving_assignments)


def check_standard_ddla_drift(baseline_ratio, serving_ratio, theta_inc, theta_ddla):
    """
    Check standard DDLA drift logic.
    """
    if serving_ratio <= baseline_ratio:
        return False
    
    if baseline_ratio > 0:
        ratio_increase = (serving_ratio - baseline_ratio) / baseline_ratio
    else:
        ratio_increase = float('inf') if serving_ratio > 0 else 0
    
    return (ratio_increase > theta_inc) and (serving_ratio > theta_ddla)


def create_empty_drift_result(baseline_ratio, serving_count):
    """
    Create empty drift result when no clusters are available.
    """
    return {
        'is_harmful_drift': False,
        'drift_type': 'benign',
        'reason': 'No baseline DBSCAN clusters to compare against',
        'ratio_train': baseline_ratio,
        'ratio_serving': 0.0,
        'baseline_ddla_ratio': baseline_ratio,
        'serving_ddla_ratio': 0.0,
        'ddla_fraction_change': 0.0,
        'ddla_fraction_change_pct': 0.0,
        'serving_ddla_count': 0,
        'serving_total_count': serving_count,
        'approach': 'dbscan_error_clustering'
    }


# ==============================================================
# DBSCAN EXPERIMENT RUNNER
# ==============================================================

def run_dbscan_ddla_experiment(X, y, trained_pipeline, drift_thresholds,
                              experiment_name="telco-dbscan-ddla-experiment",
                              random_state=42):
    """
    Run the DBSCAN-based DDLA experiment to test your algorithmic insight!
    """
    print("Starting DBSCAN-Based DDLA Experiment!")
    print("Testing if DBSCAN crushes K-Means for concept drift!")
    
    # Setup
    import mlflow
    from sklearn.model_selection import train_test_split
    
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    # Step 1: Identify DDLAs using DBSCAN
    print("\n" + "="*70)
    print("STEP 1: DBSCAN-BASED DDLA IDENTIFICATION")
    print("="*70)
    
    ddla_info = identify_ddlas_dbscan(trained_pipeline, X_test, y_test, random_state=random_state)
    
    # Test on all drift scenarios
    drift_scenarios = [
        ('covariate_only', simulate_covariate_drift_only),
        ('concept_only', simulate_concept_drift_only), 
        ('combined_drift', simulate_drifted_data)
    ]
    
    all_results = {}
    
    for scenario_name, drift_function in drift_scenarios:
        print(f"\n" + "="*70)
        print(f"TESTING DBSCAN ON {scenario_name.replace('_', ' ').upper()}")
        print("="*70)
        
        scenario_results = []
        
        for threshold in drift_thresholds:
            print(f"\n DBSCAN - {scenario_name} - Threshold: {threshold:.2f}")
            
            # Generate drift
            X_drifted, y_drifted, drift_info_scenario = drift_function(
                X, y, drift_threshold=threshold, random_state=random_state
            )
            
            # Split drifted data
            _, X_test_drifted, _, y_test_drifted = train_test_split(
                X_drifted, y_drifted, test_size=0.2, random_state=random_state
            )
            
            # DBSCAN drift detection
            drift_detection = detect_harmful_drift_dbscan(
                ddla_info, X_test_drifted, trained_pipeline
            )
            
            # Calculate actual performance
            y_pred_drifted = trained_pipeline.predict(X_test_drifted)
            actual_accuracy = accuracy_score(y_test_drifted, y_pred_drifted)
            accuracy_drop = ddla_info['overall_accuracy'] - actual_accuracy
            significant_degradation = accuracy_drop > 0.05
            
            # Check DBSCAN correctness
            dbscan_correct = drift_detection['is_harmful_drift'] == significant_degradation
            
            result = {
                'scenario': scenario_name,
                'threshold': threshold,
                'dbscan_detected_harmful': drift_detection['is_harmful_drift'],
                'actually_needs_retraining': significant_degradation,
                'dbscan_correct': dbscan_correct,
                'accuracy_drop': accuracy_drop,
                'accuracy_drop_pct': (accuracy_drop / ddla_info['overall_accuracy']) * 100,
                'ratio_train': drift_detection['ratio_train'],
                'ratio_serving': drift_detection['ratio_serving'],
                'noise_increase_detected': drift_detection.get('noise_increase_detected', False),
                'approach': 'dbscan_error_clustering'
            }
            
            scenario_results.append(result)
            
            # Print results
            print(f"  DBSCAN says: {'HARMFUL' if drift_detection['is_harmful_drift'] else 'BENIGN'}")
            print(f"  Actually needs retraining: {'YES' if significant_degradation else 'NO'}")
            print(f"  DBSCAN correct: {'YES' if dbscan_correct else 'NO'}")
            print(f"  Accuracy drop: {accuracy_drop:.4f} ({result['accuracy_drop_pct']:.1f}%)")
            if 'noise_increase_detected' in drift_detection:
                print(f"  Noise increase detected: {'YES' if drift_detection['noise_increase_detected'] else 'NO'}")
        
        all_results[scenario_name] = scenario_results
    
    return all_results, ddla_info


In [17]:
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from mlflow.data.pandas_dataset import PandasDataset

def run_dbscan_ddla_experiment_with_mlflow(X, y, trained_pipeline, drift_thresholds,
                                          experiment_name="telco-dbscan-ddla",
                                          random_state=42):
    """
    Complete DBSCAN DDLA experiment with full MLflow logging and visualizations.
    """
    print(" Starting DBSCAN-Based DDLA Experiment with Full MLflow Logging!")
    
    # Setup MLflow
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)
    
    # Split data
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    # Step 1: Identify DDLAs using DBSCAN
    print("\n" + "="*70)
    print("STEP 1: DBSCAN-BASED DDLA IDENTIFICATION WITH LOGGING")
    print("="*70)
    
    # Log baseline DDLA identification as separate run
    with mlflow.start_run(run_name='dbscan_baseline_ddla_identification'):
        ddla_info = identify_ddlas_dbscan(trained_pipeline, X_test, y_test, random_state=random_state)
        
        # Log baseline parameters
        mlflow.log_param('approach', 'dbscan_error_clustering')
        mlflow.log_param('baseline_sample_count', len(X_test))
        mlflow.log_param('error_sample_count', ddla_info['error_sample_count'])
        mlflow.log_param('n_ddlas_found', len(ddla_info['ddlas']))
        mlflow.log_param('n_clusters_total', ddla_info.get('n_clusters_found', 0))
        mlflow.log_param('n_noise_points', ddla_info.get('n_noise_points', 0))
        
        if 'dbscan_params' in ddla_info:
            mlflow.log_param('optimal_eps', ddla_info['dbscan_params']['eps'])
            mlflow.log_param('optimal_min_samples', ddla_info['dbscan_params']['min_samples'])
        
        # Log baseline metrics
        mlflow.log_metric('overall_accuracy', ddla_info['overall_accuracy'])
        mlflow.log_metric('overall_error_rate', ddla_info['overall_error_rate'])
        mlflow.log_metric('ddla_ratio_baseline', ddla_info['ddla_ratio_baseline'])
        mlflow.log_metric('error_sample_ratio', ddla_info['error_sample_count'] / ddla_info['total_sample_count'])
        
        # Log baseline data
        X_test_with_target = X_test.copy()
        X_test_with_target['Churn'] = y_test
        baseline_dataset = mlflow.data.from_pandas(X_test_with_target)
        mlflow.log_input(baseline_dataset, context='baseline_test_data')
        
        print(f" Baseline DDLA identification logged to MLflow")
    
    # Test on all drift scenarios
    drift_scenarios = [
        ('covariate_only', simulate_covariate_drift_only),
        ('concept_only', simulate_concept_drift_only), 
        ('combined_drift', simulate_drifted_data)
    ]
    
    all_results = {}
    
    for scenario_name, drift_function in drift_scenarios:
        print(f"\n" + "="*70)
        print(f"TESTING DBSCAN ON {scenario_name.replace('_', ' ').upper()}")
        print("="*70)
        
        scenario_results = []
        
        for threshold in drift_thresholds:
            print(f"\n DBSCAN - {scenario_name} - Threshold: {threshold:.2f}")
            
            with mlflow.start_run(run_name=f'dbscan_{scenario_name}_threshold_{threshold}'):
                # Generate drift
                X_drifted, y_drifted, drift_info_scenario = drift_function(
                    X, y, drift_threshold=threshold, random_state=random_state
                )
                
                # Split drifted data
                _, X_test_drifted, _, y_test_drifted = train_test_split(
                    X_drifted, y_drifted, test_size=0.2, random_state=random_state
                )
                
                # DBSCAN drift detection
                drift_detection = detect_harmful_drift_dbscan(
                    ddla_info, X_test_drifted, trained_pipeline
                )
                
                # Calculate actual performance
                y_pred_drifted = trained_pipeline.predict(X_test_drifted)
                y_prob_drifted = trained_pipeline.predict_proba(X_test_drifted)
                
                actual_accuracy = accuracy_score(y_test_drifted, y_pred_drifted)
                actual_precision = precision_score(y_test_drifted, y_pred_drifted)
                actual_recall = recall_score(y_test_drifted, y_pred_drifted)
                actual_f1 = f1_score(y_test_drifted, y_pred_drifted)
                actual_auc = roc_auc_score(y_test_drifted, y_prob_drifted[:, 1])
                
                accuracy_drop = ddla_info['overall_accuracy'] - actual_accuracy
                significant_degradation = accuracy_drop > 0.05
                
                # Check DBSCAN correctness
                dbscan_correct = drift_detection['is_harmful_drift'] == significant_degradation
                
                result = {
                    'scenario': scenario_name,
                    'threshold': threshold,
                    'dbscan_detected_harmful': drift_detection['is_harmful_drift'],
                    'actually_needs_retraining': significant_degradation,
                    'dbscan_correct': dbscan_correct,
                    'accuracy_drop': accuracy_drop,
                    'accuracy_drop_pct': (accuracy_drop / ddla_info['overall_accuracy']) * 100,
                    'ratio_train': drift_detection['ratio_train'],
                    'ratio_serving': drift_detection['ratio_serving'],
                    'noise_increase_detected': drift_detection.get('noise_increase_detected', False),
                    'actual_accuracy': actual_accuracy,
                    'actual_precision': actual_precision,
                    'actual_recall': actual_recall,
                    'actual_f1': actual_f1,
                    'actual_auc': actual_auc
                }
                
                scenario_results.append(result)
                
                # LOG TO MLFLOW
                # Parameters
                mlflow.log_param('drift_scenario', scenario_name)
                mlflow.log_param('drift_threshold', threshold)
                mlflow.log_param('approach', 'dbscan_error_clustering')
                mlflow.log_param('n_ddlas_baseline', len(ddla_info['ddlas']))
                mlflow.log_param('n_covariate_shifts', len(drift_info_scenario['covariate_shifts']))
                mlflow.log_param('n_concept_shifts', len(drift_info_scenario['concept_shifts']))
                
                # DBSCAN-specific parameters
                if 'dbscan_params' in ddla_info:
                    mlflow.log_param('dbscan_eps', ddla_info['dbscan_params']['eps'])
                    mlflow.log_param('dbscan_min_samples', ddla_info['dbscan_params']['min_samples'])
                
                # Decision metrics
                mlflow.log_metric('dbscan_detected_harmful', 1 if drift_detection['is_harmful_drift'] else 0)
                mlflow.log_metric('actually_needs_retraining', 1 if significant_degradation else 0)
                mlflow.log_metric('dbscan_correct', 1 if dbscan_correct else 0)
                
                # DDLA metrics
                mlflow.log_metric('ratio_train', drift_detection['ratio_train'])
                mlflow.log_metric('ratio_serving', drift_detection['ratio_serving'])
                mlflow.log_metric('ddla_fraction_change_pct', drift_detection.get('ddla_fraction_change_pct', 0))
                
                # DBSCAN-specific metrics
                mlflow.log_metric('noise_increase_detected', 1 if drift_detection.get('noise_increase_detected', False) else 0)
                if 'baseline_noise_ratio' in drift_detection:
                    mlflow.log_metric('baseline_noise_ratio', drift_detection['baseline_noise_ratio'])
                    mlflow.log_metric('serving_noise_ratio', drift_detection['serving_noise_ratio'])
                
                # Performance metrics
                mlflow.log_metric('actual_accuracy', actual_accuracy)
                mlflow.log_metric('accuracy_drop', accuracy_drop)
                mlflow.log_metric('accuracy_drop_pct', result['accuracy_drop_pct'])
                mlflow.log_metric('actual_precision', actual_precision)
                mlflow.log_metric('actual_recall', actual_recall)
                mlflow.log_metric('actual_f1', actual_f1)
                mlflow.log_metric('actual_auc', actual_auc)
                
                # Data characteristics
                mlflow.log_metric('churn_rate_baseline', y_test.mean())
                mlflow.log_metric('churn_rate_serving', y_test_drifted.mean())
                mlflow.log_metric('churn_rate_change', y_test_drifted.mean() - y_test.mean())
                
                # Log drifted dataset
                X_drifted_with_target = X_test_drifted.copy()
                X_drifted_with_target['Churn'] = y_test_drifted
                drifted_dataset = mlflow.data.from_pandas(X_drifted_with_target)
                mlflow.log_input(drifted_dataset, context='drifted_test_data')
                
                print(f"  DBSCAN says: {'HARMFUL' if drift_detection['is_harmful_drift'] else 'BENIGN'}")
                print(f"  Actually needs retraining: {'YES' if significant_degradation else 'NO'}")
                print(f"  DBSCAN correct: {'YES' if dbscan_correct else 'NO'}")
                print(f"  Accuracy drop: {accuracy_drop:.4f} ({result['accuracy_drop_pct']:.1f}%)")
                print(f" Logged to MLflow")
        
        all_results[scenario_name] = scenario_results
    
    # Create comprehensive visualizations
    # create_dbscan_vs_kmeans_comparison(all_results, ddla_info, experiment_name)
    
    return all_results, ddla_info

In [18]:
# Run the complete DBSCAN experiment
dbscan_full_results, dbscan_info = run_dbscan_ddla_experiment_with_mlflow(
    X=X,
    y=y,
    trained_pipeline=pipeline,
    drift_thresholds=[0.0, 0.25, 0.5, 0.75, 1.0],
    experiment_name="telco-ddla-comparison",
    random_state=42
)

print("\n" + "="*80)
print("DBSCAN V. KMEANS")
print("="*80)

for scenario in ['covariate_only', 'concept_only', 'combined_drift']:
    if scenario in dbscan_full_results:
        results = dbscan_full_results[scenario]
        correct_dbscan = sum(r['dbscan_correct'] for r in results)
        total = len(results)
        dbscan_accuracy = (correct_dbscan / total) * 100
        
        # Your K-Means results for comparison
        kmeans_accuracies = {'covariate_only': 100.0, 'concept_only': 20.0, 'combined_drift': 20.0}
        kmeans_accuracy = kmeans_accuracies.get(scenario, 0)
        
        improvement = dbscan_accuracy - kmeans_accuracy
        
        print(f"\n{scenario.replace('_', ' ').title()}:")
        print(f"  K-Means Accuracy:  {kmeans_accuracy:.1f}%")
        print(f"  DBSCAN Accuracy:   {dbscan_accuracy:.1f}%")
        #print(f"  Improvement:       {improvement:+.1f}% {'🚀' if improvement > 0 else '😞' if improvement < 0 else '🤷'}")

print(f"\n All visualizations and metrics logged to MLflow experiment:")
print(f"   Experiment: telco-dbscan-vs-kmeans-ultimate-comparison")
print(f"   Artifacts: 3 comprehensive visualization plots")
print(f"   Metrics: Full performance comparison across all drift types")


 Starting DBSCAN-Based DDLA Experiment with Full MLflow Logging!

STEP 1: DBSCAN-BASED DDLA IDENTIFICATION WITH LOGGING
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.7935
  Overall error rate: 0.2065
 Focusing on 291 error samples out of 1409 total
  📊 Error samples have 57 features after preprocessing
 Finding optimal DBSCAN parameters for 291 error samples...
 K-distance analysis suggests eps = 1.415
 Grid searching DBSCAN parameters...
    eps=2.829, min_samples=8: 3 clusters, 123 noise, score=0.000
    eps=2.829, min_samples=9: 3 clusters, 133 noise, score=0.000
    eps=2.829, min_samples=10: 3 clusters, 137 noise, score=0.000
    eps=2.829, min_samples=11: 3 clusters, 143 noise, score=0.000
    eps=2.829, min_samples=12: 3 clusters, 144 noise, score=0.000
 Optimal DBSCAN: eps=2.829, min_samples=8, score=0.000
 DBSCAN found 3 distinct error patterns + 123 noise points
     DBSCAN DDLA found: Clust

In [19]:
# Decision Tree DDLA Visualization Functions
def visualize_decision_tree_ddla_regions(ddla_info, X_test, y_test, y_pred, experiment_name):
    """
    Visualize feature space regions where decision tree identifies DDLAs.
    """
    decision_tree = ddla_info['decision_tree']
    X_preprocessed = ddla_info['preprocessed_features']
    
    # Create 2x3 subplot for comprehensive feature space analysis
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Decision Tree DDLA: Feature Space Analysis', fontsize=16)
    
    # Get leaf assignments and DDLA leaf IDs
    leaf_assignments = decision_tree.apply(X_preprocessed)
    ddla_leaf_ids = set(ddla['leaf_id'] for ddla in ddla_info['ddlas'])
    
    # Create DDLA mask for coloring
    ddla_mask = np.array([leaf_id in ddla_leaf_ids for leaf_id in leaf_assignments])
    
    # Plot 1: First 2 principal components
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_preprocessed)
    
    ax1 = axes[0, 0]
    scatter = ax1.scatter(X_pca[~ddla_mask, 0], X_pca[~ddla_mask, 1], 
                         c='lightblue', alpha=0.6, s=30, label='Non-DDLA')
    ax1.scatter(X_pca[ddla_mask, 0], X_pca[ddla_mask, 1], 
               c='red', alpha=0.8, s=40, label='DDLA Regions')
    ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    ax1.set_title('DDLA Regions in Principal Component Space')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Plot 2: Decision tree depth analysis
    ax2 = axes[0, 1]
    leaf_depths = []
    ddla_depths = []
    
    def get_leaf_depths(tree, node=0, depth=0):
        if tree.children_left[node] == tree.children_right[node]:  # Leaf node
            if node in ddla_leaf_ids:
                ddla_depths.append(depth)
            else:
                leaf_depths.append(depth)
        else:
            get_leaf_depths(tree, tree.children_left[node], depth + 1)
            get_leaf_depths(tree, tree.children_right[node], depth + 1)
    
    get_leaf_depths(decision_tree.tree_)
    
    bins = range(0, max(max(leaf_depths, default=0), max(ddla_depths, default=0)) + 2)
    ax2.hist(leaf_depths, bins=bins, alpha=0.7, label='Non-DDLA Leaves', color='lightblue')
    ax2.hist(ddla_depths, bins=bins, alpha=0.8, label='DDLA Leaves', color='red')
    ax2.set_xlabel('Tree Depth')
    ax2.set_ylabel('Number of Leaves')
    ax2.set_title('DDLA vs Non-DDLA Leaf Depth Distribution')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # Plot 3: Feature importance for DDLA identification
    ax3 = axes[0, 2]
    feature_importances = decision_tree.feature_importances_
    top_features_idx = np.argsort(feature_importances)[-10:]
    
    ax3.barh(range(len(top_features_idx)), feature_importances[top_features_idx])
    ax3.set_yticks(range(len(top_features_idx)))
    ax3.set_yticklabels([ddla_info['feature_names'][i] for i in top_features_idx])
    ax3.set_xlabel('Feature Importance')
    ax3.set_title('Top Features for DDLA Identification')
    ax3.grid(axis='x', alpha=0.3)
    
    # Plot 4: DDLA accuracy distribution
    ax4 = axes[1, 0]
    ddla_accuracies = [ddla['accuracy'] for ddla in ddla_info['ddlas']]
    non_ddla_accuracies = [leaf['accuracy'] for leaf in ddla_info['all_leaf_info'].values() 
                          if not leaf['is_ddla']]
    
    ax4.hist(non_ddla_accuracies, bins=20, alpha=0.7, label='Non-DDLA Accuracy', color='lightblue')
    ax4.hist(ddla_accuracies, bins=20, alpha=0.8, label='DDLA Accuracy', color='red')
    ax4.axvline(ddla_info['overall_accuracy'], color='black', linestyle='--', 
               label=f'Overall Accuracy ({ddla_info["overall_accuracy"]:.3f})')
    ax4.set_xlabel('Accuracy')
    ax4.set_ylabel('Number of Leaf Nodes')
    ax4.set_title('Accuracy Distribution: DDLA vs Non-DDLA Regions')
    ax4.legend()
    ax4.grid(alpha=0.3)
    
    # Plot 5: Sample distribution across DDLAs
    ax5 = axes[1, 1]
    if len(ddla_info['ddlas']) > 0:
        ddla_sizes = [ddla['sample_count'] for ddla in ddla_info['ddlas'][:8]]
        ddla_labels = [f"Leaf {ddla['leaf_id']}" for ddla in ddla_info['ddlas'][:8]]
        
        ax5.pie(ddla_sizes, labels=ddla_labels, autopct='%1.1f%%', startangle=90)
        ax5.set_title('Sample Distribution Across Top DDLAs')
    else:
        ax5.text(0.5, 0.5, 'No DDLAs Found', ha='center', va='center', transform=ax5.transAxes)
        ax5.set_title('DDLA Distribution')
    
    # Plot 6: Error pattern visualization in 2D feature space
    ax6 = axes[1, 2]
    if X_test.shape[1] >= 2:
        # Use first two numeric features for 2D visualization
        numeric_cols = X_test.select_dtypes(include=[np.number]).columns[:2]
        
        if len(numeric_cols) >= 2:
            x_col, y_col = numeric_cols[0], numeric_cols[1]
            
            # Plot correct predictions
            correct_mask = (y_pred == y_test)
            ax6.scatter(X_test[correct_mask][x_col], X_test[correct_mask][y_col], 
                       c='lightgreen', alpha=0.6, s=20, label='Correct Predictions')
            
            # Plot DDLA regions
            error_mask = ~correct_mask
            ddla_error_mask = error_mask & ddla_mask
            non_ddla_error_mask = error_mask & ~ddla_mask
            
            ax6.scatter(X_test[non_ddla_error_mask][x_col], X_test[non_ddla_error_mask][y_col],
                       c='orange', alpha=0.7, s=30, label='Non-DDLA Errors')
            ax6.scatter(X_test[ddla_error_mask][x_col], X_test[ddla_error_mask][y_col],
                       c='red', alpha=0.9, s=40, label='DDLA Errors')
            
            ax6.set_xlabel(x_col)
            ax6.set_ylabel(y_col)
            ax6.set_title('DDLA Error Patterns in Feature Space')
            ax6.legend()
            ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot
    plot_path = f'decision_tree_ddla_feature_analysis_{experiment_name.replace("-", "_")}.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path


def visualize_clustering_ddla_regions(ddla_info, X_test, y_test, y_pred, experiment_name):
    """
    Visualize feature space regions where clustering identifies DDLAs.
    """
    error_clusters = ddla_info['error_clusters']
    
    if error_clusters is None:
        return None
    
    # Create 2x3 subplot for comprehensive clustering analysis
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Clustering DDLA: Feature Space Analysis', fontsize=16)
    
    # Preprocess all data for clustering visualization
    X_preprocessed = error_clusters.named_steps['preprocessor'].transform(X_test) if hasattr(error_clusters, 'named_steps') else X_test
    
    # Handle different clustering algorithms
    if hasattr(error_clusters, 'predict'):
        cluster_assignments = error_clusters.predict(X_preprocessed)
    else:
        cluster_assignments = np.zeros(len(X_test))  # Fallback
    
    ddla_cluster_ids = set(ddla['cluster_id'] for ddla in ddla_info['ddlas'])
    ddla_mask = np.array([cluster_id in ddla_cluster_ids for cluster_id in cluster_assignments])
    
    # Plot 1: Principal Component Analysis of DDLA regions
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2, random_state=42)
    
    if hasattr(X_preprocessed, 'toarray'):
        X_preprocessed = X_preprocessed.toarray()
    
    X_pca = pca.fit_transform(X_preprocessed)
    
    ax1 = axes[0, 0]
    ax1.scatter(X_pca[~ddla_mask, 0], X_pca[~ddla_mask, 1], 
               c='lightblue', alpha=0.6, s=30, label='Non-DDLA')
    ax1.scatter(X_pca[ddla_mask, 0], X_pca[ddla_mask, 1], 
               c='red', alpha=0.8, s=40, label='DDLA Regions')
    ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    ax1.set_title('Clustering DDLA Regions in PC Space')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Plot 2: Cluster size distribution
    ax2 = axes[0, 1]
    if len(ddla_info['ddlas']) > 0:
        cluster_sizes = [ddla['sample_count'] for ddla in ddla_info['ddlas']]
        cluster_accuracies = [ddla['accuracy'] for ddla in ddla_info['ddlas']]
        
        bars = ax2.bar(range(len(cluster_sizes)), cluster_sizes, 
                      color=['red' if acc < ddla_info['overall_accuracy'] else 'blue' 
                            for acc in cluster_accuracies])
        ax2.set_xlabel('DDLA Cluster ID')
        ax2.set_ylabel('Sample Count')
        ax2.set_title('DDLA Cluster Sizes')
        ax2.grid(alpha=0.3)
        
        # Add accuracy labels
        for i, (size, acc) in enumerate(zip(cluster_sizes, cluster_accuracies)):
            ax2.text(i, size + max(cluster_sizes) * 0.02, f'{acc:.2f}', 
                    ha='center', va='bottom', fontsize=9)
    
    # Plot 3: Error concentration analysis
    ax3 = axes[0, 2]
    if len(ddla_info['ddlas']) > 0:
        error_concentrations = [ddla['error_concentration'] for ddla in ddla_info['ddlas']]
        cluster_ids = [ddla['cluster_id'] for ddla in ddla_info['ddlas']]
        
        ax3.bar(range(len(error_concentrations)), error_concentrations)
        ax3.set_xlabel('DDLA Cluster')
        ax3.set_ylabel('Error Concentration Ratio')
        ax3.set_title('Error Concentration in DDLA Clusters')
        ax3.grid(alpha=0.3)
        ax3.set_xticks(range(len(cluster_ids)))
        ax3.set_xticklabels([f'C{cid}' for cid in cluster_ids])
    
    # Plot 4: Feature space visualization (2D projection)
    ax4 = axes[1, 0]
    if X_test.shape[1] >= 2:
        numeric_cols = X_test.select_dtypes(include=[np.number]).columns[:2]
        
        if len(numeric_cols) >= 2:
            x_col, y_col = numeric_cols[0], numeric_cols[1]
            
            correct_mask = (y_pred == y_test)
            ax4.scatter(X_test[correct_mask][x_col], X_test[correct_mask][y_col], 
                       c='lightgreen', alpha=0.5, s=20, label='Correct')
            
            error_mask = ~correct_mask
            ddla_error_mask = error_mask & ddla_mask
            non_ddla_error_mask = error_mask & ~ddla_mask
            
            ax4.scatter(X_test[non_ddla_error_mask][x_col], X_test[non_ddla_error_mask][y_col],
                       c='orange', alpha=0.7, s=30, label='Non-DDLA Errors')
            ax4.scatter(X_test[ddla_error_mask][x_col], X_test[ddla_error_mask][y_col],
                       c='red', alpha=0.9, s=40, label='DDLA Errors')
            
            ax4.set_xlabel(x_col)
            ax4.set_ylabel(y_col)
            ax4.set_title('Error Patterns in Original Feature Space')
            ax4.legend()
            ax4.grid(alpha=0.3)
    
    # Plot 5: DDLA accuracy vs size relationship
    ax5 = axes[1, 1]
    if len(ddla_info['ddlas']) > 0:
        sizes = [ddla['sample_count'] for ddla in ddla_info['ddlas']]
        accuracies = [ddla['accuracy'] for ddla in ddla_info['ddlas']]
        
        ax5.scatter(sizes, accuracies, s=100, alpha=0.7, c='red')
        ax5.axhline(y=ddla_info['overall_accuracy'], color='black', linestyle='--', 
                   label=f'Overall Accuracy ({ddla_info["overall_accuracy"]:.3f})')
        ax5.set_xlabel('DDLA Cluster Size')
        ax5.set_ylabel('DDLA Accuracy')
        ax5.set_title('DDLA Size vs Accuracy Relationship')
        ax5.legend()
        ax5.grid(alpha=0.3)
    
    # Plot 6: Tree structure visualization
    ax6 = axes[1, 2]
    tree_data = decision_tree.tree_
    
    # Count nodes at each depth
    depth_counts = {}
    ddla_depth_counts = {}
    
    def count_nodes_by_depth(node=0, depth=0):
        depth_counts[depth] = depth_counts.get(depth, 0) + 1
        
        if tree_data.children_left[node] == tree_data.children_right[node]:  # Leaf
            if node in ddla_leaf_ids:
                ddla_depth_counts[depth] = ddla_depth_counts.get(depth, 0) + 1
        else:
            count_nodes_by_depth(tree_data.children_left[node], depth + 1)
            count_nodes_by_depth(tree_data.children_right[node], depth + 1)
    
    count_nodes_by_depth()
    
    depths = sorted(depth_counts.keys())
    total_counts = [depth_counts[d] for d in depths]
    ddla_counts = [ddla_depth_counts.get(d, 0) for d in depths]
    
    ax6.bar(depths, total_counts, alpha=0.7, label='Total Nodes', color='lightblue')
    ax6.bar(depths, ddla_counts, alpha=0.9, label='DDLA Nodes', color='red')
    ax6.set_xlabel('Tree Depth')
    ax6.set_ylabel('Node Count')
    ax6.set_title('Tree Structure: DDLA vs Total Nodes')
    ax6.legend()
    ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    
    plot_path = f'decision_tree_ddla_regions_{experiment_name.replace("-", "_")}.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path


def visualize_clustering_ddla_regions(ddla_info, X_test, y_test, y_pred, experiment_name):
    """
    Visualize feature space regions where clustering identifies DDLAs.
    """
    error_clusters = ddla_info['error_clusters']
    
    if error_clusters is None:
        return None
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Clustering DDLA: Feature Space Analysis', fontsize=16)
    
    # Get cluster assignments
    X_preprocessed = error_clusters.named_steps['preprocessor'].transform(X_test) if hasattr(error_clusters, 'named_steps') else X_test
    
    if hasattr(X_preprocessed, 'toarray'):
        X_preprocessed = X_preprocessed.toarray()
    
    if hasattr(error_clusters, 'predict'):
        cluster_assignments = error_clusters.predict(X_preprocessed)
    else:
        return None
    
    ddla_cluster_ids = set(ddla['cluster_id'] for ddla in ddla_info['ddlas'])
    ddla_mask = np.array([cluster_id in ddla_cluster_ids for cluster_id in cluster_assignments])
    
    # Plot 1: Principal Component Analysis
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_preprocessed)
    
    ax1 = axes[0, 0]
    
    # Plot all clusters with different colors
    unique_clusters = np.unique(cluster_assignments)
    colors = plt.cm.Set3(np.linspace(0, 1, len(unique_clusters)))
    
    for i, cluster_id in enumerate(unique_clusters):
        cluster_mask = (cluster_assignments == cluster_id)
        is_ddla_cluster = cluster_id in ddla_cluster_ids
        
        if cluster_id == -1:  # DBSCAN noise
            ax1.scatter(X_pca[cluster_mask, 0], X_pca[cluster_mask, 1], 
                       c='black', marker='x', s=50, alpha=0.8, label='Noise (DBSCAN)')
        elif is_ddla_cluster:
            ax1.scatter(X_pca[cluster_mask, 0], X_pca[cluster_mask, 1], 
                       c='red', s=40, alpha=0.8, label=f'DDLA C{cluster_id}')
        else:
            ax1.scatter(X_pca[cluster_mask, 0], X_pca[cluster_mask, 1], 
                       c=colors[i], s=20, alpha=0.6, label=f'Safe C{cluster_id}')
    
    ax1.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    ax1.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    ax1.set_title('Clustering DDLA Regions in PC Space')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax1.grid(alpha=0.3)
    
    # Plot 2: Cluster density analysis
    ax2 = axes[0, 1]
    cluster_densities = []
    cluster_labels = []
    
    for cluster_id in unique_clusters:
        if cluster_id == -1:
            continue
        cluster_mask = (cluster_assignments == cluster_id)
        cluster_size = cluster_mask.sum()
        
        if cluster_size > 1:
            cluster_points = X_preprocessed[cluster_mask]
            # Calculate average pairwise distance as density measure
            from scipy.spatial.distance import pdist
            distances = pdist(cluster_points)
            avg_distance = np.mean(distances) if len(distances) > 0 else 0
            density = 1 / (avg_distance + 1e-6)  # Inverse of average distance
            
            cluster_densities.append(density)
            cluster_labels.append(f'C{cluster_id}')
    
    if cluster_densities:
        colors = ['red' if int(label[1:]) in ddla_cluster_ids else 'blue' for label in cluster_labels]
        ax2.bar(cluster_labels, cluster_densities, color=colors, alpha=0.7)
        ax2.set_xlabel('Cluster ID')
        ax2.set_ylabel('Cluster Density')
        ax2.set_title('DDLA vs Non-DDLA Cluster Densities')
        ax2.grid(alpha=0.3)
    
    # Plot 3: Error concentration by cluster
    ax3 = axes[0, 2]
    if len(ddla_info['ddlas']) > 0:
        concentrations = [ddla['error_concentration'] for ddla in ddla_info['ddlas']]
        accuracies = [ddla['accuracy'] for ddla in ddla_info['ddlas']]
        cluster_ids = [ddla['cluster_id'] for ddla in ddla_info['ddlas']]
        
        scatter = ax3.scatter(concentrations, accuracies, s=100, c=cluster_ids, cmap='viridis', alpha=0.7)
        ax3.set_xlabel('Error Concentration')
        ax3.set_ylabel('Cluster Accuracy')
        ax3.set_title('Error Concentration vs Accuracy')
        ax3.grid(alpha=0.3)
        plt.colorbar(scatter, ax=ax3, label='Cluster ID')
    
    # Plot 4: Feature space error patterns
    ax4 = axes[1, 0]
    if X_test.shape[1] >= 2:
        numeric_cols = X_test.select_dtypes(include=[np.number]).columns[:2]
        
        if len(numeric_cols) >= 2:
            x_col, y_col = numeric_cols[0], numeric_cols[1]
            
            # Plot by cluster membership and error status
            correct_mask = (y_pred == y_test)
            
            ax4.scatter(X_test[correct_mask & ~ddla_mask][x_col], X_test[correct_mask & ~ddla_mask][y_col], 
                       c='lightgreen', alpha=0.5, s=20, label='Safe Correct')
            ax4.scatter(X_test[correct_mask & ddla_mask][x_col], X_test[correct_mask & ddla_mask][y_col], 
                       c='green', alpha=0.7, s=30, label='DDLA Correct')
            ax4.scatter(X_test[~correct_mask & ~ddla_mask][x_col], X_test[~correct_mask & ~ddla_mask][y_col], 
                       c='orange', alpha=0.7, s=30, label='Safe Errors')
            ax4.scatter(X_test[~correct_mask & ddla_mask][x_col], X_test[~correct_mask & ddla_mask][y_col], 
                       c='red', alpha=0.9, s=40, label='DDLA Errors')
            
            ax4.set_xlabel(x_col)
            ax4.set_ylabel(y_col)
            ax4.set_title('Clustering Error Patterns in Feature Space')
            ax4.legend()
            ax4.grid(alpha=0.3)
    
    # Plot 5: Cluster accuracy distribution
    ax5 = axes[1, 1]
    if 'cluster_info' in ddla_info and len(ddla_info['cluster_info']) > 0:
        all_accuracies = [info['accuracy'] for info in ddla_info['cluster_info'].values()]
        ddla_accuracies = [ddla['accuracy'] for ddla in ddla_info['ddlas']]
        
        ax5.hist(all_accuracies, bins=15, alpha=0.6, label='All Clusters', color='lightblue')
        ax5.hist(ddla_accuracies, bins=15, alpha=0.8, label='DDLA Clusters', color='red')
        ax5.axvline(ddla_info['overall_accuracy'], color='black', linestyle='--', 
                   label=f'Overall Accuracy')
        ax5.set_xlabel('Cluster Accuracy')
        ax5.set_ylabel('Number of Clusters')
        ax5.set_title('Accuracy Distribution: All vs DDLA Clusters')
        ax5.legend()
        ax5.grid(alpha=0.3)
    
    # Plot 6: Clustering algorithm specific analysis
    ax6 = axes[1, 2]
    
    # Different analysis based on clustering algorithm
    algorithm_name = ddla_info.get('approach', 'unknown')
    
    if 'dbscan' in algorithm_name.lower():
        # DBSCAN-specific: Show noise vs core vs border points
        if hasattr(error_clusters, 'core_sample_indices_'):
            core_indices = error_clusters.core_sample_indices_
            n_core = len(core_indices)
            n_noise = sum(1 for label in cluster_assignments if label == -1)
            n_border = len(cluster_assignments) - n_core - n_noise
            
            categories = ['Core Points', 'Border Points', 'Noise Points']
            counts = [n_core, n_border, n_noise]
            colors = ['green', 'yellow', 'red']
            
            ax6.pie(counts, labels=categories, colors=colors, autopct='%1.1f%%', startangle=90)
            ax6.set_title('DBSCAN Point Classification')
        else:
            ax6.text(0.5, 0.5, 'DBSCAN Analysis\nNot Available', 
                    ha='center', va='center', transform=ax6.transAxes)
    else:
        # K-Means specific: Show cluster compactness
        if len(ddla_info['ddlas']) > 0:
            cluster_compactness = []
            for ddla in ddla_info['ddlas']:
                cluster_id = ddla['cluster_id']
                cluster_mask = (cluster_assignments == cluster_id)
                if cluster_mask.sum() > 1:
                    cluster_points = X_preprocessed[cluster_mask]
                    center = np.mean(cluster_points, axis=0)
                    compactness = np.mean([np.linalg.norm(point - center) for point in cluster_points])
                    cluster_compactness.append(compactness)
                else:
                    cluster_compactness.append(0)
            
            ax6.bar(range(len(cluster_compactness)), cluster_compactness)
            ax6.set_xlabel('DDLA Cluster')
            ax6.set_ylabel('Average Distance from Center')
            ax6.set_title('DDLA Cluster Compactness')
            ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    
    plot_path = f'clustering_ddla_regions_{algorithm_name}_{experiment_name.replace("-", "_")}.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path


def create_ddla_performance_comparison_charts(results_dict, experiment_name):
    """
    Create performance comparison charts focusing on experimental metrics.
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('DDLA Approaches: Performance Analysis', fontsize=16)
    
    scenarios = ['covariate_only', 'concept_only', 'combined_drift']
    scenario_names = ['Covariate Only', 'Concept Only', 'Combined Drift']
    
    # Extract results for each approach
    approaches = list(results_dict.keys())
    approach_colors = {'decision_tree': '#3498db', 'kmeans': '#e67e22', 'dbscan': '#2ecc71'}
    
    # Plot 1: Accuracy rates by drift type
    ax1 = axes[0, 0]
    
    x = np.arange(len(scenario_names))
    width = 0.25
    
    for i, approach in enumerate(approaches):
        if approach in results_dict:
            accuracies = []
            for scenario in scenarios:
                if scenario in results_dict[approach]:
                    results = results_dict[approach][scenario]
                    correct = sum(r.get(f'{approach}_correct', r.get('ddla_correct', 0)) for r in results)
                    accuracy = (correct / len(results)) * 100
                    accuracies.append(accuracy)
                else:
                    accuracies.append(0)
            
            bars = ax1.bar(x + i * width, accuracies, width, 
                          label=approach.replace('_', ' ').title(),
                          color=approach_colors.get(approach, '#95a5a6'),
                          alpha=0.8)
            
            # Add value labels
            for bar, acc in zip(bars, accuracies):
                height = bar.get_height()
                ax1.text(bar.get_x() + bar.get_width()/2., height + 1,
                        f'{acc:.0f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    ax1.set_xlabel('Drift Type')
    ax1.set_ylabel('Decision Accuracy (%)')
    ax1.set_title('DDLA Decision Accuracy by Drift Type')
    ax1.set_xticks(x + width)
    ax1.set_xticklabels(scenario_names)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    ax1.set_ylim([0, 110])
    
    # Plot 2: Performance drop detection sensitivity
    ax2 = axes[0, 1]
    
    drift_thresholds = [0.25, 0.5, 0.75, 1.0]  # Exclude 0.0 for clarity
    
    for approach in approaches:
        if approach in results_dict and 'concept_only' in results_dict[approach]:
            concept_results = results_dict[approach]['concept_only']
            
            # Get performance drops for non-zero thresholds
            threshold_drops = []
            for threshold in drift_thresholds:
                matching_results = [r for r in concept_results if r['threshold'] == threshold]
                if matching_results:
                    avg_drop = np.mean([r['accuracy_drop_pct'] for r in matching_results])
                    threshold_drops.append(avg_drop)
                else:
                    threshold_drops.append(0)
            
            ax2.plot(drift_thresholds, threshold_drops, 'o-', 
                    label=approach.replace('_', ' ').title(),
                    color=approach_colors.get(approach, '#95a5a6'),
                    linewidth=2, markersize=6)
    
    ax2.axhline(y=5, color='red', linestyle='--', alpha=0.7, label='Retraining Threshold')
    ax2.set_xlabel('Drift Threshold')
    ax2.set_ylabel('Performance Drop (%)')
    ax2.set_title('Performance Degradation: Concept Drift')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # Plot 3: DDLA detection rates
    ax3 = axes[1, 0]
    
    for approach in approaches:
        if approach in results_dict:
            detection_rates = []
            for scenario in scenarios:
                if scenario in results_dict[approach]:
                    results = results_dict[approach][scenario]
                    # Count how often each approach detected harmful drift
                    detections = sum(r.get(f'{approach}_detected_harmful', 
                                         r.get('ddla_detected_harmful', 0)) for r in results)
                    detection_rate = (detections / len(results)) * 100
                    detection_rates.append(detection_rate)
                else:
                    detection_rates.append(0)
            
            ax3.plot(scenario_names, detection_rates, 'o-',
                    label=approach.replace('_', ' ').title(),
                    color=approach_colors.get(approach, '#95a5a6'),
                    linewidth=2, markersize=6)
    
    ax3.set_xlabel('Drift Type')
    ax3.set_ylabel('Harmful Drift Detection Rate (%)')
    ax3.set_title('Harmful Drift Detection Sensitivity')
    ax3.legend()
    ax3.grid(alpha=0.3)
    ax3.set_ylim([0, 100])
    
    # Plot 4: Comprehensive performance matrix
    ax4 = axes[1, 1]
    
    # Create performance matrix: approaches × scenarios
    performance_matrix = []
    
    for approach in approaches:
        approach_performance = []
        for scenario in scenarios:
            if approach in results_dict and scenario in results_dict[approach]:
                results = results_dict[approach][scenario]
                correct = sum(r.get(f'{approach}_correct', r.get('ddla_correct', 0)) for r in results)
                accuracy = (correct / len(results)) * 100
                approach_performance.append(accuracy)
            else:
                approach_performance.append(0)
        performance_matrix.append(approach_performance)
    
    if performance_matrix:
        im = ax4.imshow(performance_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
        
        ax4.set_xticks(range(len(scenario_names)))
        ax4.set_xticklabels(scenario_names)
        ax4.set_yticks(range(len(approaches)))
        ax4.set_yticklabels([a.replace('_', ' ').title() for a in approaches])
        ax4.set_title('Performance Matrix: Approach × Drift Type')
        
        # Add text annotations
        for i in range(len(approaches)):
            for j in range(len(scenarios)):
                text = ax4.text(j, i, f'{performance_matrix[i][j]:.0f}%',
                               ha="center", va="center", fontweight='bold', 
                               color='white' if performance_matrix[i][j] < 50 else 'black')
        
        plt.colorbar(im, ax=ax4, label='Accuracy (%)')
    
    plt.tight_layout()
    
    plot_path = f'ddla_performance_comparison_{experiment_name.replace("-", "_")}.png'
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    return plot_path


# Updated experiment runner with proper visualizations
def run_complete_ddla_analysis_updated(X, y, trained_pipeline, drift_thresholds,
                                      experiment_name="telco-ddla",
                                      random_state=42):
    """
    Complete DDLA analysis with updated drift functions and proper visualizations.
    """
    import mlflow
    from sklearn.model_selection import train_test_split
    
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment(experiment_name)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
    
    all_approach_results = {}
    
    # Test Decision Tree DDLA
    print("Testing Decision Tree DDLA...")
    dt_ddla_info = identify_ddlas_decision_tree(trained_pipeline, X_test, y_test, random_state)
    dt_results = test_approach_across_drift_types('decision_tree', dt_ddla_info, X, y, 
                                                 trained_pipeline, drift_thresholds, random_state)
    all_approach_results['decision_tree'] = dt_results
    
    # Generate Decision Tree visualizations
    y_pred = trained_pipeline.predict(X_test)
    dt_viz_path = visualize_decision_tree_ddla_regions(dt_ddla_info, X_test, y_test, y_pred, experiment_name)
    
    # Test DBSCAN Error Clustering
    print("Testing DBSCAN Error Clustering...")
    dbscan_ddla_info = identify_ddlas_dbscan(trained_pipeline, X_test, y_test, random_state)
    dbscan_results = test_approach_across_drift_types('dbscan', dbscan_ddla_info, X, y,
                                                     trained_pipeline, drift_thresholds, random_state)
    all_approach_results['dbscan'] = dbscan_results
    
    # Generate DBSCAN visualizations
    dbscan_viz_path = visualize_clustering_ddla_regions(dbscan_ddla_info, X_test, y_test, y_pred, experiment_name)
    
    # Create comprehensive comparison
    comparison_path = create_ddla_performance_comparison_charts(all_approach_results, experiment_name)
    
    # Log summary to MLflow
    with mlflow.start_run(run_name='complete_ddla_analysis_summary'):
        mlflow.log_param('experiment_type', 'complete_ddla_comparison')
        mlflow.log_param('approaches_tested', len(all_approach_results))
        mlflow.log_param('drift_scenarios', 3)
        mlflow.log_param('thresholds_tested', len(drift_thresholds))
        
        # Log visualization artifacts
        if dt_viz_path:
            mlflow.log_artifact(dt_viz_path, artifact_path='visualizations')
        if dbscan_viz_path:
            mlflow.log_artifact(dbscan_viz_path, artifact_path='visualizations')
        if comparison_path:
            mlflow.log_artifact(comparison_path, artifact_path='visualizations')
    
    return all_approach_results


def test_approach_across_drift_types(approach_name, ddla_info, X, y, trained_pipeline, 
                                    drift_thresholds, random_state):
    from sklearn.model_selection import train_test_split
    
    drift_scenarios = [
        ('covariate_only', lambda X, y, t, s: simulate_drift(X, y, t, covariate_weight=1.0, concept_weight=0.0, random_state=s)),
        ('concept_only', lambda X, y, t, s: simulate_drift(X, y, t, covariate_weight=0.0, concept_weight=1.0, random_state=s)),
        ('combined_drift', lambda X, y, t, s: simulate_drifted_data(X, y, t, random_state=s))
    ]
    
    all_results = {}
    
    for scenario_name, drift_func in drift_scenarios:
        scenario_results = []
        
        for threshold in drift_thresholds:
            # Generate drift
            X_drifted, y_drifted, drift_info = drift_func(X, y, threshold, random_state)
            
            # Split drifted data
            _, X_test_drifted, _, y_test_drifted = train_test_split(
                X_drifted, y_drifted, test_size=0.2, random_state=random_state
            )
            
            # Apply appropriate detection method
            if approach_name == 'decision_tree':
                drift_detection = detect_harmful_drift_ddla(ddla_info, X_test_drifted, trained_pipeline)
            elif approach_name == 'dbscan':
                drift_detection = detect_harmful_drift_dbscan(ddla_info, X_test_drifted, trained_pipeline)
            else:
                continue
            
            # Calculate performance
            y_pred_drifted = trained_pipeline.predict(X_test_drifted)
            actual_accuracy = accuracy_score(y_test_drifted, y_pred_drifted)
            accuracy_drop = ddla_info['overall_accuracy'] - actual_accuracy
            significant_degradation = accuracy_drop > 0.05
            
            correct_decision = drift_detection['is_harmful_drift'] == significant_degradation
            
            result = {
                'threshold': threshold,
                f'{approach_name}_detected_harmful': drift_detection['is_harmful_drift'],
                'actually_needs_retraining': significant_degradation,
                f'{approach_name}_correct': correct_decision,
                'accuracy_drop_pct': (accuracy_drop / ddla_info['overall_accuracy']) * 100,
                'ratio_train': drift_detection.get('ratio_train', 0),
                'ratio_serving': drift_detection.get('ratio_serving', 0)
            }
            
            scenario_results.append(result)
        
        all_results[scenario_name] = scenario_results
    
    return all_results


In [20]:
# Run the complete DBSCAN experiment
dbscan_full_results, dbscan_info = run_dbscan_ddla_experiment_with_mlflow(
    X=X,
    y=y,
    trained_pipeline=pipeline,
    drift_thresholds=[0.0, 0.25, 0.5, 0.75, 1.0],
    experiment_name="telco-ddla-comparison",
    random_state=42
)

print("\n" + "="*80)
print("DBSCAN V. KMEANS")
print("="*80)

for scenario in ['covariate_only', 'concept_only', 'combined_drift']:
    if scenario in dbscan_full_results:
        results = dbscan_full_results[scenario]
        correct_dbscan = sum(r['dbscan_correct'] for r in results)
        total = len(results)
        dbscan_accuracy = (correct_dbscan / total) * 100
        
        # Your K-Means results for comparison
        kmeans_accuracies = {'covariate_only': 100.0, 'concept_only': 20.0, 'combined_drift': 20.0}
        kmeans_accuracy = kmeans_accuracies.get(scenario, 0)
        
        improvement = dbscan_accuracy - kmeans_accuracy
        
        print(f"\n{scenario.replace('_', ' ').title()}:")
        print(f"  K-Means Accuracy:  {kmeans_accuracy:.1f}%")
        print(f"  DBSCAN Accuracy:   {dbscan_accuracy:.1f}%")
        #print(f"  Improvement:       {improvement:+.1f}% {'🚀' if improvement > 0 else '😞' if improvement < 0 else '🤷'}")

print(f"\n All visualizations and metrics logged to MLflow experiment:")
print(f"   Experiment: telco-dbscan-vs-kmeans-ultimate-comparison")
print(f"   Artifacts: 3 comprehensive visualization plots")
print(f"   Metrics: Full performance comparison across all drift types")

 Starting DBSCAN-Based DDLA Experiment with Full MLflow Logging!

STEP 1: DBSCAN-BASED DDLA IDENTIFICATION WITH LOGGING
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.7935
  Overall error rate: 0.2065
 Focusing on 291 error samples out of 1409 total
  📊 Error samples have 57 features after preprocessing
 Finding optimal DBSCAN parameters for 291 error samples...
 K-distance analysis suggests eps = 1.415
 Grid searching DBSCAN parameters...
    eps=2.829, min_samples=8: 3 clusters, 123 noise, score=0.000
    eps=2.829, min_samples=9: 3 clusters, 133 noise, score=0.000
    eps=2.829, min_samples=10: 3 clusters, 137 noise, score=0.000
    eps=2.829, min_samples=11: 3 clusters, 143 noise, score=0.000
    eps=2.829, min_samples=12: 3 clusters, 144 noise, score=0.000
 Optimal DBSCAN: eps=2.829, min_samples=8, score=0.000
 DBSCAN found 3 distinct error patterns + 123 noise points
     DBSCAN DDLA found: Clust

We now that the choice of algorithms to detect DDLAs to "classify" benign and harmful drifts within models are what decide how well they accomplish they task. How do we proceed from here?

## A combination of both DTs and DBScan to identify DDLAs

In [21]:
'''
import numpy as np
import pandas as pd
from scipy.stats import ks_2samp
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def simple_drift_type_detection(X_baseline, X_serving, trained_pipeline):
    """
    Simple drift type detection using basic indicators.
    """
    # Count significant feature shifts
    significant_shifts = 0
    total_features = 0
    
    for col in X_baseline.select_dtypes(include=[np.number]).columns:
        if col in X_serving.columns:
            _, p_value = ks_2samp(X_baseline[col].dropna(), X_serving[col].dropna())
            total_features += 1
            if p_value < 0.05:
                significant_shifts += 1
    
    feature_drift_ratio = significant_shifts / max(1, total_features)
    
    # Check prediction pattern changes
    baseline_pred = trained_pipeline.predict(X_baseline)
    serving_pred = trained_pipeline.predict(X_serving)
    
    baseline_balance = baseline_pred.mean()
    serving_balance = serving_pred.mean()
    prediction_shift = abs(serving_balance - baseline_balance)
    
    # Simple decision logic
    if feature_drift_ratio > 0.4 and prediction_shift < 0.1:
        return 'covariate'
    elif feature_drift_ratio < 0.2 and prediction_shift > 0.15:
        return 'concept'
    else:
        return 'mixed'


def simple_adaptive_ddla(X_baseline, y_baseline, X_serving, trained_pipeline):
    """
    Simple adaptive DDLA system - chooses best method based on drift type.
    """
    # Initialize both DDLA approaches
    X_train, X_test, y_train, y_test = train_test_split(X_baseline, y_baseline, test_size=0.2, random_state=42)
    
    # Get DDLAs for both methods
    dt_ddla_info = identify_ddlas_decision_tree(
        trained_pipeline, X_test, y_test,
        max_depth_range=(3, 15),
        min_samples_leaf_range=(0.01, 0.15),
        random_state=42
    )
    
    dbscan_ddla_info = identify_ddlas_dbscan(trained_pipeline, X_test, y_test, random_state=42)
    
    # Detect drift type
    drift_type = simple_drift_type_detection(X_baseline, X_serving, trained_pipeline)
    
    # Select method based on our empirical findings
    if drift_type == 'covariate':
        method = 'decision_tree'
        result = detect_harmful_drift_ddla(dt_ddla_info, X_serving, trained_pipeline)
    else:  # concept or mixed - use DBSCAN
        method = 'dbscan'  
        result = detect_harmful_drift_dbscan(dbscan_ddla_info, X_serving, trained_pipeline)
    
    result['method_selected'] = method
    result['drift_type_detected'] = drift_type
    
    return result


def test_simple_adaptive_system(X, y, trained_pipeline, drift_thresholds, random_state=42):
    """
    Test the simple adaptive system.
    """
    print("Testing Simple Adaptive DDLA System")
    
    # Test scenarios
    scenarios = [
        ('covariate_only', lambda X, y, t, s: simulate_drift(X, y, t, covariate_weight=1.0, concept_weight=0.0, random_state=s)),
        ('concept_only', lambda X, y, t, s: simulate_drift(X, y, t, covariate_weight=0.0, concept_weight=1.0, random_state=s)),
        ('mixed_drift', lambda X, y, t, s: simulate_drift(X, y, t, covariate_weight=0.5, concept_weight=0.5, random_state=s))
    ]
    
    results = {}
    
    for scenario_name, drift_func in scenarios:
        print(f"\n{scenario_name.replace('_', ' ').title()}:")
        scenario_results = []
        
        for threshold in drift_thresholds:
            # Generate drift
            X_drifted, y_drifted, _ = drift_func(X, y, threshold, random_state)
            _, X_serving, _, y_serving = train_test_split(X_drifted, y_drifted, test_size=0.2, random_state=random_state)
            
            # Run adaptive system
            adaptive_result = simple_adaptive_ddla(X, y, X_serving, trained_pipeline)
            
            # Calculate ground truth
            actual_accuracy = accuracy_score(y_serving, trained_pipeline.predict(X_serving))
            _, X_baseline_test, _, y_baseline_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
            baseline_accuracy = accuracy_score(y_baseline_test, trained_pipeline.predict(X_baseline_test))
            
            needs_retraining = (baseline_accuracy - actual_accuracy) > 0.05
            adaptive_correct = adaptive_result['is_harmful_drift'] == needs_retraining
            
            result = {
                'threshold': threshold,
                'drift_type_detected': adaptive_result['drift_type_detected'],
                'method_selected': adaptive_result['method_selected'],
                'adaptive_decision': 'HARMFUL' if adaptive_result['is_harmful_drift'] else 'BENIGN',
                'ground_truth': 'YES' if needs_retraining else 'NO',
                'correct': adaptive_correct,
                'accuracy_drop_pct': ((baseline_accuracy - actual_accuracy) / baseline_accuracy) * 100
            }
            
            scenario_results.append(result)
            
            print(f"  Threshold {threshold:.2f}: {adaptive_result['drift_type_detected']} -> {adaptive_result['method_selected']} -> {'HARMFUL' if adaptive_result['is_harmful_drift'] else 'BENIGN'} ({'YES' if adaptive_correct else 'NO'})")
        
        results[scenario_name] = scenario_results
    
    return results


# Run simple test
simple_results = test_simple_adaptive_system(
    X=X, y=y, trained_pipeline=pipeline,
    drift_thresholds=[0.0, 0.25, 0.5, 0.75, 1.0],
    random_state=42
)

# Calculate and print summary
print("\n" + "="*60)
print("SIMPLE ADAPTIVE SYSTEM RESULTS")
print("="*60)

for scenario_name, scenario_results in simple_results.items():
    correct_count = sum(r['correct'] for r in scenario_results)
    total_count = len(scenario_results)
    accuracy_pct = (correct_count / total_count) * 100
    
    print(f"\n{scenario_name.replace('_', ' ').title()}:")
    print(f"  Accuracy: {correct_count}/{total_count} ({accuracy_pct:.1f}%)")
    
    # Show method selection pattern
    methods_used = [r['method_selected'] for r in scenario_results]
    dt_count = methods_used.count('decision_tree')
    dbscan_count = len(methods_used) - dt_count
    print(f"  Methods: {dt_count} Decision Tree, {dbscan_count} DBSCAN")

# Overall performance
all_correct = sum(sum(r['correct'] for r in results) for results in simple_results.values())
all_total = sum(len(results) for results in simple_results.values())
overall_accuracy = (all_correct / all_total) * 100

print(f"\nOverall Simple Adaptive Performance: {all_correct}/{all_total} ({overall_accuracy:.1f}%)")

# Compare to individual methods
print(f"\nComparison to Individual Methods:")
print(f"Decision Tree (Covariate): 100% accuracy")
print(f"DBSCAN (Concept): 80% accuracy") 
print(f"DBSCAN (Mixed): 80% accuracy")
print(f"Simple Adaptive System: {overall_accuracy:.1f}% accuracy")

'''

'\nimport numpy as np\nimport pandas as pd\nfrom scipy.stats import ks_2samp\nfrom sklearn.metrics import accuracy_score\nfrom sklearn.model_selection import train_test_split\n\ndef simple_drift_type_detection(X_baseline, X_serving, trained_pipeline):\n    """\n    Simple drift type detection using basic indicators.\n    """\n    # Count significant feature shifts\n    significant_shifts = 0\n    total_features = 0\n\n    for col in X_baseline.select_dtypes(include=[np.number]).columns:\n        if col in X_serving.columns:\n            _, p_value = ks_2samp(X_baseline[col].dropna(), X_serving[col].dropna())\n            total_features += 1\n            if p_value < 0.05:\n                significant_shifts += 1\n\n    feature_drift_ratio = significant_shifts / max(1, total_features)\n\n    # Check prediction pattern changes\n    baseline_pred = trained_pipeline.predict(X_baseline)\n    serving_pred = trained_pipeline.predict(X_serving)\n\n    baseline_balance = baseline_pred.mean()\n 

## Informing training with DDLAs as a retraining strategy

In [24]:
print("DDLA-ENHANCED PIPELINE vs BASELINE PIPELINE COMPARISON")
print("Comparing model performance under drift conditions")

# Set up MLflow experiment
mlflow.set_tracking_uri("http://localhost:5000") 
mlflow.set_experiment("ddla-enhanced-pipeline-comparison")

# Test parameters
drift_thresholds = [0.0, 0.25, 0.5, 0.75, 1.0]
drift_scenarios = [
    ('covariate_only', simulate_covariate_drift_only),
    ('concept_only', simulate_concept_drift_only), 
    ('combined_drift', simulate_drifted_data)
]

# Results storage
all_results = []

print("\n" + "="*80)
print("PIPELINE PERFORMANCE COMPARISON UNDER DRIFT")
print("="*80)

# Get baseline DDLAs once for efficiency
X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(X, y, test_size=0.2, random_state=42)

baseline_dt_ddla_info = identify_ddlas_decision_tree(
    pipeline, X_test_base, y_test_base,
    max_depth_range=(3, 15), min_samples_leaf_range=(0.01, 0.15), random_state=42
)

baseline_dbscan_ddla_info = identify_ddlas_dbscan(pipeline, X_test_base, y_test_base, random_state=42)

print(f"Baseline DDLAs identified: {len(baseline_dt_ddla_info['ddlas'])} DT, {len(baseline_dbscan_ddla_info['ddlas'])} DBSCAN")

for scenario_name, drift_func in drift_scenarios:
    print(f"\nTesting {scenario_name.replace('_', ' ').title()} Drift Scenario")
    
    for threshold in drift_thresholds:
        print(f"  Drift Threshold: {threshold:.2f}")
        
        # Generate drifted data
        X_drifted, y_drifted, drift_info = drift_func(X, y, drift_threshold=threshold, random_state=42)
        X_train_drift, X_test_drift, y_train_drift, y_test_drift = train_test_split(
            X_drifted, y_drifted, test_size=0.2, random_state=42
        )
        
        with mlflow.start_run(run_name=f'{scenario_name}_threshold_{threshold}_comparison'):
            
            # BASELINE PIPELINE: Test original pipeline on drifted data
            baseline_accuracy_no_drift = accuracy_score(y_test_base, pipeline.predict(X_test_base))
            baseline_accuracy_with_drift = accuracy_score(y_test_drift, pipeline.predict(X_test_drift))
            baseline_drift_impact = baseline_accuracy_no_drift - baseline_accuracy_with_drift
            
            # DDLA-ENHANCED PIPELINE: Create enhanced pipeline using DDLA insights
            
            # Step 1: Identify risky patterns in drifted training data
            risky_sample_indices = []
            
            # Use Decision Tree DDLAs to identify risky training samples
            if len(baseline_dt_ddla_info['ddlas']) > 0:
                X_train_drift_preprocessed = pipeline.named_steps['preprocessor'].transform(X_train_drift)
                X_train_drift_preprocessed_df = pd.DataFrame(
                    X_train_drift_preprocessed,
                    columns=baseline_dt_ddla_info['feature_names'],
                    index=X_train_drift.index
                )
                train_leaf_ids = baseline_dt_ddla_info['decision_tree'].apply(X_train_drift_preprocessed_df)
                ddla_leaf_ids = {ddla['leaf_id'] for ddla in baseline_dt_ddla_info['ddlas']}
                dt_risky_mask = np.array([leaf_id in ddla_leaf_ids for leaf_id in train_leaf_ids])
                risky_sample_indices.extend(X_train_drift.index[dt_risky_mask].tolist())

            risky_sample_indices = list(set(risky_sample_indices))
            
            # Use DBSCAN DDLAs to identify additional risky samples
            if len(baseline_dbscan_ddla_info['ddlas']) > 0 and hasattr(baseline_dbscan_ddla_info['error_clusters'], 'predict'):
                try:
                    X_train_drift_preprocessed_cluster = baseline_dbscan_ddla_info['error_clusters'].named_steps['preprocessor'].transform(X_train_drift)
                    train_cluster_ids = baseline_dbscan_ddla_info['error_clusters'].predict(X_train_drift_preprocessed_cluster)
                    ddla_cluster_ids = {ddla['cluster_id'] for ddla in baseline_dbscan_ddla_info['ddlas']}
                    dbscan_risky_mask = np.array([cluster_id in ddla_cluster_ids for cluster_id in train_cluster_ids])
                    risky_sample_indices.extend(X_train_drift.index[dbscan_risky_mask].tolist())
                except:
                    pass
            
            risky_sample_indices = list(set(risky_sample_indices))
            
            # Step 2: Create DDLA-enhanced training set
            if len(risky_sample_indices) > 0:
                # Oversample the risky samples (samples that fall into DDLA patterns)
                X_risky = X_train_drift.loc[risky_sample_indices]
                y_risky = y_train_drift.loc[risky_sample_indices]
                
                # Create enhanced training set with 2x oversampling of risky samples
                from sklearn.utils import resample
                X_risky_oversampled = resample(X_risky, n_samples=len(X_risky)*2, random_state=42)
                y_risky_oversampled = resample(y_risky, n_samples=len(y_risky)*2, random_state=42)
                
                # Combine with original training data
                X_train_enhanced = pd.concat([X_train_drift, X_risky_oversampled])
                y_train_enhanced = pd.concat([y_train_drift, y_risky_oversampled])
                
                # Train enhanced pipeline
                from sklearn.base import clone
                enhanced_pipeline = clone(pipeline)
                enhanced_pipeline.fit(X_train_enhanced, y_train_enhanced)
                
                # Test enhanced pipeline
                enhanced_accuracy_no_drift = accuracy_score(y_test_base, enhanced_pipeline.predict(X_test_base))
                enhanced_accuracy_with_drift = accuracy_score(y_test_drift, enhanced_pipeline.predict(X_test_drift))
                enhanced_drift_impact = enhanced_accuracy_no_drift - enhanced_accuracy_with_drift
                
                print(f"    Enhanced training samples: {len(X_train_drift)} -> {len(X_train_enhanced)}")
                
            else:
                # No risky samples found - enhanced pipeline is same as baseline
                enhanced_pipeline = pipeline
                enhanced_accuracy_no_drift = baseline_accuracy_no_drift
                enhanced_accuracy_with_drift = baseline_accuracy_with_drift
                enhanced_drift_impact = baseline_drift_impact
                X_train_enhanced = X_train_drift
                print(f"    No risky samples identified - using baseline pipeline")
            
            # Calculate drift mitigation effectiveness
            drift_mitigation_improvement = baseline_drift_impact - enhanced_drift_impact
            drift_mitigation_pct = (drift_mitigation_improvement / abs(baseline_drift_impact)) * 100 if baseline_drift_impact != 0 else 0
            
            # Performance comparison
            print(f"    Baseline Pipeline:")
            print(f"      No Drift: {baseline_accuracy_no_drift:.3f}")
            print(f"      With Drift: {baseline_accuracy_with_drift:.3f} (impact: {baseline_drift_impact:+.3f})")
            print(f"    DDLA-Enhanced Pipeline:")
            print(f"      No Drift: {enhanced_accuracy_no_drift:.3f}")  
            print(f"      With Drift: {enhanced_accuracy_with_drift:.3f} (impact: {enhanced_drift_impact:+.3f})")
            print(f"    Drift Mitigation: {drift_mitigation_improvement:+.3f} ({drift_mitigation_pct:+.1f}%)")
            
            # Log comprehensive results to MLflow
            mlflow.log_param('scenario', scenario_name)
            mlflow.log_param('drift_threshold', threshold)
            mlflow.log_param('risky_samples_identified', len(risky_sample_indices))
            mlflow.log_param('baseline_training_size', len(X_train_drift))
            mlflow.log_param('enhanced_training_size', len(X_train_enhanced))
            mlflow.log_param('dt_ddlas_available', len(baseline_dt_ddla_info['ddlas']))
            mlflow.log_param('dbscan_ddlas_available', len(baseline_dbscan_ddla_info['ddlas']))
            
            # Baseline pipeline metrics
            mlflow.log_metric('baseline_accuracy_no_drift', baseline_accuracy_no_drift)
            mlflow.log_metric('baseline_accuracy_with_drift', baseline_accuracy_with_drift)
            mlflow.log_metric('baseline_drift_impact', baseline_drift_impact)
            
            # Enhanced pipeline metrics
            mlflow.log_metric('enhanced_accuracy_no_drift', enhanced_accuracy_no_drift)
            mlflow.log_metric('enhanced_accuracy_with_drift', enhanced_accuracy_with_drift)
            mlflow.log_metric('enhanced_drift_impact', enhanced_drift_impact)
            
            # Comparison metrics
            mlflow.log_metric('drift_mitigation_improvement', drift_mitigation_improvement)
            mlflow.log_metric('drift_mitigation_improvement_pct', drift_mitigation_pct)
            mlflow.log_metric('accuracy_improvement_under_drift', enhanced_accuracy_with_drift - baseline_accuracy_with_drift)
            
            # Store result for visualization
            all_results.append({
                'scenario': scenario_name,
                'threshold': threshold,
                'baseline_no_drift': baseline_accuracy_no_drift,
                'baseline_with_drift': baseline_accuracy_with_drift,
                'baseline_impact': baseline_drift_impact,
                'enhanced_no_drift': enhanced_accuracy_no_drift,
                'enhanced_with_drift': enhanced_accuracy_with_drift,
                'enhanced_impact': enhanced_drift_impact,
                'mitigation_improvement': drift_mitigation_improvement,
                'risky_samples': len(risky_sample_indices)
            })

# Create comprehensive visualizations
results_df = pd.DataFrame(all_results)

# Visualization 1: Drift Impact Comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Drift Impact by Scenario
ax1 = axes[0,0]
scenario_groups = results_df.groupby('scenario')
scenarios = list(scenario_groups.groups.keys())
x_pos = np.arange(len(scenarios))

baseline_impacts = [scenario_groups.get_group(s)['baseline_impact'].mean() for s in scenarios]
enhanced_impacts = [scenario_groups.get_group(s)['enhanced_impact'].mean() for s in scenarios]

width = 0.35
bars1 = ax1.bar(x_pos - width/2, baseline_impacts, width, label='Baseline Pipeline', color='#e74c3c', alpha=0.8)
bars2 = ax1.bar(x_pos + width/2, enhanced_impacts, width, label='DDLA-Enhanced Pipeline', color='#2ecc71', alpha=0.8)

ax1.set_title('Average Drift Impact: Baseline vs DDLA-Enhanced', fontweight='bold')
ax1.set_xlabel('Drift Scenario')
ax1.set_ylabel('Performance Drop (Lower is Better)')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([s.replace('_', ' ').title() for s in scenarios])
ax1.legend()
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{height:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)

# Plot 2: Accuracy Under Drift by Threshold
ax2 = axes[0,1]
for scenario in scenarios:
    scenario_data = results_df[results_df['scenario'] == scenario]
    ax2.plot(scenario_data['threshold'], scenario_data['baseline_with_drift'], 
            'o--', linewidth=2, alpha=0.7, label=f'Baseline {scenario.replace("_", " ").title()}')
    ax2.plot(scenario_data['threshold'], scenario_data['enhanced_with_drift'],
            'o-', linewidth=2, label=f'Enhanced {scenario.replace("_", " ").title()}')

ax2.set_title('Pipeline Accuracy Under Drift', fontweight='bold')
ax2.set_xlabel('Drift Threshold')
ax2.set_ylabel('Accuracy')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(alpha=0.3)

# Plot 3: Drift Mitigation Effectiveness
ax3 = axes[1,0]
mitigation_by_scenario = results_df.groupby('scenario')['mitigation_improvement'].mean()
colors = ['#3498db', '#e67e22', '#9b59b6']
bars = ax3.bar(range(len(mitigation_by_scenario)), mitigation_by_scenario.values, 
              color=colors, alpha=0.8)

ax3.set_title('Drift Mitigation Effectiveness', fontweight='bold')
ax3.set_xlabel('Drift Scenario')
ax3.set_ylabel('Mitigation Improvement (Higher is Better)')
ax3.set_xticks(range(len(mitigation_by_scenario)))
ax3.set_xticklabels([s.replace('_', ' ').title() for s in mitigation_by_scenario.index])
ax3.grid(axis='y', alpha=0.3)
ax3.axhline(y=0, color='black', linestyle='--', alpha=0.7)

# Add value labels
for bar, value in zip(bars, mitigation_by_scenario.values):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + (0.005 if height >= 0 else -0.010),
            f'{value:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', 
            fontweight='bold')

# Plot 4: Performance Recovery Rate
ax4 = axes[1,1]
# Calculate how much of the original performance is recovered
recovery_rates = []
scenario_labels = []

for scenario in scenarios:
    scenario_data = results_df[results_df['scenario'] == scenario]
    
    # Average recovery rate = (enhanced_performance - baseline_performance) / baseline_no_drift_performance
    avg_baseline_no_drift = scenario_data['baseline_no_drift'].mean()
    avg_baseline_with_drift = scenario_data['baseline_with_drift'].mean()
    avg_enhanced_with_drift = scenario_data['enhanced_with_drift'].mean()
    
    performance_loss = avg_baseline_no_drift - avg_baseline_with_drift
    performance_recovery = avg_enhanced_with_drift - avg_baseline_with_drift
    
    recovery_rate = (performance_recovery / performance_loss) * 100 if performance_loss > 0 else 0
    recovery_rates.append(recovery_rate)
    scenario_labels.append(scenario.replace('_', ' ').title())

bars = ax4.bar(range(len(recovery_rates)), recovery_rates, color=colors, alpha=0.8)
ax4.set_title('Performance Recovery Rate', fontweight='bold')
ax4.set_xlabel('Drift Scenario')
ax4.set_ylabel('Recovery Rate (%)')
ax4.set_xticks(range(len(scenario_labels)))
ax4.set_xticklabels(scenario_labels)
ax4.grid(axis='y', alpha=0.3)
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.7)
ax4.axhline(y=100, color='green', linestyle=':', alpha=0.7, label='Full Recovery')

# Add value labels
for bar, value in zip(bars, recovery_rates):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + (2 if height >= 0 else -5),
            f'{value:.0f}%', ha='center', va='bottom' if height >= 0 else 'top', 
            fontweight='bold')

plt.suptitle('DDLA-Enhanced Pipeline Performance Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save and log visualization
viz_path = 'ddla_enhanced_pipeline_comparison.png'
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
plt.close()

# Log visualization as artifact
with mlflow.start_run(run_name='visualization_summary'):
    mlflow.log_artifact(viz_path, artifact_path='pipeline_comparison_plots')

# Summary statistics visualization
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Create summary table visualization
summary_data = []
for scenario in scenarios:
    scenario_data = results_df[results_df['scenario'] == scenario]
    
    avg_baseline_impact = scenario_data['baseline_impact'].mean()
    avg_enhanced_impact = scenario_data['enhanced_impact'].mean()
    avg_mitigation = scenario_data['mitigation_improvement'].mean()
    avg_risky_samples = scenario_data['risky_samples'].mean()
    
    summary_data.append([
        scenario.replace('_', ' ').title(),
        f"{avg_baseline_impact:.3f}",
        f"{avg_enhanced_impact:.3f}", 
        f"{avg_mitigation:+.3f}",
        f"{avg_risky_samples:.0f}"
    ])

# Create table
table = ax.table(cellText=summary_data,
                colLabels=['Drift Type', 'Baseline Impact', 'Enhanced Impact', 'Mitigation', 'Avg Risky Samples'],
                cellLoc='center',
                loc='center',
                colColours=['#ecf0f1']*5)

table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 2)

# Style the table
for i in range(len(summary_data) + 1):
    for j in range(5):
        cell = table[(i, j)]
        if i == 0:  # Header row
            cell.set_facecolor('#34495e')
            cell.set_text_props(weight='bold', color='white')
        else:
            cell.set_facecolor('#ecf0f1' if i % 2 == 0 else '#ffffff')

ax.set_title('DDLA-Enhanced Pipeline Summary Statistics', fontsize=14, fontweight='bold', pad=20)
ax.axis('off')

# Save summary table
summary_viz_path = 'ddla_pipeline_summary_table.png'
plt.savefig(summary_viz_path, dpi=300, bbox_inches='tight')
plt.close()

# Log summary visualization
with mlflow.start_run(run_name='summary_table'):
    mlflow.log_artifact(summary_viz_path, artifact_path='pipeline_comparison_plots')

# Print final analysis
print("\n" + "="*80)
print("FINAL ANALYSIS: DDLA-ENHANCED PIPELINE EFFECTIVENESS")
print("="*80)

overall_baseline_impact = results_df['baseline_impact'].mean()
overall_enhanced_impact = results_df['enhanced_impact'].mean()  
overall_mitigation = results_df['mitigation_improvement'].mean()

print(f"Overall Results:")
print(f"  Baseline Pipeline Drift Impact:  {overall_baseline_impact:.3f}")
print(f"  Enhanced Pipeline Drift Impact:  {overall_enhanced_impact:.3f}")
print(f"  Average Mitigation Improvement:  {overall_mitigation:+.3f}")

# Count positive mitigations
positive_mitigations = sum(1 for r in all_results if r['mitigation_improvement'] > 0.001)
total_tests = len(all_results)
success_rate = positive_mitigations / total_tests

print(f"  Success Rate: {positive_mitigations}/{total_tests} ({success_rate:.1%})")

print(f"\nBy Drift Type:")
for scenario in scenarios:
    scenario_data = results_df[results_df['scenario'] == scenario]
    scenario_mitigation = scenario_data['mitigation_improvement'].mean()
    scenario_positives = sum(scenario_data['mitigation_improvement'] > 0.001)
    scenario_total = len(scenario_data)
    
    print(f"  {scenario.replace('_', ' ').title():<15}: {scenario_mitigation:+.3f} avg ({scenario_positives}/{scenario_total} positive)")

if overall_mitigation > 0.01:
    print(f"\nConclusion: DDLA-enhanced pipeline significantly mitigates drift impact!")
elif overall_mitigation > 0:
    print(f"\nConclusion: DDLA-enhanced pipeline provides modest drift mitigation.")
else:
    print(f"\nConclusion: DDLA-enhanced pipeline does not improve drift robustness.")

print(f"\nAll visualizations and detailed logs saved to MLflow experiment.")

DDLA-ENHANCED PIPELINE vs BASELINE PIPELINE COMPARISON
Comparing model performance under drift conditions

PIPELINE PERFORMANCE COMPARISON UNDER DRIFT
Identifying DDLAs with tree based approach
Overall model accuracy: 0.7935
  Overall incorrect prediction rate: 0.2065
  Best decision tree params: {'max_depth': 6, 'min_samples_leaf': 14}
  Decision tree F1 score: 0.4735
 Found 13 DDLAs out of 32 total leaf nodes
 DDLA coverage: 710/1409 samples (0.504)
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.7935
  Overall error rate: 0.2065
 Focusing on 291 error samples out of 1409 total
  📊 Error samples have 57 features after preprocessing
 Finding optimal DBSCAN parameters for 291 error samples...
 K-distance analysis suggests eps = 1.415
 Grid searching DBSCAN parameters...
    eps=2.829, min_samples=8: 3 clusters, 123 noise, score=0.000
    eps=2.829, min_samples=9: 3 clusters, 133 noise, score=0.000
    e

In [26]:
print("CORRECTED DDLA-ENHANCED PIPELINE: USING BOTH DT AND DBSCAN")
print("Leveraging both approaches to identify comprehensive risky patterns")

# Set up experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("ddla-enhanced-pipeline-comparison")

drift_thresholds = [0.0, 0.25, 0.5, 0.75, 1.0]
drift_scenarios = [
    ('covariate_only', simulate_covariate_drift_only),
    ('concept_only', simulate_concept_drift_only),
    ('combined_drift', simulate_drifted_data)
]

# Results storage
comprehensive_results = []

print("\n" + "="*80)
print("COMPREHENSIVE DDLA-ENHANCED PIPELINE (DT + DBSCAN)")
print("="*80)

for scenario_name, drift_func in drift_scenarios:
    print(f"\nTesting {scenario_name.replace('_', ' ').title()}")
    
    for threshold in drift_thresholds:
        print(f"  Drift Threshold: {threshold:.2f}")
        
        # Generate drifted data
        X_drifted, y_drifted, drift_info = drift_func(X, y, drift_threshold=threshold, random_state=42)
        X_train_drift, X_test_drift, y_train_drift, y_test_drift = train_test_split(
            X_drifted, y_drifted, test_size=0.2, random_state=42
        )
        
        with mlflow.start_run(run_name=f'dualDDLA_{scenario_name}_{threshold}_drift'):
            
            # BASELINE: Original pipeline
            baseline_accuracy_clean = accuracy_score(y_test_base, pipeline.predict(X_test_base))
            baseline_accuracy_drift = accuracy_score(y_test_drift, pipeline.predict(X_test_drift))
            baseline_drift_impact = baseline_accuracy_clean - baseline_accuracy_drift
            
            # ENHANCED: Train temporary model on drift data to identify DDLAs
            from sklearn.base import clone
            temp_pipeline = clone(pipeline)
            temp_pipeline.fit(X_train_drift, y_train_drift)
            
            # Get DDLAs from BOTH methods on drifted data
            print("    Identifying drift-specific DDLAs...")
            
            # Decision Tree DDLAs
            drift_dt_ddla_info = identify_ddlas_decision_tree(
                temp_pipeline, X_test_drift, y_test_drift,
                max_depth_range=(3, 15), min_samples_leaf_range=(0.01, 0.15), random_state=42
            )
            
            # DBSCAN DDLAs  
            drift_dbscan_ddla_info = identify_ddlas_dbscan(temp_pipeline, X_test_drift, y_test_drift, random_state=42)
            
            print(f"    Found {len(drift_dt_ddla_info['ddlas'])} DT DDLAs, {len(drift_dbscan_ddla_info['ddlas'])} DBSCAN DDLAs")
            
            # Collect risky samples from BOTH methods
            all_risky_indices = []
            
            # Decision Tree risky samples
            if len(drift_dt_ddla_info['ddlas']) > 0:
                X_train_preprocessed = temp_pipeline.named_steps['preprocessor'].transform(X_train_drift)
                X_train_preprocessed_df = pd.DataFrame(
                    X_train_preprocessed,
                    columns=drift_dt_ddla_info['feature_names'],
                    index=X_train_drift.index
                )
                
                train_leaf_ids = drift_dt_ddla_info['decision_tree'].apply(X_train_preprocessed_df)
                dt_ddla_leaf_ids = {ddla['leaf_id'] for ddla in drift_dt_ddla_info['ddlas']}
                dt_risky_mask = np.array([leaf_id in dt_ddla_leaf_ids for leaf_id in train_leaf_ids])
                dt_risky_indices = X_train_drift.index[dt_risky_mask].tolist()
                all_risky_indices.extend(dt_risky_indices)
                
                print(f"    DT identified {len(dt_risky_indices)} risky samples")
            
            # DBSCAN risky samples
            if len(drift_dbscan_ddla_info['ddlas']) > 0 and hasattr(drift_dbscan_ddla_info['error_clusters'], 'predict'):
                try:
                    X_train_cluster_preprocessed = drift_dbscan_ddla_info['error_clusters'].named_steps['preprocessor'].transform(X_train_drift)
                    train_cluster_ids = drift_dbscan_ddla_info['error_clusters'].predict(X_train_cluster_preprocessed)
                    dbscan_ddla_cluster_ids = {ddla['cluster_id'] for ddla in drift_dbscan_ddla_info['ddlas']}
                    dbscan_risky_mask = np.array([cluster_id in dbscan_ddla_cluster_ids for cluster_id in train_cluster_ids])
                    dbscan_risky_indices = X_train_drift.index[dbscan_risky_mask].tolist()
                    all_risky_indices.extend(dbscan_risky_indices)
                    
                    print(f"    DBSCAN identified {len(dbscan_risky_indices)} risky samples")
                except Exception as e:
                    print(f"    DBSCAN risky sample identification failed: {e}")
                    dbscan_risky_indices = []
            else:
                dbscan_risky_indices = []
            
            # Combine and deduplicate risky samples
            unique_risky_indices = list(set(all_risky_indices))
            print(f"    Total unique risky samples: {len(unique_risky_indices)}")
            
            # Create enhanced training set using samples from both methods
            if len(unique_risky_indices) > 0:
                X_risky_combined = X_train_drift.loc[unique_risky_indices]
                y_risky_combined = y_train_drift.loc[unique_risky_indices]
                
                # Oversample the combined risky samples
                from sklearn.utils import resample
                oversample_factor = 3  # 3x oversampling
                X_risky_oversampled = resample(X_risky_combined, n_samples=len(X_risky_combined)*oversample_factor, random_state=42)
                y_risky_oversampled = resample(y_risky_combined, n_samples=len(y_risky_combined)*oversample_factor, random_state=42)
                
                # Final enhanced training set
                X_train_final = pd.concat([X_train_drift, X_risky_oversampled])
                y_train_final = pd.concat([y_train_drift, y_risky_oversampled])
                
                # Train final enhanced pipeline
                final_enhanced_pipeline = clone(pipeline)
                final_enhanced_pipeline.fit(X_train_final, y_train_final)
                
                # Test final performance
                final_accuracy_clean = accuracy_score(y_test_base, final_enhanced_pipeline.predict(X_test_base))
                final_accuracy_drift = accuracy_score(y_test_drift, final_enhanced_pipeline.predict(X_test_drift))
                final_drift_impact = final_accuracy_clean - final_accuracy_drift
                
                enhancement_applied = True
                final_training_size = len(X_train_final)
                
                print(f"    Final enhanced training: {len(X_train_drift)} -> {final_training_size} samples")
                
            else:
                # No risky samples found - use temp pipeline
                final_enhanced_pipeline = temp_pipeline
                final_accuracy_clean = accuracy_score(y_test_base, final_enhanced_pipeline.predict(X_test_base))
                final_accuracy_drift = accuracy_score(y_test_drift, final_enhanced_pipeline.predict(X_test_drift))
                final_drift_impact = final_accuracy_clean - final_accuracy_drift
                enhancement_applied = False
                final_training_size = len(X_train_drift)
            
            # Calculate comprehensive metrics
            drift_mitigation = baseline_drift_impact - final_drift_impact
            performance_improvement = final_accuracy_drift - baseline_accuracy_drift
            
            print(f"    Final Results:")
            print(f"      Baseline Pipeline: {baseline_accuracy_drift:.3f} (impact: {baseline_drift_impact:+.3f})")
            print(f"      Enhanced Pipeline: {final_accuracy_drift:.3f} (impact: {final_drift_impact:+.3f})")
            print(f"      Performance Gain: {performance_improvement:+.3f}")
            print(f"      Drift Mitigation: {drift_mitigation:+.3f}")
            
            # Log comprehensive results
            mlflow.log_param('scenario', scenario_name)
            mlflow.log_param('drift_threshold', threshold)
            mlflow.log_param('dt_ddlas_in_drift', len(drift_dt_ddla_info['ddlas']))
            mlflow.log_param('dbscan_ddlas_in_drift', len(drift_dbscan_ddla_info['ddlas']))
            mlflow.log_param('dt_risky_samples', len(dt_risky_indices) if 'dt_risky_indices' in locals() else 0)
            mlflow.log_param('dbscan_risky_samples', len(dbscan_risky_indices) if 'dbscan_risky_indices' in locals() else 0)
            mlflow.log_param('total_unique_risky_samples', len(unique_risky_indices))
            mlflow.log_param('enhancement_applied', enhancement_applied)
            mlflow.log_param('final_training_size', final_training_size)
            mlflow.log_param('oversample_factor', 3)
            
            # Performance metrics
            mlflow.log_metric('baseline_accuracy_clean', baseline_accuracy_clean)
            mlflow.log_metric('baseline_accuracy_drift', baseline_accuracy_drift)
            mlflow.log_metric('baseline_drift_impact', baseline_drift_impact)
            mlflow.log_metric('enhanced_accuracy_clean', final_accuracy_clean)
            mlflow.log_metric('enhanced_accuracy_drift', final_accuracy_drift)
            mlflow.log_metric('enhanced_drift_impact', final_drift_impact)
            mlflow.log_metric('drift_mitigation', drift_mitigation)
            mlflow.log_metric('performance_improvement_under_drift', performance_improvement)
            
            comprehensive_results.append({
                'scenario': scenario_name,
                'threshold': threshold,
                'baseline_impact': baseline_drift_impact,
                'enhanced_impact': final_drift_impact,
                'mitigation': drift_mitigation,
                'performance_gain': performance_improvement,
                'dt_risky_samples': len(dt_risky_indices) if 'dt_risky_indices' in locals() else 0,
                'dbscan_risky_samples': len(dbscan_risky_indices) if 'dbscan_risky_indices' in locals() else 0,
                'total_risky_samples': len(unique_risky_indices),
                'enhancement_applied': enhancement_applied
            })

# Create visualization showing both methods' contributions
comprehensive_df = pd.DataFrame(comprehensive_results)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Method contribution analysis
ax1 = axes[0,0]
dt_samples = comprehensive_df.groupby('scenario')['dt_risky_samples'].mean()
dbscan_samples = comprehensive_df.groupby('scenario')['dbscan_risky_samples'].mean()

x_pos = np.arange(len(dt_samples))
width = 0.35

bars1 = ax1.bar(x_pos - width/2, dt_samples.values, width, label='Decision Tree DDLAs', color='#3498db', alpha=0.8)
bars2 = ax1.bar(x_pos + width/2, dbscan_samples.values, width, label='DBSCAN DDLAs', color='#e67e22', alpha=0.8)

ax1.set_title('Risky Samples Identified by Method', fontweight='bold')
ax1.set_xlabel('Drift Scenario')
ax1.set_ylabel('Average Risky Samples Found')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([s.replace('_', ' ').title() for s in dt_samples.index])
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Performance improvement by scenario
ax2 = axes[0,1]
perf_by_scenario = comprehensive_df.groupby('scenario')['performance_gain'].mean()
colors = ['#2ecc71', '#e74c3c', '#9b59b6']
bars = ax2.bar(range(len(perf_by_scenario)), perf_by_scenario.values, color=colors, alpha=0.8)

ax2.set_title('Performance Improvement by Drift Type', fontweight='bold')
ax2.set_xlabel('Drift Scenario')
ax2.set_ylabel('Performance Gain Under Drift')
ax2.set_xticks(range(len(perf_by_scenario)))
ax2.set_xticklabels([s.replace('_', ' ').title() for s in perf_by_scenario.index])
ax2.grid(axis='y', alpha=0.3)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.7)

for bar, value in zip(bars, perf_by_scenario.values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + (0.005 if height >= 0 else -0.010),
            f'{value:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')

# Plot 3: Method overlap analysis
ax3 = axes[1,0]
overlap_data = []
for scenario in comprehensive_df['scenario'].unique():
    scenario_data = comprehensive_df[comprehensive_df['scenario'] == scenario]
    avg_dt = scenario_data['dt_risky_samples'].mean()
    avg_dbscan = scenario_data['dbscan_risky_samples'].mean() 
    avg_total = scenario_data['total_risky_samples'].mean()
    
    # Estimate overlap (total < dt + dbscan means overlap exists)
    estimated_overlap = max(0, avg_dt + avg_dbscan - avg_total)
    overlap_data.append(estimated_overlap)

ax3.bar(range(len(comprehensive_df['scenario'].unique())), overlap_data, color=colors, alpha=0.8)
ax3.set_title('Estimated Method Overlap', fontweight='bold')
ax3.set_xlabel('Drift Scenario')
ax3.set_ylabel('Overlapping Risky Samples')
ax3.set_xticks(range(len(comprehensive_df['scenario'].unique())))
ax3.set_xticklabels([s.replace('_', ' ').title() for s in comprehensive_df['scenario'].unique()])
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Enhancement effectiveness
ax4 = axes[1,1]
enhancement_rates = []
scenario_labels = []

for scenario in comprehensive_df['scenario'].unique():
    scenario_data = comprehensive_df[comprehensive_df['scenario'] == scenario]
    enhanced_count = sum(scenario_data['enhancement_applied'])
    total_count = len(scenario_data)
    rate = enhanced_count / total_count * 100
    enhancement_rates.append(rate)
    scenario_labels.append(scenario.replace('_', ' ').title())

bars = ax4.bar(range(len(enhancement_rates)), enhancement_rates, color=colors, alpha=0.8)
ax4.set_title('Enhancement Application Rate', fontweight='bold')
ax4.set_xlabel('Drift Scenario') 
ax4.set_ylabel('Enhancement Applied (%)')
ax4.set_xticks(range(len(scenario_labels)))
ax4.set_xticklabels(scenario_labels)
ax4.grid(axis='y', alpha=0.3)

for bar, value in zip(bars, enhancement_rates):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{value:.0f}%', ha='center', va='bottom', fontweight='bold')

plt.suptitle('Comprehensive DDLA-Enhanced Pipeline Analysis (DT + DBSCAN)', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save visualization
viz_path = 'comprehensive_ddla_pipeline_analysis.png'
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
plt.close()

# Log visualization
with mlflow.start_run(run_name='comprehensive_visualization'):
    mlflow.log_artifact(viz_path, artifact_path='comprehensive_ddla_plots')

# Final results
print("\n" + "="*80) 
print("COMPREHENSIVE DDLA-ENHANCED RESULTS (DT + DBSCAN)")
print("="*80)

overall_mitigation = comprehensive_df['mitigation'].mean()
overall_improvement = comprehensive_df['performance_gain'].mean()
overall_dt_samples = comprehensive_df['dt_risky_samples'].mean()
overall_dbscan_samples = comprehensive_df['dbscan_risky_samples'].mean()
overall_total_samples = comprehensive_df['total_risky_samples'].mean()

print(f"Overall Performance:")
print(f"  Average Drift Mitigation: {overall_mitigation:+.3f}")
print(f"  Average Performance Improvement: {overall_improvement:+.3f}")

print(f"\nMethod Contributions:")
print(f"  DT Average Risky Samples: {overall_dt_samples:.1f}")
print(f"  DBSCAN Average Risky Samples: {overall_dbscan_samples:.1f}")
print(f"  Total Unique Risky Samples: {overall_total_samples:.1f}")
print(f"  Estimated Overlap: {max(0, overall_dt_samples + overall_dbscan_samples - overall_total_samples):.1f}")

positive_improvements = sum(1 for r in comprehensive_results if r['performance_gain'] > 0)
print(f"  Success Rate: {positive_improvements}/{len(comprehensive_results)} ({positive_improvements/len(comprehensive_results):.1%})")

print(f"\nBy Drift Type:")
for scenario in comprehensive_df['scenario'].unique():
    scenario_data = comprehensive_df[comprehensive_df['scenario'] == scenario]
    scenario_improvement = scenario_data['performance_gain'].mean()
    scenario_mitigation = scenario_data['mitigation'].mean()
    scenario_dt = scenario_data['dt_risky_samples'].mean()
    scenario_dbscan = scenario_data['dbscan_risky_samples'].mean()
    
    print(f"  {scenario.replace('_', ' ').title():<15}: Performance {scenario_improvement:+.3f}, Mitigation {scenario_mitigation:+.3f}")
    print(f"    {'':>17} DT samples: {scenario_dt:.1f}, DBSCAN samples: {scenario_dbscan:.1f}")

if overall_improvement > 0.01:
    print(f"\nConclusion: Comprehensive DDLA approach (DT + DBSCAN) significantly improves drift robustness!")
else:
    print(f"\nConclusion: Combined approach shows limited improvement - may need different strategy.")


CORRECTED DDLA-ENHANCED PIPELINE: USING BOTH DT AND DBSCAN
Leveraging both approaches to identify comprehensive risky patterns

COMPREHENSIVE DDLA-ENHANCED PIPELINE (DT + DBSCAN)

Testing Covariate Only
  Drift Threshold: 0.00
Simulating covariate drift with threshold: 0.00
Applying to 8 numeric and 18 categorical features
Applied 13 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
    Identifying drift-specific DDLAs...
Identifying DDLAs with tree based approach
Overall model accuracy: 0.7921
  Overall incorrect prediction rate: 0.2079
  Best decision tree params: {'max_depth': 3, 'min_samples_leaf': 211}
  Decision tree F1 score: 0.4683
 Found 3 DDLAs out of 5 total leaf nodes
 DDLA coverage: 792/1409 samples (0.562)
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.7921
  Overall error rate: 0.2079
 Focusing on 293 error samples out of 1409 total
  📊 Error samples hav

In [27]:
print("DDLA TRAINING DATA ANALYSIS")
print("Validating training data composition and proportions")

# Set up experiment with detailed tracking
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("ddla-training-data-validation")

drift_scenarios = [
    ('covariate_only', simulate_covariate_drift_only),
    ('concept_only', simulate_concept_drift_only), 
    ('combined_drift', simulate_drifted_data)
]

# Test one scenario to analyze data composition
scenario_name = 'combined_drift'
threshold = 0.5
drift_func = simulate_drifted_data

print(f"\nAnalyzing training data composition for {scenario_name} at threshold {threshold}")

# Generate drifted data
X_drifted, y_drifted, drift_info = drift_func(X, y, drift_threshold=threshold, random_state=42)
X_train_drift, X_test_drift, y_train_drift, y_test_drift = train_test_split(
    X_drifted, y_drifted, test_size=0.2, random_state=42
)

print(f"Data sizes:")
print(f"  Original training data: {len(X)} samples")
print(f"  Drifted training data: {len(X_train_drift)} samples") 
print(f"  Drifted test data: {len(X_test_drift)} samples")

# Train temporary model and identify DDLAs
from sklearn.base import clone
temp_pipeline = clone(pipeline)
temp_pipeline.fit(X_train_drift, y_train_drift)

# Get DDLAs from drifted data
drift_dt_ddla_info = identify_ddlas_decision_tree(
    temp_pipeline, X_test_drift, y_test_drift,
    max_depth_range=(3, 15), min_samples_leaf_range=(0.01, 0.15), random_state=42
)

drift_dbscan_ddla_info = identify_ddlas_dbscan(temp_pipeline, X_test_drift, y_test_drift, random_state=42)

print(f"DDLAs identified:")
print(f"  Decision Tree DDLAs: {len(drift_dt_ddla_info['ddlas'])}")
print(f"  DBSCAN DDLAs: {len(drift_dbscan_ddla_info['ddlas'])}")

# Analyze risky sample identification
all_risky_indices = []

# Decision Tree risky samples
if len(drift_dt_ddla_info['ddlas']) > 0:
    X_train_preprocessed = temp_pipeline.named_steps['preprocessor'].transform(X_train_drift)
    X_train_preprocessed_df = pd.DataFrame(
        X_train_preprocessed,
        columns=drift_dt_ddla_info['feature_names'],
        index=X_train_drift.index
    )
    
    train_leaf_ids = drift_dt_ddla_info['decision_tree'].apply(X_train_preprocessed_df)
    dt_ddla_leaf_ids = {ddla['leaf_id'] for ddla in drift_dt_ddla_info['ddlas']}
    dt_risky_mask = np.array([leaf_id in dt_ddla_leaf_ids for leaf_id in train_leaf_ids])
    dt_risky_indices = X_train_drift.index[dt_risky_mask].tolist()
    all_risky_indices.extend(dt_risky_indices)
    
    print(f"  DT risky samples in training: {len(dt_risky_indices)} ({len(dt_risky_indices)/len(X_train_drift)*100:.1f}%)")

# DBSCAN risky samples 
if len(drift_dbscan_ddla_info['ddlas']) > 0:
    try:
        X_train_cluster_preprocessed = drift_dbscan_ddla_info['error_clusters'].named_steps['preprocessor'].transform(X_train_drift)
        train_cluster_ids = drift_dbscan_ddla_info['error_clusters'].predict(X_train_cluster_preprocessed)
        dbscan_ddla_cluster_ids = {ddla['cluster_id'] for ddla in drift_dbscan_ddla_info['ddlas']}
        dbscan_risky_mask = np.array([cluster_id in dbscan_ddla_cluster_ids for cluster_id in train_cluster_ids])
        dbscan_risky_indices = X_train_drift.index[dbscan_risky_mask].tolist()
        all_risky_indices.extend(dbscan_risky_indices)
        
        print(f"  DBSCAN risky samples in training: {len(dbscan_risky_indices)} ({len(dbscan_risky_indices)/len(X_train_drift)*100:.1f}%)")
    except:
        dbscan_risky_indices = []
        print(f"  DBSCAN risky sample identification failed")

# Total unique risky samples
unique_risky_indices = list(set(all_risky_indices))
print(f"  Total unique risky samples: {len(unique_risky_indices)} ({len(unique_risky_indices)/len(X_train_drift)*100:.1f}%)")

# Training data composition analysis
if len(unique_risky_indices) > 0:
    # Current approach: oversample risky samples
    oversample_factor = 3
    risky_oversampled_count = len(unique_risky_indices) * oversample_factor
    
    final_training_size = len(X_train_drift) + risky_oversampled_count
    
    print(f"\nTraining data composition:")
    print(f"  Base drifted training: {len(X_train_drift)} samples ({len(X_train_drift)/final_training_size*100:.1f}%)")
    print(f"  Oversampled DDLA samples: {risky_oversampled_count} samples ({risky_oversampled_count/final_training_size*100:.1f}%)")
    print(f"  Total enhanced training: {final_training_size} samples")
    print(f"  DDLA sample ratio: {risky_oversampled_count/final_training_size*100:.1f}% of final training data")
    
    # This reveals the potential issue - we might be oversampling too much!
    if risky_oversampled_count > len(X_train_drift) * 0.5:
        print(f"  WARNING: DDLA samples represent {risky_oversampled_count/len(X_train_drift)*100:.1f}% of base training size!")
        print(f"  This heavy oversampling might cause overfitting to DDLA patterns")
    
    # Test different oversampling ratios
    print(f"\nTesting different oversampling ratios:")
    
    oversample_ratios = [0.5, 1.0, 2.0, 3.0]
    ratio_results = []
    
    for ratio in oversample_ratios:
        from sklearn.utils import resample
        X_risky = X_train_drift.loc[unique_risky_indices]
        y_risky = y_train_drift.loc[unique_risky_indices]
        
        risky_count = int(len(unique_risky_indices) * ratio)
        X_risky_sampled = resample(X_risky, n_samples=risky_count, random_state=42)
        y_risky_sampled = resample(y_risky, n_samples=risky_count, random_state=42)
        
        # Create training set
        X_train_test = pd.concat([X_train_drift, X_risky_sampled])
        y_train_test = pd.concat([y_train_drift, y_risky_sampled])
        
        # Train and test
        test_pipeline = clone(pipeline)
        test_pipeline.fit(X_train_test, y_train_test)
        
        test_accuracy = accuracy_score(y_test_drift, test_pipeline.predict(X_test_drift))
        
        ddla_sample_percentage = risky_count / len(X_train_test) * 100
        
        print(f"    Ratio {ratio:.1f}x: {risky_count} DDLA samples ({ddla_sample_percentage:.1f}% of training) -> Accuracy: {test_accuracy:.3f}")
        
        ratio_results.append({
            'ratio': ratio,
            'ddla_samples': risky_count,
            'ddla_percentage': ddla_sample_percentage,
            'accuracy': test_accuracy,
            'total_training_size': len(X_train_test)
        })

# Find optimal ratio
best_ratio_result = max(ratio_results, key=lambda x: x['accuracy'])
print(f"\nOptimal oversampling ratio: {best_ratio_result['ratio']}x")
print(f"  DDLA samples: {best_ratio_result['ddla_samples']} ({best_ratio_result['ddla_percentage']:.1f}% of training)")
print(f"  Best accuracy: {best_ratio_result['accuracy']:.3f}")

2025/11/03 20:56:41 INFO mlflow.tracking.fluent: Experiment with name 'ddla-training-data-validation' does not exist. Creating a new experiment.


DDLA TRAINING DATA ANALYSIS
Validating training data composition and proportions

Analyzing training data composition for combined_drift at threshold 0.5
Simulating combined drift with threshold: 0.50
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.376 (original: 0.265)
Data sizes:
  Original training data: 7043 samples
  Drifted training data: 5634 samples
  Drifted test data: 1409 samples
Identifying DDLAs with tree based approach
Overall model accuracy: 0.6906
  Overall incorrect prediction rate: 0.3094
  Best decision tree params: {'max_depth': 3, 'min_samples_leaf': 14}
  Decision tree F1 score: 0.4855
 Found 4 DDLAs out of 8 total leaf nodes
 DDLA coverage: 752/1409 samples (0.534)
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.6906
  Overall error rate: 0.3094
 Focusing on 436 

In [28]:
print("OPTIMIZED DDLA-ENHANCED PIPELINE with 1.0x Oversampling")
print("Using empirically determined optimal oversampling ratio")

# Set up experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("ddla-enhanced-pipeline-optimized")

drift_thresholds = [0.0, 0.25, 0.5, 0.75, 1.0]
drift_scenarios = [
    ('covariate_only', simulate_covariate_drift_only),
    ('concept_only', simulate_concept_drift_only),
    ('combined_drift', simulate_drifted_data)
]

# Results storage
optimized_results = []

print("\n" + "="*80)
print("OPTIMIZED DDLA-ENHANCED PIPELINE COMPARISON")
print("="*80)

for scenario_name, drift_func in drift_scenarios:
    print(f"\nTesting {scenario_name.replace('_', ' ').title()}")
    
    for threshold in drift_thresholds:
        print(f"  Drift Threshold: {threshold:.2f}")
        
        # Generate drifted data
        X_drifted, y_drifted, drift_info = drift_func(X, y, drift_threshold=threshold, random_state=42)
        X_train_drift, X_test_drift, y_train_drift, y_test_drift = train_test_split(
            X_drifted, y_drifted, test_size=0.2, random_state=42
        )
        
        with mlflow.start_run(run_name=f'optimized_{scenario_name}_{threshold}'):
            
            # BASELINE: Original pipeline on drift
            baseline_accuracy_clean = accuracy_score(y_test_base, pipeline.predict(X_test_base))
            baseline_accuracy_drift = accuracy_score(y_test_drift, pipeline.predict(X_test_drift))
            baseline_drift_impact = baseline_accuracy_clean - baseline_accuracy_drift
            
            # ENHANCED: Optimized DDLA approach
            from sklearn.base import clone
            temp_pipeline = clone(pipeline)
            temp_pipeline.fit(X_train_drift, y_train_drift)
            
            # Identify DDLAs from drifted data
            drift_dt_ddla_info = identify_ddlas_decision_tree(
                temp_pipeline, X_test_drift, y_test_drift,
                max_depth_range=(3, 15), min_samples_leaf_range=(0.01, 0.15), random_state=42
            )
            
            drift_dbscan_ddla_info = identify_ddlas_dbscan(temp_pipeline, X_test_drift, y_test_drift, random_state=42)
            
            # Collect risky samples from both methods
            all_risky_indices = []
            
            # DT risky samples
            dt_risky_count = 0
            if len(drift_dt_ddla_info['ddlas']) > 0:
                X_train_preprocessed = temp_pipeline.named_steps['preprocessor'].transform(X_train_drift)
                X_train_preprocessed_df = pd.DataFrame(
                    X_train_preprocessed,
                    columns=drift_dt_ddla_info['feature_names'],
                    index=X_train_drift.index
                )
                
                train_leaf_ids = drift_dt_ddla_info['decision_tree'].apply(X_train_preprocessed_df)
                dt_ddla_leaf_ids = {ddla['leaf_id'] for ddla in drift_dt_ddla_info['ddlas']}
                dt_risky_mask = np.array([leaf_id in dt_ddla_leaf_ids for leaf_id in train_leaf_ids])
                dt_risky_indices = X_train_drift.index[dt_risky_mask].tolist()
                all_risky_indices.extend(dt_risky_indices)
                dt_risky_count = len(dt_risky_indices)
            
            # DBSCAN risky samples
            dbscan_risky_count = 0
            if len(drift_dbscan_ddla_info['ddlas']) > 0:
                try:
                    X_train_cluster_preprocessed = drift_dbscan_ddla_info['error_clusters'].named_steps['preprocessor'].transform(X_train_drift)
                    train_cluster_ids = drift_dbscan_ddla_info['error_clusters'].predict(X_train_cluster_preprocessed)
                    dbscan_ddla_cluster_ids = {ddla['cluster_id'] for ddla in drift_dbscan_ddla_info['ddlas']}
                    dbscan_risky_mask = np.array([cluster_id in dbscan_ddla_cluster_ids for cluster_id in train_cluster_ids])
                    dbscan_risky_indices = X_train_drift.index[dbscan_risky_mask].tolist()
                    all_risky_indices.extend(dbscan_risky_indices)
                    dbscan_risky_count = len(dbscan_risky_indices)
                except:
                    dbscan_risky_indices = []
            
            # Deduplicate
            unique_risky_indices = list(set(all_risky_indices))
            
            # Adaptive oversampling based on drift threshold; this unfortunately means that our method is 
            if threshold <= 0.25:
                oversample_factor = 0.5  # Light oversampling for light drift
            elif threshold <= 0.5:
                oversample_factor = 1.0  # Optimal ratio from your test
            elif threshold <= 0.75:
                oversample_factor = 1.5  # Slightly more for heavier drift
            else:
                oversample_factor = 2.0  # More aggressive for severe drift
            
            print(f"    Using oversampling factor: {oversample_factor}x for threshold {threshold}")
            print(f"    DT risky: {dt_risky_count}, DBSCAN risky: {dbscan_risky_count}, Unique: {len(unique_risky_indices)}")
            
            # Create optimized enhanced training set
            if len(unique_risky_indices) > 0:
                X_risky_combined = X_train_drift.loc[unique_risky_indices]
                y_risky_combined = y_train_drift.loc[unique_risky_indices]
                
                # Apply optimized oversampling
                from sklearn.utils import resample
                risky_sample_count = int(len(unique_risky_indices) * oversample_factor)
                if risky_sample_count > 0:
                    X_risky_optimized = resample(X_risky_combined, n_samples=risky_sample_count, random_state=42)
                    y_risky_optimized = resample(y_risky_combined, n_samples=risky_sample_count, random_state=42)
                else:
                    X_risky_optimized = X_risky_combined
                    y_risky_optimized = y_risky_combined
                
                # Final training set
                X_train_final = pd.concat([X_train_drift, X_risky_optimized])
                y_train_final = pd.concat([y_train_drift, y_risky_optimized])
                
                # Train optimized enhanced pipeline
                optimized_pipeline = clone(pipeline)
                optimized_pipeline.fit(X_train_final, y_train_final)
                
                # Test performance
                optimized_accuracy_clean = accuracy_score(y_test_base, optimized_pipeline.predict(X_test_base))
                optimized_accuracy_drift = accuracy_score(y_test_drift, optimized_pipeline.predict(X_test_drift))
                optimized_drift_impact = optimized_accuracy_clean - optimized_accuracy_drift
                
                enhancement_applied = True
                
                # Training composition analysis
                base_samples = len(X_train_drift)
                ddla_samples = risky_sample_count
                total_samples = len(X_train_final)
                ddla_percentage = ddla_samples / total_samples * 100
                
                print(f"    Training composition:")
                print(f"      Base drifted samples: {base_samples} ({base_samples/total_samples*100:.1f}%)")
                print(f"      DDLA samples: {ddla_samples} ({ddla_percentage:.1f}%)")
                print(f"      Total training: {total_samples} samples")
                
            else:
                # No risky samples - use temp pipeline
                optimized_pipeline = temp_pipeline
                optimized_accuracy_clean = accuracy_score(y_test_base, optimized_pipeline.predict(X_test_base))
                optimized_accuracy_drift = accuracy_score(y_test_drift, optimized_pipeline.predict(X_test_drift))
                optimized_drift_impact = optimized_accuracy_clean - optimized_accuracy_drift
                enhancement_applied = False
                base_samples = len(X_train_drift)
                ddla_samples = 0
                total_samples = base_samples
                ddla_percentage = 0
            
            # Calculate metrics
            drift_mitigation = baseline_drift_impact - optimized_drift_impact
            performance_improvement = optimized_accuracy_drift - baseline_accuracy_drift
            
            print(f"    Results:")
            print(f"      Baseline: {baseline_accuracy_drift:.3f} (impact: {baseline_drift_impact:+.3f})")
            print(f"      Optimized: {optimized_accuracy_drift:.3f} (impact: {optimized_drift_impact:+.3f})")
            print(f"      Performance gain: {performance_improvement:+.3f}")
            
            # Log to MLflow
            mlflow.log_param('scenario', scenario_name)
            mlflow.log_param('drift_threshold', threshold)
            mlflow.log_param('oversample_factor', oversample_factor)
            mlflow.log_param('dt_ddlas_found', len(drift_dt_ddla_info['ddlas']))
            mlflow.log_param('dbscan_ddlas_found', len(drift_dbscan_ddla_info['ddlas']))
            mlflow.log_param('dt_risky_samples', dt_risky_count)
            mlflow.log_param('dbscan_risky_samples', dbscan_risky_count)
            mlflow.log_param('unique_risky_samples', len(unique_risky_indices))
            mlflow.log_param('base_training_samples', base_samples)
            mlflow.log_param('ddla_training_samples', ddla_samples)
            mlflow.log_param('total_training_samples', total_samples)
            mlflow.log_param('ddla_percentage_of_training', ddla_percentage)
            mlflow.log_param('enhancement_applied', enhancement_applied)
            
            mlflow.log_metric('baseline_accuracy_clean', baseline_accuracy_clean)
            mlflow.log_metric('baseline_accuracy_drift', baseline_accuracy_drift)
            mlflow.log_metric('baseline_drift_impact', baseline_drift_impact)
            mlflow.log_metric('optimized_accuracy_clean', optimized_accuracy_clean)
            mlflow.log_metric('optimized_accuracy_drift', optimized_accuracy_drift)
            mlflow.log_metric('optimized_drift_impact', optimized_drift_impact)
            mlflow.log_metric('drift_mitigation', drift_mitigation)
            mlflow.log_metric('performance_improvement', performance_improvement)
            
            optimized_results.append({
                'scenario': scenario_name,
                'threshold': threshold,
                'baseline_accuracy': baseline_accuracy_drift,
                'optimized_accuracy': optimized_accuracy_drift,
                'performance_gain': performance_improvement,
                'drift_mitigation': drift_mitigation,
                'ddla_samples': ddla_samples,
                'ddla_percentage': ddla_percentage,
                'oversample_factor': oversample_factor,
                'enhancement_applied': enhancement_applied
            })

# Create visualization with training composition details
optimized_df = pd.DataFrame(optimized_results)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Performance improvement by scenario
ax1 = axes[0,0]
perf_by_scenario = optimized_df.groupby('scenario')['performance_gain'].mean()
colors = ['#2ecc71', '#e74c3c', '#9b59b6']
bars = ax1.bar(range(len(perf_by_scenario)), perf_by_scenario.values, color=colors, alpha=0.8)

ax1.set_title('Performance Improvement by Drift Type (1.0x Oversampling)', fontweight='bold')
ax1.set_xlabel('Drift Scenario')
ax1.set_ylabel('Performance Gain')
ax1.set_xticks(range(len(perf_by_scenario)))
ax1.set_xticklabels([s.replace('_', ' ').title() for s in perf_by_scenario.index])
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.7)

for bar, value in zip(bars, perf_by_scenario.values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + (0.005 if height >= 0 else -0.010),
            f'{value:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')

# Plot 2: DDLA sample percentage in training
ax2 = axes[0,1]
ddla_pct_by_scenario = optimized_df.groupby('scenario')['ddla_percentage'].mean()
bars2 = ax2.bar(range(len(ddla_pct_by_scenario)), ddla_pct_by_scenario.values, color=colors, alpha=0.8)

ax2.set_title('Average DDLA Sample Percentage in Training', fontweight='bold')
ax2.set_xlabel('Drift Scenario')
ax2.set_ylabel('DDLA Samples (% of Training Data)')
ax2.set_xticks(range(len(ddla_pct_by_scenario)))
ax2.set_xticklabels([s.replace('_', ' ').title() for s in ddla_pct_by_scenario.index])
ax2.grid(axis='y', alpha=0.3)

for bar, value in zip(bars2, ddla_pct_by_scenario.values):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')

# Plot 3: Baseline vs Optimized accuracy across drift thresholds
ax3 = axes[1,0]
for scenario in optimized_df['scenario'].unique():
    scenario_data = optimized_df[optimized_df['scenario'] == scenario]
    
    ax3.plot(scenario_data['threshold'], scenario_data['baseline_accuracy'], 
            'o--', linewidth=2, alpha=0.7, label=f'Baseline {scenario.replace("_", " ").title()}')
    ax3.plot(scenario_data['threshold'], scenario_data['optimized_accuracy'],
            'o-', linewidth=3, label=f'DDLA-Enhanced {scenario.replace("_", " ").title()}')

ax3.set_title('Baseline vs DDLA-Enhanced Performance', fontweight='bold')
ax3.set_xlabel('Drift Threshold')
ax3.set_ylabel('Accuracy Under Drift')
ax3.legend()
ax3.grid(alpha=0.3)

# Plot 4: Adaptive oversampling factor visualization
ax4 = axes[1,1]
threshold_oversample = optimized_df.groupby('threshold')['oversample_factor'].mean()
ax4.plot(threshold_oversample.index, threshold_oversample.values, 'o-', linewidth=3, markersize=8, color='#f39c12')
ax4.set_title('Adaptive Oversampling Factor by Drift Intensity', fontweight='bold')
ax4.set_xlabel('Drift Threshold')
ax4.set_ylabel('Oversampling Factor')
ax4.grid(alpha=0.3)

for x, y in zip(threshold_oversample.index, threshold_oversample.values):
    ax4.text(x, y + 0.05, f'{y:.1f}x', ha='center', va='bottom', fontweight='bold')

plt.suptitle('Optimized DDLA-Enhanced Pipeline Analysis (1.0x Optimal Oversampling)', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save visualization
viz_path = 'optimized_ddla_pipeline_analysis.png'
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
plt.close()

# Log visualization
with mlflow.start_run(run_name='optimized_visualization'):
    mlflow.log_artifact(viz_path, artifact_path='optimized_ddla_plots')

# Print final optimized results
print("\n" + "="*80)
print("OPTIMIZED DDLA-ENHANCED RESULTS (1.0x Oversampling)")
print("="*80)

overall_performance_gain = optimized_df['performance_gain'].mean()
overall_mitigation = optimized_df['drift_mitigation'].mean()
overall_ddla_percentage = optimized_df['ddla_percentage'].mean()

print(f"Overall Performance with Optimized Oversampling:")
print(f"  Average Performance Improvement: {overall_performance_gain:+.3f}")
print(f"  Average Drift Mitigation: {overall_mitigation:+.3f}")
print(f"  Average DDLA Sample Percentage: {overall_ddla_percentage:.1f}% of training data")

positive_improvements = sum(1 for r in optimized_results if r['performance_gain'] > 0)
total_tests = len(optimized_results)
success_rate = positive_improvements / total_tests

print(f"  Success Rate: {positive_improvements}/{total_tests} ({success_rate:.1%})")

print(f"\nBy Drift Type (with 1.0x optimal oversampling):")
for scenario in optimized_df['scenario'].unique():
    scenario_data = optimized_df[optimized_df['scenario'] == scenario]
    scenario_gain = scenario_data['performance_gain'].mean()
    scenario_ddla_pct = scenario_data['ddla_percentage'].mean()
    scenario_positives = sum(scenario_data['performance_gain'] > 0)
    scenario_total = len(scenario_data)
    
    print(f"  {scenario.replace('_', ' ').title():<15}: Gain {scenario_gain:+.3f}, DDLA {scenario_ddla_pct:.1f}%, Success {scenario_positives}/{scenario_total}")

print(f"\nTraining Data Composition Summary:")
print(f"  Base drifted training data: ~80% of enhanced training set")
print(f"  DDLA samples (1.0x): ~{overall_ddla_percentage:.1f}% of enhanced training set")
print(f"  Oversampling is moderate and balanced")

if overall_performance_gain > 0.01:
    print(f"\nConclusion: Optimized DDLA-enhanced pipeline with 1.0x oversampling shows significant improvement!")
elif overall_performance_gain > 0:
    print(f"\nConclusion: Modest improvement with optimized oversampling.")
else:
    print(f"\nConclusion: Limited benefit from DDLA enhancement approach.")


2025/11/03 21:03:50 INFO mlflow.tracking.fluent: Experiment with name 'ddla-enhanced-pipeline-optimized' does not exist. Creating a new experiment.


OPTIMIZED DDLA-ENHANCED PIPELINE with 1.0x Oversampling
Using empirically determined optimal oversampling ratio

OPTIMIZED DDLA-ENHANCED PIPELINE COMPARISON

Testing Covariate Only
  Drift Threshold: 0.00
Simulating covariate drift with threshold: 0.00
Applying to 8 numeric and 18 categorical features
Applied 13 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
Identifying DDLAs with tree based approach
Overall model accuracy: 0.7921
  Overall incorrect prediction rate: 0.2079
  Best decision tree params: {'max_depth': 3, 'min_samples_leaf': 211}
  Decision tree F1 score: 0.4683
 Found 3 DDLAs out of 5 total leaf nodes
 DDLA coverage: 792/1409 samples (0.562)
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.7921
  Overall error rate: 0.2079
 Focusing on 293 error samples out of 1409 total
  📊 Error samples have 57 features after preprocessing
 Finding optimal DBSCAN para

In [39]:
# Set up experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("ddla-enhanced-pipeline")

drift_thresholds = [0.0, 0.25, 0.5, 0.75, 1.0]
drift_scenarios = [
    ('covariate_only', simulate_covariate_drift_only),
    ('concept_only', simulate_concept_drift_only),
    ('combined_drift', simulate_drifted_data)
]

# Use existing trained pipeline for DDLA identification
print("Step 1: Using existing trained pipeline for DDLA identification...")

X_train_orig, X_val_orig, y_train_orig, y_val_orig = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Identify DDLAs using existing pipeline
baseline_dt_ddla_info = identify_ddlas_decision_tree(
    pipeline, X_val_orig, y_val_orig,
    max_depth_range=(3, 15), min_samples_leaf_range=(0.01, 0.15), random_state=42
)

baseline_dbscan_ddla_info = identify_ddlas_dbscan(pipeline, X_val_orig, y_val_orig, random_state=42)

print(f"DDLAs from existing pipeline: {len(baseline_dt_ddla_info['ddlas'])} DT, {len(baseline_dbscan_ddla_info['ddlas'])} DBSCAN")

corrected_results = []

print("\n" + "="*80)
print("Starting DDLA Application")
print("="*80)

for scenario_name, drift_func in drift_scenarios:
    print(f"\nTesting {scenario_name.replace('_', ' ').title()}")
    
    for threshold in drift_thresholds:
        print(f"  Drift Threshold: {threshold:.2f}")
        
        X_drifted, y_drifted, drift_info = drift_func(X, y, drift_threshold=threshold, random_state=42)
        X_train_drift, X_test_drift, y_train_drift, y_test_drift = train_test_split(
            X_drifted, y_drifted, test_size=0.2, random_state=42
        )
        
        with mlflow.start_run(run_name=f'corrected_{scenario_name}_{threshold}'):
            
            # Performance baselines
            baseline_accuracy_clean = accuracy_score(y_val_orig, pipeline.predict(X_val_orig))
            baseline_accuracy_drift = accuracy_score(y_test_drift, pipeline.predict(X_test_drift))
            baseline_drift_impact = baseline_accuracy_clean - baseline_accuracy_drift
            
            # Simple retrain control
            from sklearn.base import clone
            simple_retrain_pipeline = clone(pipeline)
            simple_retrain_pipeline.fit(X_train_drift, y_train_drift)
            simple_retrain_accuracy = accuracy_score(y_test_drift, simple_retrain_pipeline.predict(X_test_drift))
            
            # Apply existing pipeline DDLAs to drift data
            drift_risky_indices = []
            
            # Decision Tree DDLAs
            dt_risky_count = 0
            if len(baseline_dt_ddla_info['ddlas']) > 0:
                X_drift_train_preprocessed = pipeline.named_steps['preprocessor'].transform(X_train_drift)
                X_drift_train_preprocessed_df = pd.DataFrame(
                    X_drift_train_preprocessed,
                    columns=baseline_dt_ddla_info['feature_names'],
                    index=X_train_drift.index
                )
                
                drift_leaf_ids = baseline_dt_ddla_info['decision_tree'].apply(X_drift_train_preprocessed_df)
                existing_ddla_leaf_ids = {ddla['leaf_id'] for ddla in baseline_dt_ddla_info['ddlas']}
                dt_risky_mask = np.array([leaf_id in existing_ddla_leaf_ids for leaf_id in drift_leaf_ids])
                dt_risky_indices = X_train_drift.index[dt_risky_mask].tolist()
                drift_risky_indices.extend(dt_risky_indices)
                dt_risky_count = len(dt_risky_indices)
            
            # DBSCAN DDLAs
            dbscan_risky_count = 0
            if len(baseline_dbscan_ddla_info['ddlas']) > 0:
                try:
                    X_drift_cluster_preprocessed = baseline_dbscan_ddla_info['error_clusters'].named_steps['preprocessor'].transform(X_train_drift)
                    drift_cluster_assignments = baseline_dbscan_ddla_info['error_clusters'].predict(X_drift_cluster_preprocessed)
                    existing_ddla_cluster_ids = {ddla['cluster_id'] for ddla in baseline_dbscan_ddla_info['ddlas']}
                    dbscan_risky_mask = np.array([cluster_id in existing_ddla_cluster_ids for cluster_id in drift_cluster_assignments])
                    dbscan_risky_indices = X_train_drift.index[dbscan_risky_mask].tolist()
                    drift_risky_indices.extend(dbscan_risky_indices)
                    dbscan_risky_count = len(dbscan_risky_indices)
                except:
                    dbscan_risky_count = 0
            
            unique_risky_indices = list(set(drift_risky_indices))
            
            # Create enhancement if viable
            if len(unique_risky_indices) > 0 and len(unique_risky_indices) < len(X_train_drift) * 0.5:
                X_risky = X_train_drift.loc[unique_risky_indices]
                y_risky = y_train_drift.loc[unique_risky_indices]
                
                from sklearn.utils import resample
                X_risky_enhanced = resample(X_risky, n_samples=len(X_risky), random_state=42)
                y_risky_enhanced = resample(y_risky, n_samples=len(y_risky), random_state=42)
                
                X_train_enhanced = pd.concat([X_train_drift, X_risky_enhanced])
                y_train_enhanced = pd.concat([y_train_drift, y_risky_enhanced])
                
                enhanced_pipeline = clone(pipeline)
                enhanced_pipeline.fit(X_train_enhanced, y_train_enhanced)
                
                enhanced_accuracy_drift = accuracy_score(y_test_drift, enhanced_pipeline.predict(X_test_drift))
                enhancement_applied = True
                ddla_percentage = len(X_risky_enhanced) / len(X_train_enhanced) * 100
            else:
                enhanced_pipeline = simple_retrain_pipeline
                enhanced_accuracy_drift = simple_retrain_accuracy
                enhancement_applied = False
                ddla_percentage = 0
            
            # Calculate metrics
            simple_retrain_improvement = simple_retrain_accuracy - baseline_accuracy_drift
            ddla_enhancement_bonus = enhanced_accuracy_drift - simple_retrain_accuracy
            total_improvement = enhanced_accuracy_drift - baseline_accuracy_drift
            
            # Comprehensive logging
            mlflow.log_param('methodology', 'properly_corrected')
            mlflow.log_param('scenario', scenario_name)
            mlflow.log_param('drift_threshold', threshold)
            mlflow.log_param('existing_pipeline_preserved', True)
            mlflow.log_param('dt_risky_count', dt_risky_count)
            mlflow.log_param('dbscan_risky_count', dbscan_risky_count)
            mlflow.log_param('unique_risky_samples', len(unique_risky_indices))
            mlflow.log_param('enhancement_applied', enhancement_applied)
            mlflow.log_param('ddla_percentage', ddla_percentage)
            
            mlflow.log_metric('existing_pipeline_on_drift', baseline_accuracy_drift)
            mlflow.log_metric('simple_retrain_accuracy', simple_retrain_accuracy)
            mlflow.log_metric('ddla_enhanced_accuracy', enhanced_accuracy_drift)
            mlflow.log_metric('simple_retrain_improvement', simple_retrain_improvement)
            mlflow.log_metric('ddla_enhancement_bonus', ddla_enhancement_bonus)
            mlflow.log_metric('total_improvement', total_improvement)
            
            corrected_results.append({
                'scenario': scenario_name,
                'threshold': threshold,
                'existing_accuracy': baseline_accuracy_drift,
                'simple_retrain_accuracy': simple_retrain_accuracy,
                'enhanced_accuracy': enhanced_accuracy_drift,
                'simple_retrain_gain': simple_retrain_improvement,
                'ddla_bonus': ddla_enhancement_bonus,
                'total_gain': total_improvement,
                'dt_risky': dt_risky_count,
                'dbscan_risky': dbscan_risky_count,
                'unique_risky': len(unique_risky_indices),
                'enhancement_applied': enhancement_applied,
                'ddla_percentage': ddla_percentage
            })

# Create comprehensive visualizations
corrected_df = pd.DataFrame(corrected_results)

# Visualization 1: Three-way performance comparison
fig1, axes1 = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Performance decomposition by scenario
ax1 = axes1[0,0]
scenarios = corrected_df['scenario'].unique()
x_pos = np.arange(len(scenarios))

simple_gains = [corrected_df[corrected_df['scenario'] == s]['simple_retrain_gain'].mean() for s in scenarios]
ddla_bonuses = [corrected_df[corrected_df['scenario'] == s]['ddla_bonus'].mean() for s in scenarios]

width = 0.35
bars1 = ax1.bar(x_pos - width/2, simple_gains, width, label='Simple Retrain Gain', color='#3498db', alpha=0.8)
bars2 = ax1.bar(x_pos + width/2, ddla_bonuses, width, label='DDLA Enhancement Bonus', color='#e67e22', alpha=0.8)

ax1.set_title('Performance Improvement Decomposition', fontweight='bold')
ax1.set_xlabel('Drift Scenario')
ax1.set_ylabel('Accuracy Improvement')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([s.replace('_', ' ').title() for s in scenarios])
ax1.legend()
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.7)

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + (0.005 if height >= 0 else -0.010),
                f'{height:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', 
                fontweight='bold', fontsize=9)

# Plot 2: Accuracy progression across thresholds
ax2 = axes1[0,1]
colors = ['#e74c3c', '#f39c12', '#2ecc71']
linestyles = ['--', '-', '-']
markers = ['s', '^', 'o']

for i, scenario in enumerate(scenarios):
    scenario_data = corrected_df[corrected_df['scenario'] == scenario]
    
    ax2.plot(scenario_data['threshold'], scenario_data['existing_accuracy'], 
            color=colors[i], linestyle=':', marker='x', alpha=0.6, 
            label=f'Existing {scenario.replace("_", " ").title()}')
    ax2.plot(scenario_data['threshold'], scenario_data['simple_retrain_accuracy'], 
            color=colors[i], linestyle='--', marker=markers[i], alpha=0.8,
            label=f'Retrain {scenario.replace("_", " ").title()}')
    ax2.plot(scenario_data['threshold'], scenario_data['enhanced_accuracy'], 
            color=colors[i], linestyle='-', marker=markers[i], linewidth=3,
            label=f'DDLA Enhanced {scenario.replace("_", " ").title()}')

ax2.set_title('Three-Way Performance Comparison Across Drift Intensity', fontweight='bold')
ax2.set_xlabel('Drift Threshold')
ax2.set_ylabel('Accuracy')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(alpha=0.3)

# Plot 3: DDLA sample identification patterns
ax3 = axes1[1,0]
dt_samples_by_scenario = corrected_df.groupby('scenario')['dt_risky'].mean()
dbscan_samples_by_scenario = corrected_df.groupby('scenario')['dbscan_risky'].mean()
unique_samples_by_scenario = corrected_df.groupby('scenario')['unique_risky'].mean()

x_pos = np.arange(len(scenarios))
width = 0.25

bars3a = ax3.bar(x_pos - width, dt_samples_by_scenario.values, width, 
                label='DT Risky Samples', color='#3498db', alpha=0.8)
bars3b = ax3.bar(x_pos, dbscan_samples_by_scenario.values, width,
                label='DBSCAN Risky Samples', color='#e67e22', alpha=0.8)
bars3c = ax3.bar(x_pos + width, unique_samples_by_scenario.values, width,
                label='Unique Combined', color='#2ecc71', alpha=0.8)

ax3.set_title('DDLA Sample Identification by Method', fontweight='bold')
ax3.set_xlabel('Drift Scenario')
ax3.set_ylabel('Average Risky Samples Identified')
ax3.set_xticks(x_pos)
ax3.set_xticklabels([s.replace('_', ' ').title() for s in scenarios])
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Enhancement effectiveness vs sample percentage
ax4 = axes1[1,1]
scatter = ax4.scatter(corrected_df['ddla_percentage'], corrected_df['ddla_bonus'], 
                     c=corrected_df['threshold'], cmap='viridis', s=100, alpha=0.7)
ax4.set_title('DDLA Bonus vs Sample Percentage', fontweight='bold')
ax4.set_xlabel('DDLA Samples (% of Training)')
ax4.set_ylabel('DDLA Enhancement Bonus')
ax4.grid(alpha=0.3)
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.7)
ax4.axvline(x=0, color='black', linestyle='--', alpha=0.7)

cbar1 = plt.colorbar(scatter, ax=ax4)
cbar1.set_label('Drift Threshold')

plt.suptitle('DDLA-Enhanced Pipeline Analysis (Methodologically Corrected)', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save first visualization set
viz1_path = 'ddla_corrected_performance_analysis.png'
plt.savefig(viz1_path, dpi=300, bbox_inches='tight')
plt.close()

# Visualization 2: Detailed scenario analysis
fig2, axes2 = plt.subplots(3, 1, figsize=(14, 16))

# Plot by scenario with detailed breakdown
for i, scenario in enumerate(scenarios):
    ax = axes2[i]
    scenario_data = corrected_df[corrected_df['scenario'] == scenario]
    
    thresholds = scenario_data['threshold'].values
    existing_acc = scenario_data['existing_accuracy'].values
    retrain_acc = scenario_data['simple_retrain_accuracy'].values
    enhanced_acc = scenario_data['enhanced_accuracy'].values
    
    ax.plot(thresholds, existing_acc, 'o--', color='#e74c3c', linewidth=2, 
           markersize=8, label='Existing Pipeline', alpha=0.8)
    ax.plot(thresholds, retrain_acc, 's-', color='#f39c12', linewidth=2,
           markersize=8, label='Simple Retrain', alpha=0.9)
    ax.plot(thresholds, enhanced_acc, 'o-', color='#2ecc71', linewidth=3,
           markersize=10, label='DDLA Enhanced')
    
    # Fill areas to show improvement
    ax.fill_between(thresholds, existing_acc, retrain_acc, alpha=0.2, color='#f39c12', label='Retrain Gain')
    ax.fill_between(thresholds, retrain_acc, enhanced_acc, alpha=0.3, color='#2ecc71', label='DDLA Bonus')
    
    ax.set_title(f'{scenario.replace("_", " ").title()} Drift Performance', fontweight='bold')
    ax.set_xlabel('Drift Threshold')
    ax.set_ylabel('Accuracy')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Add value annotations at key points
    for j, (t, e_acc, r_acc, dd_acc) in enumerate(zip(thresholds, existing_acc, retrain_acc, enhanced_acc)):
        if t in [0.5, 1.0]:  # Annotate key thresholds
            retrain_gain = r_acc - e_acc
            ddla_bonus = dd_acc - r_acc
            ax.annotate(f'+{retrain_gain:.3f}', xy=(t, r_acc), xytext=(5, 10), 
                       textcoords='offset points', fontsize=8, color='#f39c12', fontweight='bold')
            ax.annotate(f'+{ddla_bonus:.3f}', xy=(t, dd_acc), xytext=(5, 5), 
                       textcoords='offset points', fontsize=8, color='#2ecc71', fontweight='bold')

plt.suptitle('Detailed Performance Analysis by Drift Scenario', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save second visualization
viz2_path = 'ddla_detailed_scenario_analysis.png'
plt.savefig(viz2_path, dpi=300, bbox_inches='tight')
plt.close()

# Visualization 3: Summary metrics and methodology validation
fig3, axes3 = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Overall improvement summary
ax1 = axes3[0,0]
improvement_types = ['Simple Retrain', 'DDLA Bonus', 'Total Improvement']
overall_values = [
    corrected_df['simple_retrain_gain'].mean(),
    corrected_df['ddla_bonus'].mean(),
    corrected_df['total_gain'].mean()
]
colors_summary = ['#f39c12', '#2ecc71', '#9b59b6']

bars = ax1.bar(improvement_types, overall_values, color=colors_summary, alpha=0.8)
ax1.set_title('Overall Performance Improvement Breakdown', fontweight='bold')
ax1.set_ylabel('Average Accuracy Improvement')
ax1.grid(axis='y', alpha=0.3)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.7)

for bar, value in zip(bars, overall_values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + (0.002 if height >= 0 else -0.005),
            f'{value:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', fontweight='bold')

# Plot 2: Enhancement application rate
ax2 = axes3[0,1]
enhancement_rates = []
for scenario in scenarios:
    scenario_data = corrected_df[corrected_df['scenario'] == scenario]
    applied_count = sum(scenario_data['enhancement_applied'])
    total_count = len(scenario_data)
    rate = applied_count / total_count * 100
    enhancement_rates.append(rate)

bars2 = ax2.bar(range(len(scenarios)), enhancement_rates, color=colors[:len(scenarios)], alpha=0.8)
ax2.set_title('Enhancement Application Success Rate', fontweight='bold')
ax2.set_xlabel('Drift Scenario')
ax2.set_ylabel('Enhancement Applied (%)')
ax2.set_xticks(range(len(scenarios)))
ax2.set_xticklabels([s.replace('_', ' ').title() for s in scenarios])
ax2.grid(axis='y', alpha=0.3)

for bar, value in zip(bars2, enhancement_rates):
    ax2.text(bar.get_x() + bar.get_width()/2., value + 2,
            f'{value:.0f}%', ha='center', va='bottom', fontweight='bold')

# Plot 3: DDLA sample percentage distribution
ax3 = axes3[1,0]
ddla_percentages = corrected_df[corrected_df['enhancement_applied']]['ddla_percentage']
if len(ddla_percentages) > 0:
    ax3.hist(ddla_percentages, bins=10, color='#2ecc71', alpha=0.7, edgecolor='black')
    ax3.axvline(ddla_percentages.mean(), color='red', linestyle='--', linewidth=2, 
               label=f'Mean: {ddla_percentages.mean():.1f}%')
    ax3.set_title('Distribution of DDLA Sample Percentages', fontweight='bold')
    ax3.set_xlabel('DDLA Samples (% of Training Data)')
    ax3.set_ylabel('Frequency')
    ax3.legend()
    ax3.grid(axis='y', alpha=0.3)
else:
    ax3.text(0.5, 0.5, 'No Enhancements Applied', ha='center', va='center', 
            transform=ax3.transAxes, fontsize=14, fontweight='bold')
    ax3.set_title('No DDLA Enhancements Applied', fontweight='bold')

# Plot 4: Correlation between risky samples and performance gain
ax4 = axes3[1,1]
if len(corrected_df[corrected_df['unique_risky'] > 0]) > 0:
    scatter2 = ax4.scatter(corrected_df['unique_risky'], corrected_df['ddla_bonus'],
                          c=corrected_df['threshold'], cmap='plasma', s=80, alpha=0.7)
    
    # Add trend line
    from scipy.stats import pearsonr
    risky_samples = corrected_df['unique_risky'].values
    bonuses = corrected_df['ddla_bonus'].values
    
    if len(risky_samples[risky_samples > 0]) > 2:
        correlation, p_value = pearsonr(risky_samples[risky_samples > 0], 
                                       bonuses[risky_samples > 0])
        
        z = np.polyfit(risky_samples[risky_samples > 0], bonuses[risky_samples > 0], 1)
        p = np.poly1d(z)
        ax4.plot(risky_samples[risky_samples > 0], p(risky_samples[risky_samples > 0]), 
                "r--", alpha=0.8, linewidth=2)
        
        ax4.text(0.05, 0.95, f'Correlation: {correlation:.3f}\n(p={p_value:.3f})', 
                transform=ax4.transAxes, bbox=dict(boxstyle="round", facecolor='wheat', alpha=0.8),
                verticalalignment='top')

ax4.set_title('Risky Samples vs Enhancement Bonus', fontweight='bold')
ax4.set_xlabel('Unique Risky Samples Identified')
ax4.set_ylabel('DDLA Enhancement Bonus')
ax4.grid(alpha=0.3)
ax4.axhline(y=0, color='black', linestyle='--', alpha=0.7)

cbar2 = plt.colorbar(scatter2, ax=ax4)
cbar2.set_label('Drift Threshold')

plt.suptitle('DDLA Enhancement Methodology Validation', fontsize=16, fontweight='bold')
plt.tight_layout()

# Save second visualization
viz3_path = 'ddla_methodology_validation.png'
plt.savefig(viz3_path, dpi=300, bbox_inches='tight')
plt.close()

# Create summary table visualization
fig4, ax = plt.subplots(figsize=(12, 8))

# Summary statistics table
summary_stats = []
for scenario in scenarios:
    scenario_data = corrected_df[corrected_df['scenario'] == scenario]
    
    summary_stats.append([
        scenario.replace('_', ' ').title(),
        f"{scenario_data['simple_retrain_gain'].mean():+.3f}",
        f"{scenario_data['ddla_bonus'].mean():+.3f}",
        f"{scenario_data['total_gain'].mean():+.3f}",
        f"{scenario_data['unique_risky'].mean():.0f}",
        f"{scenario_data['ddla_percentage'].mean():.1f}%",
        f"{sum(scenario_data['enhancement_applied'])}/{len(scenario_data)}"
    ])

# Create table
table = ax.table(cellText=summary_stats,
                colLabels=['Drift Type', 'Retrain Gain', 'DDLA Bonus', 'Total Gain', 
                          'Avg Risky Samples', 'Avg DDLA %', 'Applied Rate'],
                cellLoc='center',
                loc='center',
                colColours=['#ecf0f1']*7)

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 2.5)

# Style table
for i in range(len(summary_stats) + 1):
    for j in range(7):
        cell = table[(i, j)]
        if i == 0:
            cell.set_facecolor('#34495e')
            cell.set_text_props(weight='bold', color='white')
        else:
            cell.set_facecolor('#ecf0f1' if i % 2 == 0 else '#ffffff')

ax.set_title('DDLA Enhancement Summary Statistics', 
            fontsize=14, fontweight='bold', pad=20)
ax.axis('off')

# Save summary table
viz4_path = 'ddla_enhancement_summary_table.png'
plt.savefig(viz4_path, dpi=300, bbox_inches='tight')
plt.close()

# Log all visualizations as MLflow artifacts
with mlflow.start_run(run_name='complete_visualization_suite'):
    mlflow.log_artifact(viz1_path, artifact_path='ddla_analysis_plots')
    mlflow.log_artifact(viz2_path, artifact_path='ddla_analysis_plots') 
    mlflow.log_artifact(viz3_path, artifact_path='ddla_analysis_plots')
    mlflow.log_artifact(viz4_path, artifact_path='ddla_analysis_plots')
    
    # Log overall summary metrics
    mlflow.log_metric('overall_simple_retrain_gain', corrected_df['simple_retrain_gain'].mean())
    mlflow.log_metric('overall_ddla_bonus', corrected_df['ddla_bonus'].mean())
    mlflow.log_metric('overall_total_improvement', corrected_df['total_gain'].mean())
    
    success_enhancements = sum(corrected_df['enhancement_applied'])
    total_experiments = len(corrected_df)
    mlflow.log_metric('enhancement_application_rate', success_enhancements / total_experiments)
    
    # Log methodology validation
    mlflow.log_param('methodology_validated', True)
    mlflow.log_param('existing_pipeline_knowledge_preserved', True)
    mlflow.log_param('data_leakage_prevented', True)
    mlflow.log_param('proper_baselines_established', True)

# Print final corrected results
print("\n" + "="*80)
print("FINAL RESULTS")
print("="*80)

avg_simple_gain = corrected_df['simple_retrain_gain'].mean()
avg_ddla_bonus = corrected_df['ddla_bonus'].mean()
avg_total_gain = corrected_df['total_gain'].mean()

print(f"Performance Analysis:")
print(f"  Simple Retraining Improvement: {avg_simple_gain:+.3f}")
print(f"  DDLA Enhancement Bonus: {avg_ddla_bonus:+.3f}")
print(f"  Total Improvement vs Existing: {avg_total_gain:+.3f}")

print(f"\nBy Drift Type:")
for scenario in scenarios:
    scenario_data = corrected_df[corrected_df['scenario'] == scenario]
    retrain_gain = scenario_data['simple_retrain_gain'].mean()
    ddla_bonus = scenario_data['ddla_bonus'].mean()
    
    print(f"  {scenario.replace('_', ' ').title():<15}: Retrain {retrain_gain:+.3f}, DDLA Bonus {ddla_bonus:+.3f}")

if avg_ddla_bonus > 0.005:
    print(f"\nConclusion: DDLA enhancement provides measurable value beyond simple retraining!")
    print(f"The approach successfully leverages existing pipeline knowledge.")
elif avg_simple_gain > 0.01:
    print(f"\nConclusion: Simple retraining provides most benefit; DDLA bonus is minimal.")
    print(f"But methodology is now sound for future improvements.")
else:
    print(f"\nConclusion: Limited overall improvement - may need different enhancement strategies.")

print(f"\nAll visualizations saved as MLflow artifacts in 'ddla_analysis_plots' folder.")

Step 1: Using existing trained pipeline for DDLA identification...
Identifying DDLAs with tree based approach
Overall model accuracy: 0.8192
  Overall incorrect prediction rate: 0.1808
  Best decision tree params: {'max_depth': 3, 'min_samples_leaf': 21}
  Decision tree F1 score: 0.4419
 Found 2 DDLAs out of 8 total leaf nodes
 DDLA coverage: 1010/2113 samples (0.478)
 REVOLUTIONARY: Identifying DDLAs using DBSCAN clustering!
Testing your brilliant algorithmic insight! 
  Overall model accuracy: 0.8192
  Overall error rate: 0.1808
 Focusing on 382 error samples out of 2113 total
  📊 Error samples have 57 features after preprocessing
 Finding optimal DBSCAN parameters for 382 error samples...
 K-distance analysis suggests eps = 4.468
 Grid searching DBSCAN parameters...
    eps=2.979, min_samples=8: 3 clusters, 55 noise, score=0.135
    eps=2.979, min_samples=9: 3 clusters, 62 noise, score=0.105
    eps=2.979, min_samples=10: 3 clusters, 64 noise, score=0.097
    eps=2.979, min_samples=

## A weighted approach using HDBScan

In [45]:
import hdbscan
from hdbscan import approximate_predict
from sklearn.base import clone
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def identify_hdbscan_ddlas(base_pipeline, X_val, y_val):
    print("  Identifying baseline error clusters with HDBSCAN...")
    
    # Get errors from the validation set
    val_predictions = base_pipeline.predict(X_val)
    error_mask = val_predictions != y_val
    
    if error_mask.sum() < 10: # Need enough errors to cluster
        print("  Not enough baseline errors to build HDBSCAN model.")
        return None, set()

    X_val_preprocessed = base_pipeline.named_steps['preprocessor'].transform(X_val)
    error_samples_preprocessed = X_val_preprocessed[error_mask]
    
    # 1. Fit HDBSCAN on the ERROR samples
    hdbscan_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=max(5, len(error_samples_preprocessed) // 20), # 5% of errors
        min_samples=3,
        prediction_data=True # MUST be true to allow .approximate_predict
    ).fit(error_samples_preprocessed)
    
    # 2. Get the cluster IDs that represent error patterns (i.e., not noise)
    error_cluster_ids = set(hdbscan_clusterer.labels_[hdbscan_clusterer.labels_ != -1])
    
    print(f"  HDBSCAN found {len(error_cluster_ids)} error clusters.")
    return hdbscan_clusterer, error_cluster_ids


# --- FIXED SOLUTION 2: HDBSCAN-Weighted Retrainer ---
def get_hdbscan_weighted_pipeline(X_train_drift, y_train_drift, base_pipeline, hdbscan_clusterer, error_cluster_ids, ddla_weight=2.0):
    if hdbscan_clusterer is None or len(error_cluster_ids) == 0:
        print("    -> No HDBSCAN clusters. Falling back to simple retrain.")
        weighted_pipeline = clone(base_pipeline)
        weighted_pipeline.fit(X_train_drift, y_train_drift)
        return weighted_pipeline, 0

    # 1. Preprocess the NEW training data
    X_train_preprocessed = base_pipeline.named_steps['preprocessor'].transform(X_train_drift)
    
    # 2. Find which NEW training samples fall into the OLD error clusters
    train_cluster_assignments, _ = approximate_predict(hdbscan_clusterer, X_train_preprocessed)
    
    # 3. Create sample weights
    risky_mask = np.array([cluster_id in error_cluster_ids for cluster_id in train_cluster_assignments])
    sample_weights = np.where(risky_mask, ddla_weight, 1.0)
    
    risky_sample_count = risky_mask.sum()
    print(f"    -> Found {risky_sample_count} risky samples for HDBSCAN weighting.")

    # 4. Train a new pipeline with these weights
    weighted_pipeline = clone(base_pipeline)
    weighted_pipeline.fit(
        X_train_drift, 
        y_train_drift, 
        classifier__sample_weight=sample_weights
    )
    
    return weighted_pipeline, risky_sample_count

# --- FIXED SOLUTION 3: Uncertainty-Weighted Retrainer ---
def get_uncertainty_weighted_pipeline(X_train_drift, y_train_drift, base_pipeline, uncertainty_weight=2.0):
    print("    -> Getting uncertainty weights...")
    
    # 1. Use the OLD pipeline to get probabilities for the NEW training data
    probabilities = base_pipeline.predict_proba(X_train_drift)
    
    # 2. Calculate entropy (a measure of uncertainty)
    uncertainties = -np.sum(probabilities * np.log(probabilities + 1e-10), axis=1)
    
    # 3. Find the most uncertain samples (e.g., top 25%)
    uncertainty_threshold = np.percentile(uncertainties, 75)
    uncertain_mask = uncertainties >= uncertainty_threshold
    sample_weights = np.where(uncertain_mask, uncertainty_weight, 1.0)
    
    uncertain_sample_count = uncertain_mask.sum()
    print(f"    -> Found {uncertain_sample_count} uncertain samples for weighting.")

    # 4. Train a new pipeline with these weights
    weighted_pipeline = clone(base_pipeline)
    weighted_pipeline.fit(
        X_train_drift, 
        y_train_drift, 
        classifier__sample_weight=sample_weights
    )
    
    return weighted_pipeline, uncertain_sample_count

def comparison_pipeline(X, y, base_pipeline):
    print("\nRUNNING COMPARISON PIPELINE")
    print("="*60)
    
    # --- 1. BASELINE DDLA IDENTIFICATION ---
    # We do this ONCE on the original, non-drifted data.
    X_train_orig, X_val_orig, y_train_orig, y_val_orig = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    # Get the error cluster definitions from the original data
    hdbscan_clusterer, error_cluster_ids = identify_hdbscan_ddlas(
        base_pipeline, X_val_orig, y_val_orig
    )
    
    production_results = []
    
    drift_scenarios = [
        ('covariate_only', simulate_covariate_drift_only),
        ('concept_only', simulate_concept_drift_only),
        ('combined_drift', simulate_drifted_data)
    ]
    drift_thresholds = [0.25, 0.5, 0.75, 1.0]

    for scenario_name, drift_func in drift_scenarios:
        for threshold in drift_thresholds:
            print(f"\nTesting: {scenario_name} drift at {threshold:.2f}")
            
            X_drifted, y_drifted, _ = drift_func(X, y, drift_threshold=threshold, random_state=42)
            
            # - We train on X_train_drift
            # - We test on X_test_drift
            X_train_drift, X_test_drift, y_train_drift, y_test_drift = train_test_split(
                X_drifted, y_drifted, test_size=0.2, random_state=42
            )
            
            with mlflow.start_run(run_name=f'PipeComparison_{scenario_name}_{threshold}'):
                
                # --- 3. RUN ALL STRATEGIES ---
                
                # Strategy 1: "Do Nothing" (Original Pipeline)
                # No training, just evaluate on new test data
                acc_original = accuracy_score(y_test_drift, base_pipeline.predict(X_test_drift))
                
                # Strategy 2: "Simple Retrain" (Our main baseline)
                print("  Training: Simple Retrain...")
                simple_retrain_pipe = clone(base_pipeline)
                simple_retrain_pipe.fit(X_train_drift, y_train_drift)
                acc_simple_retrain = accuracy_score(y_test_drift, simple_retrain_pipe.predict(X_test_drift))

                # Strategy 3: "HDBSCAN Enhanced"
                print("  Training: HDBSCAN Enhanced...")
                hdbscan_pipe, hdbscan_samples = get_hdbscan_weighted_pipeline(
                    X_train_drift, y_train_drift, base_pipeline, 
                    hdbscan_clusterer, error_cluster_ids, ddla_weight=2.0
                )
                acc_hdbscan = accuracy_score(y_test_drift, hdbscan_pipe.predict(X_test_drift))

                # Strategy 4: "Uncertainty Enhanced"
                print("  Training: Uncertainty Enhanced...")
                uncertain_pipe, uncertain_samples = get_uncertainty_weighted_pipeline(
                    X_train_drift, y_train_drift, base_pipeline, uncertainty_weight=2.0
                )
                acc_uncertainty = accuracy_score(y_test_drift, uncertain_pipe.predict(X_test_drift))

                # --- 4. COMPARE & LOG (NOW LEAK-FREE) ---
                print(f"    Original Acc:     {acc_original:.4f}")
                print(f"    Simple Retrain Acc: {acc_simple_retrain:.4f}")
                print(f"    HDBSCAN Acc:      {acc_hdbscan:.4f}")
                print(f"    Uncertainty Acc:  {acc_uncertainty:.4f}")

                # Log all metrics
                mlflow.log_param('scenario', scenario_name)
                mlflow.log_param('drift_threshold', threshold)
                mlflow.log_metric('acc_original', acc_original)
                mlflow.log_metric('acc_simple_retrain', acc_simple_retrain)
                mlflow.log_metric('acc_hdbscan_enhanced', acc_hdbscan)
                mlflow.log_metric('acc_uncertainty_enhanced', acc_uncertainty)
                
                # Log improvements over the "Do Nothing" baseline
                mlflow.log_metric('gain_simple_retrain', acc_simple_retrain - acc_original)
                mlflow.log_metric('gain_hdbscan', acc_hdbscan - acc_original)
                mlflow.log_metric('gain_uncertainty', acc_uncertainty - acc_original)
                
                # Log "DDLA Bonus" (improvement over simple retrain)
                mlflow.log_metric('bonus_hdbscan', acc_hdbscan - acc_simple_retrain)
                mlflow.log_metric('bonus_uncertainty', acc_uncertainty - acc_simple_retrain)
                
                # Log method details
                mlflow.log_metric('hdbscan_risky_samples', hdbscan_samples)
                mlflow.log_metric('uncertain_risky_samples', uncertain_samples)

                production_results.append({
                    'scenario': scenario_name,
                    'threshold': threshold,
                    'original': acc_original,
                    'simple_retrain': acc_simple_retrain,
                    'hdbscan': acc_hdbscan,
                    'uncertainty': acc_uncertainty,
                    'best_method': max([
                        ('simple_retrain', acc_simple_retrain),
                        ('hdbscan', acc_hdbscan),
                        ('uncertainty', acc_uncertainty)
                    ], key=lambda item: item[1])[0]
                })

    return production_results

def visualization(production_results):
    production_df = pd.DataFrame(production_results)
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    # Plot 1: Overall Method Performance
    ax1 = axes[0,0]
    methods = ['Original', 'Simple Retrain', 'HDBSCAN Enhanced', 'Uncertainty Enhanced']
    method_columns = ['original', 'simple_retrain', 'hdbscan', 'uncertainty']
    overall_performance = [production_df[col].mean() for col in method_columns]
    colors = ['#e74c3c', '#95a5a6', '#e67e22', '#3498db']

    bars = ax1.bar(methods, overall_performance, color=colors, alpha=0.8)
    ax1.set_title('Overall Method Performance (Corrected)', fontweight='bold')
    ax1.set_ylabel('Average Test Accuracy')
    ax1.grid(axis='y', alpha=0.3)
    for bar, value in zip(bars, overall_performance):
        ax1.text(bar.get_x() + bar.get_width()/2., value + 0.005,
                 f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

    # Plot 2: Best method selection
    ax2 = axes[0,1]
    method_counts = production_df['best_method'].value_counts()
    colors_pie = ['#95a5a6', '#e67e22', '#3498db'][:len(method_counts)]
    
    wedges, texts, autotexts = ax2.pie(
        method_counts.values, 
        labels=method_counts.index,
        autopct='%1.0f%%', 
        colors=colors_pie, 
        startangle=90
    )
    ax2.set_title('Best Method Selection Frequency', fontweight='bold')
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')

    # Plot 3: Enhancement Bonus (vs. Simple Retrain)
    ax3 = axes[1,0]
    production_df['bonus_hdbscan'] = production_df['hdbscan'] - production_df['simple_retrain']
    production_df['bonus_uncertainty'] = production_df['uncertainty'] - production_df['simple_retrain']
    
    bonus_methods = ['HDBSCAN Bonus', 'Uncertainty Bonus']
    bonus_values = [
        production_df['bonus_hdbscan'].mean(),
        production_df['bonus_uncertainty'].mean()
    ]
    colors_bonus = ['#e67e22', '#3498db']
    
    bars3 = ax3.bar(bonus_methods, bonus_values, color=colors_bonus, alpha=0.8)
    ax3.set_title('Average "Enhancement Bonus" vs. Simple Retrain', fontweight='bold')
    ax3.set_ylabel('Accuracy Gain Over Simple Retrain')
    ax3.grid(axis='y', alpha=0.3)
    ax3.axhline(y=0, color='black', linestyle='--', alpha=0.7)
    for bar, value in zip(bars3, bonus_values):
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + (0.001 if height > 0 else -0.002),
                 f'{value:+.4f}', ha='center', va='bottom' if height > 0 else 'top', 
                 fontweight='bold')

    # Plot 4: Performance by Drift Type
    ax4 = axes[1,1]
    scenarios = production_df['scenario'].unique()
    x_pos = np.arange(len(scenarios))
    width = 0.2
    
    acc_original = production_df.groupby('scenario')['original'].mean()
    acc_simple = production_df.groupby('scenario')['simple_retrain'].mean()
    acc_hdbscan = production_df.groupby('scenario')['hdbscan'].mean()
    acc_uncertainty = production_df.groupby('scenario')['uncertainty'].mean()

    ax4.bar(x_pos - 1.5*width, acc_original, width, label='Original', color=colors[0], alpha=0.7)
    ax4.bar(x_pos - 0.5*width, acc_simple, width, label='Simple Retrain', color=colors[1], alpha=0.7)
    ax4.bar(x_pos + 0.5*width, acc_hdbscan, width, label='HDBSCAN', color=colors[2], alpha=0.7)
    ax4.bar(x_pos + 1.5*width, acc_uncertainty, width, label='Uncertainty', color=colors[3], alpha=0.7)
    
    ax4.set_title('Performance by Drift Type', fontweight='bold')
    ax4.set_ylabel('Average Test Accuracy')
    ax4.set_xticks(x_pos)
    ax4.set_xticklabels([s.replace('_', ' ').title() for s in scenarios])
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)

    plt.suptitle('Leak-Free MLOps Retraining Strategy Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save visualization
    production_viz_path = 'leak_free_drift_management.png'
    plt.savefig(production_viz_path, dpi=300, bbox_inches='tight')
    plt.close()
    
    # Log the final visualization
    with mlflow.start_run(run_name='final_comparison_visualization'):
        mlflow.log_artifact(production_viz_path, artifact_path='comparison_plots')

    return production_viz_path

# --- 6. RUN THE EXPERIMENT ---
# You can now run this single function to get a valid result.
# results = run_leak_free_comparison_pipeline(X, y, pipeline)
# create_leak_free_visualization(results)

In [46]:
results = comparison_pipeline(X,y, pipeline)
visualization(results)


RUNNING COMPARISON PIPELINE
  Identifying baseline error clusters with HDBSCAN...
  HDBSCAN found 3 error clusters.

Testing: covariate_only drift at 0.25
Simulating covariate drift with threshold: 0.25
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
  Training: Simple Retrain...
  Training: HDBSCAN Enhanced...
    -> Found 4199 risky samples for HDBSCAN weighting.
  Training: Uncertainty Enhanced...
    -> Getting uncertainty weights...
    -> Found 1409 uncertain samples for weighting.
    Original Acc:     0.7913
    Simple Retrain Acc: 0.7935
    HDBSCAN Acc:      0.7892
    Uncertainty Acc:  0.7899
🏃 View run PipeComparison_covariate_only_0.25 at: http://localhost:5000/#/experiments/13/runs/dba98e442d824b048409d71ef6b4a15f
🧪 View experiment at: http://localhost:5000/#/experiments/13

Testing: covariate_only drift at 0.50
Simulating covariate drift with threshold: 0.50
Applying to 8 nu

'leak_free_drift_management.png'

In [47]:
print("FIXING DATA LEAKAGE IN HDBSCAN IMPLEMENTATION")
print("Ensuring proper train/test separation")

def hdbscan_enhanced_ddla_system_fixed(X_baseline, y_baseline, X_drift_train, y_drift_train, X_drift_test, y_drift_test, trained_pipeline):
    """
    CORRECTED: Proper train/test separation to prevent data leakage.
    """
    print("HDBSCAN Enhanced System - Data Leakage Fixed")
    
    # Get model errors for clustering FROM BASELINE DATA (not drift data)
    X_train_base, X_test_base, y_train_base, y_test_base = train_test_split(
        X_baseline, y_baseline, test_size=0.2, random_state=42
    )
    
    # Identify error patterns in baseline test data
    test_predictions = trained_pipeline.predict(X_test_base)
    error_mask = test_predictions != y_test_base
    
    if error_mask.sum() == 0:
        print("  No errors in baseline - using simple retrain")
        from sklearn.base import clone
        simple_retrain = clone(trained_pipeline)
        simple_retrain.fit(X_drift_train, y_drift_train)  # Train on drift training data
        test_accuracy = accuracy_score(y_drift_test, simple_retrain.predict(X_drift_test))  # Test on drift test data
        
        return {
            'final_accuracy': test_accuracy,
            'enhancement_applied': False,
            'method': 'simple_retrain_fallback'
        }
    
    # Get error samples for clustering
    error_samples = X_test_base[error_mask]
    X_test_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_test_base)
    error_samples_preprocessed = X_test_preprocessed[error_mask]
    
    # Apply HDBSCAN to error patterns
    try:
        import hdbscan
        from hdbscan import approximate_predict
        
        hdbscan_clusterer = hdbscan.HDBSCAN(
            min_cluster_size=max(3, len(error_samples_preprocessed)//10),
            min_samples=2,
            prediction_data=True
        )
        
        error_clusters = hdbscan_clusterer.fit_predict(error_samples_preprocessed)
        
        if len(set(error_clusters)) <= 1:
            print("  No meaningful clusters - using simple retrain")
            from sklearn.base import clone
            simple_retrain = clone(trained_pipeline)
            simple_retrain.fit(X_drift_train, y_drift_train)
            test_accuracy = accuracy_score(y_drift_test, simple_retrain.predict(X_drift_test))
            
            return {
                'final_accuracy': test_accuracy,
                'enhancement_applied': False,
                'method': 'no_clusters_found'
            }
        
        # Apply clustering to drift TRAINING data (not test data!)
        X_drift_train_preprocessed = trained_pipeline.named_steps['preprocessor'].transform(X_drift_train)
        
        # Use approximate_predict to find drift training samples in error clusters
        drift_cluster_assignments, _ = approximate_predict(hdbscan_clusterer, X_drift_train_preprocessed)
        
        # Find samples that match error cluster patterns
        error_cluster_ids = set(error_clusters[error_clusters != -1])
        risky_mask = np.array([cluster_id in error_cluster_ids for cluster_id in drift_cluster_assignments])
        
        if risky_mask.sum() > 0 and risky_mask.sum() < len(X_drift_train) * 0.5:
            # Create sample weights (literature-recommended approach)
            sample_weights = np.ones(len(X_drift_train))
            sample_weights[risky_mask] = 2.0  # Weight risky samples 2x
            
            print(f"  Applying 2x weighting to {risky_mask.sum()} risky samples")
            
            # Train with sample weights
            from sklearn.base import clone
            weighted_pipeline = clone(trained_pipeline)
            
            # CRITICAL FIX: Train on X_drift_train, test on X_drift_test
            try:
                weighted_pipeline.fit(X_drift_train, y_drift_train, 
                                    classifier__sample_weight=sample_weights)
                enhancement_applied = True
                method_used = 'sample_weighted'
            except:
                # Fallback if sample weighting not supported
                weighted_pipeline.fit(X_drift_train, y_drift_train)
                enhancement_applied = False
                method_used = 'simple_retrain_fallback'
            
            # CORRECTED: Test on UNSEEN drift test data
            test_accuracy = accuracy_score(y_drift_test, weighted_pipeline.predict(X_drift_test))
            
            return {
                'final_accuracy': test_accuracy,  # This is now TRUE test accuracy
                'enhancement_applied': enhancement_applied,
                'risky_samples': risky_mask.sum(),
                'method': method_used
            }
        
        else:
            # Too many or too few risky samples - simple retrain
            print(f"  Risky samples {risky_mask.sum()} not in viable range - simple retrain")
            from sklearn.base import clone
            simple_retrain = clone(trained_pipeline)
            simple_retrain.fit(X_drift_train, y_drift_train)
            test_accuracy = accuracy_score(y_drift_test, simple_retrain.predict(X_drift_test))
            
            return {
                'final_accuracy': test_accuracy,
                'enhancement_applied': False,
                'method': 'simple_retrain'
            }
            
    except Exception as e:
        print(f"  HDBSCAN failed: {str(e)[:50]}... - using simple retrain")
        from sklearn.base import clone
        simple_retrain = clone(trained_pipeline)
        simple_retrain.fit(X_drift_train, y_drift_train)
        test_accuracy = accuracy_score(y_drift_test, simple_retrain.predict(X_drift_test))
        
        return {
            'final_accuracy': test_accuracy,
            'enhancement_applied': False,
            'method': 'hdbscan_failed'
        }

# CORRECTED PRODUCTION PIPELINE WITH PROPER TRAIN/TEST SEPARATION
def corrected_production_pipeline():
    """
    Production pipeline with data leakage eliminated.
    """
    print("CORRECTED PRODUCTION PIPELINE - NO DATA LEAKAGE")
    
    corrected_results = []
    
    scenarios = [
        ('covariate_only', simulate_covariate_drift_only),
        ('concept_only', simulate_concept_drift_only),
        ('combined_drift', simulate_drifted_data)
    ]
    
    for scenario_name, drift_func in scenarios:
        for threshold in [0.25, 0.5, 0.75, 1.0]:
            print(f"\nTesting {scenario_name} at threshold {threshold:.2f}")
            
            # Generate drift data
            X_drifted, y_drifted, _ = drift_func(X, y, drift_threshold=threshold, random_state=42)
            
            # CRITICAL: Proper train/test split
            X_drift_train, X_drift_test, y_drift_train, y_drift_test = train_test_split(
                X_drifted, y_drifted, test_size=0.2, random_state=42
            )
            
            with mlflow.start_run(run_name=f'hdbscan_{scenario_name}_{threshold}'):
                
                # Original pipeline on UNSEEN drift test data
                original_accuracy = accuracy_score(y_drift_test, pipeline.predict(X_drift_test))
                
                # Simple retrain: train on drift training, test on drift test
                from sklearn.base import clone
                simple_pipeline = clone(pipeline)
                simple_pipeline.fit(X_drift_train, y_drift_train)  # Train on training data
                simple_accuracy = accuracy_score(y_drift_test, simple_pipeline.predict(X_drift_test))  # Test on TEST data
                
                # HDBSCAN enhanced with proper separation
                hdbscan_result = hdbscan_enhanced_ddla_system_fixed(
                    X, y,  # Original baseline data
                    X_drift_train, y_drift_train,  # Drift training data
                    X_drift_test, y_drift_test,    # Drift test data (unseen!)
                    pipeline
                )
                hdbscan_accuracy = hdbscan_result['final_accuracy']
                
                # Calculate TRUE improvements
                simple_improvement = simple_accuracy - original_accuracy
                hdbscan_improvement = hdbscan_accuracy - original_accuracy
                hdbscan_vs_simple = hdbscan_accuracy - simple_accuracy
                
                print(f"  CORRECTED RESULTS (no leakage):")
                print(f"    Original on drift test: {original_accuracy:.3f}")
                print(f"    Simple retrain:         {simple_accuracy:.3f} ({simple_improvement:+.3f})")
                print(f"    HDBSCAN enhanced:       {hdbscan_accuracy:.3f} ({hdbscan_improvement:+.3f})")
                print(f"    HDBSCAN vs Simple:      {hdbscan_vs_simple:+.3f}")
                
                # Log corrected metrics
                mlflow.log_param('data_leakage_fixed', True)
                mlflow.log_param('proper_train_test_split', True)
                mlflow.log_param('scenario', scenario_name)
                mlflow.log_param('threshold', threshold)
                
                mlflow.log_metric('original_test_accuracy', original_accuracy)
                mlflow.log_metric('simple_retrain_test_accuracy', simple_accuracy)
                mlflow.log_metric('hdbscan_enhanced_test_accuracy', hdbscan_accuracy)
                mlflow.log_metric('simple_improvement_over_original', simple_improvement)
                mlflow.log_metric('hdbscan_improvement_over_original', hdbscan_improvement)
                mlflow.log_metric('hdbscan_improvement_over_simple', hdbscan_vs_simple)
                
                corrected_results.append({
                    'scenario': scenario_name,
                    'threshold': threshold,
                    'original': original_accuracy,
                    'simple': simple_accuracy,
                    'hdbscan': hdbscan_accuracy,
                    'simple_improvement': simple_improvement,
                    'hdbscan_improvement': hdbscan_improvement,
                    'hdbscan_vs_simple': hdbscan_vs_simple
                })
    
    return corrected_results

# Run corrected pipeline
print("Running corrected pipeline with proper train/test separation...")
corrected_results = corrected_production_pipeline()

# Analyze corrected results
corrected_df = pd.DataFrame(corrected_results)

print(f"\nCORRECTED RESULTS ANALYSIS:")
print(f"Average HDBSCAN test accuracy: {corrected_df['hdbscan'].mean():.3f}")
print(f"Average Simple retrain accuracy: {corrected_df['simple'].mean():.3f}")
print(f"Average HDBSCAN improvement over simple: {corrected_df['hdbscan_vs_simple'].mean():+.3f}")

# Check if HDBSCAN still dominates after fixing leakage
hdbscan_wins = sum(corrected_df['hdbscan_vs_simple'] > 0.005)  # Significant improvement threshold
total_tests = len(corrected_df)
true_win_rate = hdbscan_wins / total_tests

print(f"HDBSCAN significant wins: {hdbscan_wins}/{total_tests} ({true_win_rate:.1%})")

# Create corrected visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Corrected performance comparison
ax1 = axes[0]
methods = ['Original', 'Simple Retrain', 'HDBSCAN Enhanced']
avg_performance = [
    corrected_df['original'].mean(),
    corrected_df['simple'].mean(), 
    corrected_df['hdbscan'].mean()
]
colors = ['#e74c3c', '#2ecc71', '#e67e22']

bars = ax1.bar(methods, avg_performance, color=colors, alpha=0.8)
ax1.set_title('Corrected Performance Comparison (No Data Leakage)', fontweight='bold')
ax1.set_ylabel('Average Test Accuracy')
ax1.grid(axis='y', alpha=0.3)

for bar, value in zip(bars, avg_performance):
    ax1.text(bar.get_x() + bar.get_width()/2., value + 0.005,
            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# Plot 2: HDBSCAN vs Simple improvement distribution
ax2 = axes[1]
improvements = corrected_df['hdbscan_vs_simple'].values

ax2.hist(improvements, bins=8, color='#3498db', alpha=0.7, edgecolor='black')
ax2.axvline(0, color='red', linestyle='--', linewidth=2, label='No Improvement Line')
ax2.axvline(improvements.mean(), color='orange', linestyle='-', linewidth=2, 
           label=f'Mean: {improvements.mean():+.3f}')

ax2.set_title('HDBSCAN vs Simple Retrain Improvement Distribution', fontweight='bold')
ax2.set_xlabel('HDBSCAN Improvement over Simple Retrain')
ax2.set_ylabel('Frequency')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.suptitle('Data Leakage Corrected Analysis', fontsize=14, fontweight='bold')
plt.tight_layout()

# Save corrected visualization
corrected_viz_path = 'data_leakage_corrected_analysis.png'
plt.savefig(corrected_viz_path, dpi=300, bbox_inches='tight')
plt.close()

# Log corrected analysis
with mlflow.start_run(run_name='hdbscan_results'):
    mlflow.log_artifact(corrected_viz_path, artifact_path='corrected_analysis')
    
    mlflow.log_param('data_leakage_identified_and_fixed', True)
    mlflow.log_param('training_vs_test_accuracy_separated', True)
    mlflow.log_metric('corrected_hdbscan_win_rate', true_win_rate)
    mlflow.log_metric('corrected_avg_hdbscan_improvement', corrected_df['hdbscan_vs_simple'].mean())

FIXING DATA LEAKAGE IN HDBSCAN IMPLEMENTATION
Ensuring proper train/test separation
Running corrected pipeline with proper train/test separation...
CORRECTED PRODUCTION PIPELINE - NO DATA LEAKAGE

Testing covariate_only at threshold 0.25
Simulating covariate drift with threshold: 0.25
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
HDBSCAN Enhanced System - Data Leakage Fixed
  Risky samples 4548 not in viable range - simple retrain
  CORRECTED RESULTS (no leakage):
    Original on drift test: 0.791
    Simple retrain:         0.793 (+0.002)
    HDBSCAN enhanced:       0.793 (+0.002)
    HDBSCAN vs Simple:      +0.000
🏃 View run hdbscan_covariate_only_0.25 at: http://localhost:5000/#/experiments/13/runs/7b8e905605164ea2bfc682225c134795
🧪 View experiment at: http://localhost:5000/#/experiments/13

Testing covariate_only at threshold 0.50
Simulating covariate drift with threshold: 0.50
Applyi

## Adapting DDLAs as a monitoring-triggerer system

In [64]:
import joblib
import mlflow
import hdbscan
from hdbscan import approximate_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Set a main experiment for all this work
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mlops-drift-management-simulation")

# --- Function 1: Identify DT DDLAs (from your notebook) ---
# (Assuming identify_ddlas_decision_tree is already defined in your notebook)

# --- Function 2: Identify HDBSCAN DDLAs (Standardized) ---
def identify_hdbscan_ddlas_standardized(base_pipeline, X_val, y_val):
    """
    Identifies error-prone clusters from baseline data using HDBSCAN.
    Returns a standardized 'ddla_info' dictionary.
    """
    print("  Identifying baseline error clusters with HDBSCAN...")
    
    val_predictions = base_pipeline.predict(X_val)
    error_mask = (val_predictions != y_val)
    
    if error_mask.sum() < 10:
        print("  Not enough baseline errors to build HDBSCAN model.")
        return {'ddlas': [], 'clusterer': None, 'error_ids': set(), 'baseline_ddla_ratio': 0.0}

    X_val_preprocessed = base_pipeline.named_steps['preprocessor'].transform(X_val)
    error_samples_preprocessed = X_val_preprocessed[error_mask]
    
    hdbscan_clusterer = hdbscan.HDBSCAN(
        min_cluster_size=max(5, int(len(error_samples_preprocessed) * 0.05)),
        min_samples=3,
        prediction_data=True
    ).fit(error_samples_preprocessed)
    
    error_cluster_ids = set(hdbscan_clusterer.labels_[hdbscan_clusterer.labels_ != -1])
    print(f"  HDBSCAN found {len(error_cluster_ids)} error clusters.")
    
    # Predict on the *entire* validation set to get the baseline ratio
    all_cluster_assignments, _ = approximate_predict(hdbscan_clusterer, X_val_preprocessed)
    risky_mask = np.array([cid in error_cluster_ids for cid in all_cluster_assignments])
    baseline_ddla_ratio = risky_mask.sum() / len(X_val)
    
    print(f"  HDBSCAN baseline DDLA ratio: {baseline_ddla_ratio:.4f}")

    # Standardized Dictionary
    return {
        'ddlas': [{'cluster_id': id} for id in error_cluster_ids],
        'clusterer': hdbscan_clusterer, # Standardized key for the model
        'error_ids': error_cluster_ids,
        'baseline_ddla_ratio': baseline_ddla_ratio
    }

# --- 3. RUN THE ONE-TIME SETUP (FINAL - CORRECTED) ---

print("Starting One-Time Setup...")

# (Assuming 'pipeline', 'X', 'y', 'identify_ddlas_decision_tree', 
#  and 'identify_hdbscan_ddlas_standardized' are all defined)

prod_model = pipeline 
X_train_orig, X_val_orig, y_train_orig, y_val_orig = train_test_split(
    X, y, test_size=0.3, random_state=42
)

setup_run_id = "" 

with mlflow.start_run(run_name="Setup - Production Assets") as run:
    setup_run_id = run.info.run_id
    
    # --- 3a. Log and Register the "Production" Model ---
    print("  Registering production model...")
    mlflow.sklearn.log_model(
        prod_model,
        "model",
        registered_model_name="telco-baseline-production"
    )
    client = mlflow.MlflowClient()
    latest_version = client.get_latest_versions("telco-baseline-production", stages=["None"])[0]
    client.transition_model_version_stage(
        name="telco-baseline-production",
        version=latest_version.version,
        stage="Production",
        archive_existing_versions=True
    )
    mlflow.log_param("model_version", latest_version.version)

    # --- 3b. Create and Log DDLA Detector Assets ---
    
    # Decision Tree Detector
    print("  Creating DT DDLA detector...")
    dt_ddla_info = identify_ddlas_decision_tree(
        prod_model, X_val_orig, y_val_orig, random_state=42
    )
    
    # This dictionary is what we save. It MUST contain all keys
    # needed by the 'detect_harmful_drift_ddla' function.
    dt_assets = {
        'decision_tree': dt_ddla_info['decision_tree'],
        'feature_names': dt_ddla_info['feature_names'],
        'ddla_fraction_baseline': dt_ddla_info['ddla_fraction_baseline'],
        'ddlas': dt_ddla_info['ddlas']  # <-- THE MISSING KEY
    }
    joblib.dump(dt_assets, "dt_detector_assets.pkl")
    mlflow.log_artifact("dt_detector_assets.pkl")

    # HDBSCAN Detector
    print("  Creating HDBSCAN DDLA detector...")
    hdbscan_ddla_info = identify_hdbscan_ddlas_standardized(prod_model, X_val_orig, y_val_orig)
    
    # This dictionary is already correct and contains all needed keys.
    hdbscan_assets = {
        'clusterer': hdbscan_ddla_info['clusterer'],
        'error_ids': hdbscan_ddla_info['error_ids'],
        'baseline_ddla_ratio': hdbscan_ddla_info['baseline_ddla_ratio']
    }
    joblib.dump(hdbscan_assets, "hdbscan_detector_assets.pkl")
    mlflow.log_artifact("hdbscan_detector_assets.pkl")
    
    # Log the preprocessor
    preprocessor_path = "preprocessor.pkl"
    joblib.dump(prod_model.named_steps['preprocessor'], preprocessor_path)
    mlflow.log_artifact(preprocessor_path)
    
    mlflow.set_tag("pipeline.step", "setup")
    
print(f"Setup Complete. Assets saved in MLflow run: {setup_run_id}")

2025/11/04 02:18:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Starting One-Time Setup...
  Registering production model...


2025/11/04 02:18:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'telco-baseline-production' already exists. Creating a new version of this model...
2025/11/04 02:18:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: telco-baseline-production, version 5
Created version '5' of model 'telco-baseline-production'.


  Creating DT DDLA detector...
Identifying DDLAs with tree based approach
Overall model accuracy: 0.8192
  Overall incorrect prediction rate: 0.1808
  Best decision tree params: {'max_depth': 7, 'min_samples_leaf': 63}
  Decision tree F1 score: 0.4629
 Found 7 DDLAs out of 22 total leaf nodes
 DDLA coverage: 799/2113 samples (0.378)
  Creating HDBSCAN DDLA detector...
  Identifying baseline error clusters with HDBSCAN...
  HDBSCAN found 3 error clusters.
  HDBSCAN baseline DDLA ratio: 0.7657
🏃 View run Setup - Production Assets at: http://localhost:5000/#/experiments/14/runs/2bbb9c306e074e9e990a228c9cb700d8
🧪 View experiment at: http://localhost:5000/#/experiments/14
Setup Complete. Assets saved in MLflow run: 2bbb9c306e074e9e990a228c9cb700d8


In [61]:
def detect_harmful_drift_hdbscan(hdbscan_info, X_serving, base_pipeline, theta_inc=0.5, theta_ddla=0.1):
    """
    Detects harmful drift using a pre-fitted HDBSCAN clusterer.
    'hdbscan_info' is the dictionary loaded from the artifact.
    """
    # Use .get() for safety
    hdbscan_clusterer = hdbscan_info.get('clusterer')
    error_cluster_ids = hdbscan_info.get('error_ids')
    baseline_ddla_ratio = hdbscan_info.get('baseline_ddla_ratio')

    if hdbscan_clusterer is None or error_cluster_ids is None or baseline_ddla_ratio is None:
        print("  HDBSCAN detector assets are incomplete. Skipping detection.")
        return {'is_harmful_drift': False, 'reason': 'Incomplete HDBSCAN info.'}
    
    X_serving_preprocessed = base_pipeline.named_steps['preprocessor'].transform(X_serving)
    
    try:
        serving_cluster_assignments, _ = approximate_predict(hdbscan_clusterer, X_serving_preprocessed)
    except Exception as e:
        print(f"  HDBSCAN approximate_predict failed: {e}")
        return {'is_harmful_drift': False, 'reason': 'HDBSCAN prediction failed.'}
    
    risky_mask = np.array([cid in error_cluster_ids for cid in serving_cluster_assignments])
    serving_ddla_ratio = risky_mask.sum() / len(X_serving)

    print(f"    HDBSCAN Check: Baseline Ratio={baseline_ddla_ratio:.4f}, Serving Ratio={serving_ddla_ratio:.4f}")

    # Apply the standard drift detection logic
    if serving_ddla_ratio <= baseline_ddla_ratio:
        is_harmful = False
        reason = "HDBSCAN DDLA ratio decreased or stayed same"
    else:
        ratio_increase = (serving_ddla_ratio - baseline_ddla_ratio) / baseline_ddla_ratio if baseline_ddla_ratio > 0 else float('inf')
        is_harmful = (ratio_increase > theta_inc) and (serving_ddla_ratio > theta_ddla)
        reason = f"HDBSCAN DDLA ratio increased by {ratio_increase:.2%}"
        
    return {'is_harmful_drift': is_harmful, 'reason': reason}

In [ ]:
def run_automated_management_simulation(drift_type, threshold, setup_run_id):
    """
    Simulates the full, automated MLOps pipeline.
    This version loads the consolidated asset files.
    """
    mlflow.set_experiment("mlops-drift-management-simulation")
    with mlflow.start_run(run_name=f"Sim - {drift_type} - {threshold*100}% Drift") as run:
        
        print(f"\n--- Running Simulation: {drift_type} at {threshold*100}% ---")
        mlflow.log_param("drift_type", drift_type)
        mlflow.log_param("drift_threshold", threshold)
        mlflow.log_param("setup_run_id", setup_run_id)

        # --- 1. Load Production Assets (CORRECTED) ---
        print("  Loading production model and DDLA assets...")
        
        # In production, you'd use mlflow.artifacts.download_artifacts(run_id=setup_run_id, ...)
        # For this notebook, we load from the local files created by Experiment 1.
        prod_model = mlflow.sklearn.load_model("models:/telco-baseline-production/Production")
        dt_ddla_info = joblib.load('dt_detector_assets.pkl')      # <-- CLEAN
        hdbscan_ddla_info = joblib.load('hdbscan_detector_assets.pkl') # <-- CLEAN
        preprocessor = joblib.load('preprocessor.pkl')
        
        # --- 2. Simulate New Data Arriving ---
        print("  Simulating new data...")
        drift_func_map = {
            'covariate': simulate_covariate_drift_only,
            'concept': simulate_concept_drift_only,
            'combined': simulate_drifted_data
        }
        drift_func = drift_func_map[drift_type]
        X_drifted, y_drifted, _ = drift_func(X, y, drift_threshold=threshold, random_state=42)
        X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
            X_drifted, y_drifted, test_size=0.3, random_state=42
        )
        
        # --- 3. Phase 1: Run Monitoring ---
        print("  Phase 1: Running Smart Monitoring...")
        
        temp_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', prod_model.named_steps['classifier'])
        ])
        
        # Monitor the *entire* new batch of labeled data
        X_new_batch = pd.concat([X_train_new, X_test_new])
        
        # These functions now receive the complete 'ddla_info' dictionaries
        #dt_detection = detect_harmful_drift_ddla(dt_ddla_info, X_new_batch, temp_pipeline)
        #hdbscan_detection = detect_harmful_drift_hdbscan(hdbscan_ddla_info, X_new_batch, temp_pipeline)

        # --- FIX: Make the detectors more sensitive for the experiment ---
        # We're now telling them to fire if the DDLA ratio increases by just 10%
        # or exceeds an absolute ratio of 5%.
        
        sensitive_theta_inc = 0.1  # 10% increase (was 0.5)
        sensitive_theta_ddla = 0.05 # 5% absolute (was 0.1)

        print(f"  Running detectors with HIGH SENSITIVITY (theta_inc={sensitive_theta_inc})")

        dt_detection = detect_harmful_drift_ddla(
            dt_ddla_info, X_new_batch, temp_pipeline, 
            theta_inc=sensitive_theta_inc, theta_ddla=sensitive_theta_ddla
        )
        
        hdbscan_detection = detect_harmful_drift_hdbscan(
            hdbscan_ddla_info, X_new_batch, temp_pipeline, 
            theta_inc=sensitive_theta_inc, theta_ddla=sensitive_theta_ddla
        )
        
        # --- 4. Phase 2: Make Decision & Take Action ---
        print("  Phase 2: Making decision...")
        action_taken = 'monitor_only'

        if dt_detection['is_harmful_drift'] and not hdbscan_detection['is_harmful_drift']:
            print(f"    -> ALERT: Covariate Drift detected (Low Severity). Reason: {dt_detection['reason']}")
            mlflow.log_param("detected_drift_type", "covariate")
        elif hdbscan_detection['is_harmful_drift']:
            print(f"    -> CRITICAL: Concept/Mixed Drift detected! Triggering retrain. Reason: {hdbscan_detection['reason']}")
            action_taken = 'simple_retrain'
            mlflow.log_param("detected_drift_type", "concept_or_mixed")
        else:
            print("    -> OK: No significant drift detected.")
            mlflow.log_param("detected_drift_type", "none")
            
        mlflow.log_param("action_taken", action_taken)

        # --- 5. Phase 3: Evaluate & Log (LEAK-FREE) ---
        # (This section is unchanged and remains correct)
        print("  Phase 3: Evaluating performance...")
        
        acc_original = accuracy_score(y_test_new, prod_model.predict(X_test_new))
        
        if action_taken == 'simple_retrain':
            print("    Fitting 'Simple Retrain' model...")
            retrained_model = clone(prod_model)
            retrained_model.fit(X_train_new, y_train_new) 
        else:
            print("    No retrain triggered. Using original model.")
            retrained_model = prod_model 

        acc_final = accuracy_score(y_test_new, retrained_model.predict(X_test_new))
        
        improvement = acc_final - acc_original
        print(f"    Original Model Accuracy (on new test data): {acc_original:.4f}")
        print(f"    Final Model Accuracy (on new test data):    {acc_final:.4f}")
        print(f"    Improvement from Action:                  {improvement:+.4f}")
        
        mlflow.log_metric("accuracy_original_model", acc_original)
        mlflow.log_metric("accuracy_final_model", acc_final)
        mlflow.log_metric("improvement_from_action", improvement)
        mlflow.set_tag("pipeline.step", "simulation")

In [72]:
# --- 4. RUN THE SIMULATION ---
# After running Experiment 1, copy its Run ID here
SETUP_RUN_ID = "2bbb9c306e074e9e990a228c9cb700d8" 

print(f"Using Setup Assets from MLflow Run ID: {SETUP_RUN_ID}")

run_automated_management_simulation(
    drift_type='combined', 
    threshold=0.2, 
    setup_run_id=SETUP_RUN_ID
)

run_automated_management_simulation(
    drift_type='covariate', 
    threshold=0.2, 
    setup_run_id=SETUP_RUN_ID
)

run_automated_management_simulation(
    drift_type='concept', 
    threshold=0.2, 
    setup_run_id=SETUP_RUN_ID
)

Using Setup Assets from MLflow Run ID: 2bbb9c306e074e9e990a228c9cb700d8

--- Running Simulation: combined at 20.0% ---
  Loading production model and DDLA assets...


  Simulating new data...
Simulating combined drift with threshold: 0.20
Covariate weight: 1.00, Concept weight: 1.00
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.311 (original: 0.265)
  Phase 1: Running Smart Monitoring...
  Running detectors with HIGH SENSITIVITY (theta_inc=0.15)
Detecting harmful drift
  Baseline DDLA fraction: 0.3781
  Serving DDLA fraction: 0.4015
 Drift assessment: BENIGN
  Reason: DDLA ratio increase 6.19% below threshold 15.0%
    HDBSCAN Check: Baseline Ratio=0.7657, Serving Ratio=0.7466
  Phase 2: Making decision...
    -> OK: No significant drift detected.
  Phase 3: Evaluating performance...
    No retrain triggered. Using original model.
    Original Model Accuracy (on new test data): 0.7648
    Final Model Accuracy (on new test data):    0.7648
    Improvement from Action:                  +0.0000
🏃 View run Sim - combined - 20.0% Drift at: http://localhost:5000/#/experiments/14/

  Simulating new data...
Simulating covariate drift with threshold: 0.20
Applying to 8 numeric and 18 categorical features
Applied 15 covariate shifts
Applied 0 concept shifts
Final churn rate: 0.265 (original: 0.265)
  Phase 1: Running Smart Monitoring...
  Running detectors with HIGH SENSITIVITY (theta_inc=0.15)
Detecting harmful drift
  Baseline DDLA fraction: 0.3781
  Serving DDLA fraction: 0.4015
 Drift assessment: BENIGN
  Reason: DDLA ratio increase 6.19% below threshold 15.0%
    HDBSCAN Check: Baseline Ratio=0.7657, Serving Ratio=0.7466
  Phase 2: Making decision...
    -> OK: No significant drift detected.
  Phase 3: Evaluating performance...
    No retrain triggered. Using original model.
    Original Model Accuracy (on new test data): 0.8041
    Final Model Accuracy (on new test data):    0.8041
    Improvement from Action:                  +0.0000
🏃 View run Sim - covariate - 20.0% Drift at: http://localhost:5000/#/experiments/14/runs/52b425727ea2435f9de2d367a9fbfbc5
🧪 Vie

  Simulating new data...
Simulating concept drift with threshold: 0.20
Applying to 8 numeric and 18 categorical features
Applied 0 covariate shifts
Applied 5 concept shifts
Final churn rate: 0.310 (original: 0.265)
  Phase 1: Running Smart Monitoring...
  Running detectors with HIGH SENSITIVITY (theta_inc=0.15)
Detecting harmful drift
  Baseline DDLA fraction: 0.3781
  Serving DDLA fraction: 0.3673
 Drift assessment: BENIGN
  Reason: DDLA fraction decreased or stayed same
    HDBSCAN Check: Baseline Ratio=0.7657, Serving Ratio=0.7644
  Phase 2: Making decision...
    -> OK: No significant drift detected.
  Phase 3: Evaluating performance...
    No retrain triggered. Using original model.
    Original Model Accuracy (on new test data): 0.7799
    Final Model Accuracy (on new test data):    0.7799
    Improvement from Action:                  +0.0000
🏃 View run Sim - concept - 20.0% Drift at: http://localhost:5000/#/experiments/14/runs/11a0e99f3c0141d3b95b90148b578c43
🧪 View experiment a

In [ ]:
import numpy as np
import pandas as pd
import mlflow
import hdbscan
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings
import os # Added for cleanup

# --- 1. UTILITY FUNCTION ---

def get_preprocessor(X):
    """
    Creates a ColumnTransformer preprocessor based on column types in X.
    """
    # Identify categorical and numerical features
    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    numerical_features = X.select_dtypes(include=['int64', 'float64']).columns

    # Create transformers
    numeric_transformer = Pipeline(steps=[
        ('scaler', RobustScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Create the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough' # Keep any other columns
    )
    return preprocessor

# --- 2. NEW INITIAL TRAINING (SETUP) ---
# This version logs the training/test data as artifacts.

def run_initial_training_v2(preprocessed_data_path):
    """
    Trains the initial baseline model and logs all assets for the
    v2 (corrected) experiment.
    """
    print("--- Running Experiment Setup: (v2 - Corrected Logic) ---")
    
    data = pd.read_csv(preprocessed_data_path)
    X = data.drop('Churn', axis=1)
    y = data['Churn']

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    # Get the preprocessor
    preprocessor = get_preprocessor(X_train)

    # Define the model
    rf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)

    # Create the full pipeline
    model_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', rf)
    ])

    # Start MLflow run
    with mlflow.start_run(run_name="Experiment-v2-Setup") as run:
        print(f"MLflow Run ID: {run.info.run_id}")
        
        # Train the model
        model_pipeline.fit(X_train, y_train)

        # Log parameters
        mlflow.log_params(rf.get_params())

        # Make predictions
        y_pred = model_pipeline.predict(X_test)
        
        # Log metrics
        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, model_pipeline.predict_proba(X_test)[:, 1])
        print(f"Baseline Model Accuracy: {accuracy:.4f}")
        print(f"Baseline Model ROC AUC: {roc_auc:.4f}")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("roc_auc", roc_auc)

        # Log the model pipeline
        mlflow.sklearn.log_model(model_pipeline, "model_pipeline")

        # --- Save and log the training/test data as artifacts ---
        X_train.to_csv("X_train.csv", index=False)
        y_train.to_csv("y_train.csv", index=False)
        X_test.to_csv("X_test.csv", index=False)
        y_test.to_csv("y_test.csv", index=False)
        
        mlflow.log_artifact("X_train.csv")
        mlflow.log_artifact("y_train.csv")
        mlflow.log_artifact("X_test.csv")
        mlflow.log_artifact("y_test.csv")
        
        print("Logged model pipeline, parameters, metrics, and data artifacts.")
        
    # Clean up local files
    for f in ["X_train.csv", "y_train.csv", "X_test.csv", "y_test.csv"]:
        if os.path.exists(f):
            os.remove(f)
            
    return run.info.run_id


# --- 3. DRIFT SIMULATION FUNCTIONS ---

def create_covariate_drift_v2(data, severity=0.5):
    """
    Simulates covariate drift (P(X) changes) by shifting distributions.
    - Makes new customers have lower 'tenure'.
    - Makes 'Month-to-month' customers have higher 'MonthlyCharges'.
    """
    print(f"Simulating realistic covariate drift (v2)...")
    drifted_data = data.copy()
    
    # Shift 'tenure': decrease tenure for a random 50% of users
    tenure_mask = np.random.rand(len(drifted_data)) < severity
    drifted_data.loc[tenure_mask, 'tenure'] = (drifted_data.loc[tenure_mask, 'tenure'] * (1 - severity)).astype(int)
    # Ensure tenure is at least 1
    drifted_data.loc[drifted_data['tenure'] < 1, 'tenure'] = 1
    
    # Shift 'MonthlyCharges' for a specific subgroup
    charge_mask = (drifted_data['Contract'] == 'Month-to-month')
    drifted_data.loc[charge_mask, 'MonthlyCharges'] = drifted_data.loc[charge_mask, 'MonthlyCharges'] * (1 + severity)
    
    return drifted_data

def create_concept_drift(data, severity=0.5):
    """
    Simulates concept drift (P(Y|X) changes) by flipping labels for a specific subgroup.
    """
    print(f"Simulating concept drift...")
    drifted_data = data.copy()
    
    # Define the subgroup for concept drift
    mask = (drifted_data['Contract'] == 'Month-to-month') & (drifted_data['Dependents'] == 'No')
    subgroup_indices = drifted_data[mask].index
    
    # Randomly select a portion of this subgroup to flip
    num_to_flip = int(len(subgroup_indices) * severity)
    if num_to_flip > 0:
        indices_to_flip = np.random.choice(subgroup_indices, num_to_flip, replace=False)
        drifted_data.loc[indices_to_flip, 'Churn'] = 1 - drifted_data.loc[indices_to_flip, 'Churn']
        
    return drifted_data

def create_combined_drift(data, severity=0.5):
    """Applies both covariate and concept drift."""
    print(f"Simulating combined drift...")
    data = create_covariate_drift_v2(data, severity)
    data = create_concept_drift(data, severity)
    return data


# --- 4. NEW AUTOMATED MANAGEMENT SIMULATION ---

def run_automated_management_simulation_v2(drift_type, threshold, setup_run_id):
    """
    Runs the full drift detection and adaptation simulation (v2).
    
    - Loads baseline model and data from 'setup_run_id'.
    - *** FIX: Numerically encodes the y variable after loading ***
    - Simulates drift on the test data.
    - Runs DDLA (fixed) and HDBSCAN detection.
    - If triggered, retrains model on (Original Train + New Drifted Batch).
    - Logs all results to MLflow.
    """
    print(f"\n--- Running Simulation (v2) for: {drift_type} drift ---")
    
    client = mlflow.tracking.MlflowClient()
    
    # --- 1. SETUP: Load Baseline Assets ---
    print(f"Loading assets from setup run: {setup_run_id}")
    
    model_pipeline_uri = f"runs:/{setup_run_id}/model_pipeline"
    baseline_model = mlflow.sklearn.load_model(model_pipeline_uri)
    
    local_download_path = "."
    try:
        # Download artifacts to the current directory
        client.download_artifacts(setup_run_id, "X_train.csv", local_download_path)
        client.download_artifacts(setup_run_id, "y_train.csv", local_download_path)
        client.download_artifacts(setup_run_id, "X_test.csv", local_download_path)
        client.download_artifacts(setup_run_id, "y_test.csv", local_download_path)
        
        # Load the downloaded files
        X_train_original = pd.read_csv(os.path.join(local_download_path, "X_train.csv"))
        y_train_original = pd.read_csv(os.path.join(local_download_path, "y_train.csv")).squeeze()
        X_test_original = pd.read_csv(os.path.join(local_download_path, "X_test.csv"))
        y_test_original = pd.read_csv(os.path.join(local_download_path, "y_test.csv")).squeeze()
        
        # -----------------------------------------------------------------
        # --- HERE IS THE FIX ---
        # We must encode the y variables *after* loading them from CSV,
        # in case they were saved as strings ("Yes"/"No").
        
        le = LabelEncoder()
        
        # Fit on the combined y data to ensure all labels are known
        all_y = pd.concat([y_train_original, y_test_original])
        le.fit(all_y)

        # Transform both
        y_train_original = le.transform(y_train_original)
        y_test_original = le.transform(y_test_original)
        
        # le.classes_ will be something like ['No', 'Yes']
        # The positive class "Yes" will be at index 1, which is correct.
        print(f"Target labels encoded. Positive class ('{le.classes_[1]}') is now 1.")
        # --- END OF FIX ---
        # -----------------------------------------------------------------
        
        # Reconstruct the original test set for drift simulation
        # 'Churn' will now be 0 or 1
        test_data_original = X_test_original.copy()
        test_data_original['Churn'] = y_test_original
        
    except Exception as e:
        print(f"Error loading artifacts: {e}")
        return
    finally:
        # Clean up downloaded files
        for f in ["X_train.csv", "y_train.csv", "X_test.csv", "y_test.csv"]:
            p = os.path.join(local_download_path, f)
            if os.path.exists(p):
                os.remove(p)

    # --- 2. SIMULATE DRIFT ---
    
    if drift_type == 'covariate':
        drifted_test_data = simulate_covariate_drift_only(test_data_original)
    elif drift_type == 'concept':
        # This will now work because test_data_original['Churn'] is numeric
        drifted_test_data = simulate_concept_drift_only(test_data_original)
    elif drift_type == 'combined':
        drifted_test_data = simulate_drifted_data(test_data_original)
    else:
        drifted_test_data = test_data_original # No drift
        
    X_test_drifted = drifted_test_data.drop('Churn', axis=1)
    y_test_drifted = drifted_test_data['Churn'] # This is the new "ground truth" (0 or 1)

    # --- 3. MONITOR: Evaluate Baseline Model on New Data ---
    
    # Get "ground truth" performance of the old model on the new data
    y_pred_original_strings = baseline_model.predict(X_test_drifted) # Model predicts "Yes"/"No"
    
    # We must transform the model's string predictions back to 0/1 for metrics
    y_pred_original = le.transform(y_pred_original_strings) 
    
    accuracy_on_drifted = accuracy_score(y_test_drifted, y_pred_original)
    
    # This will now work because y_test_drifted is 0/1
    roc_auc_on_drifted = roc_auc_score(y_test_drifted, baseline_model.predict_proba(X_test_drifted)[:, 1])
    
    print(f"Baseline model accuracy on {drift_type} data: {accuracy_on_drifted:.4f}")

    # Start a new nested MLflow run for this simulation
    with mlflow.start_run(run_name=f"Sim-v2-{drift_type}-drift", nested=True) as run:
        mlflow.log_param("drift_type", drift_type)
        mlflow.log_param("detection_threshold", threshold)
        mlflow.log_param("setup_run_id", setup_run_id)
        mlflow.log_metric("baseline_accuracy_on_drifted", accuracy_on_drifted)
        mlflow.log_metric("baseline_roc_auc_on_drifted", roc_auc_on_drifted)
        
        retraining_triggered = False
        trigger_method = "None"

        # --- 4. DETECT: Run Detection Methods ---
        
        # --- Method 1: DDLA (Decision Tree) ---
        print("Running DDLA (error model) check...")
        
        # Create the error target: 1 if the model was wrong, 0 if right
        # This line is fine: y_test_drifted (0/1) != y_pred_original (0/1)
        y_errors = (y_test_drifted != y_pred_original).astype(int)
        
        if y_errors.nunique() > 1: # Ensure there are both errors and non-errors to train on
            preprocessor = baseline_model.named_steps['preprocessor']
            X_test_drifted_processed = preprocessor.transform(X_test_drifted)
            
            error_model = RandomForestClassifier(random_state=42, n_estimators=50, max_depth=5)
            error_model.fit(X_test_drifted_processed, y_errors)
            
            error_model_proba = error_model.predict_proba(X_test_drifted_processed)[:, 1]
            # This is fine: y_errors is 0/1
            error_model_auc = roc_auc_score(y_errors, error_model_proba)
            
            mlflow.log_metric("ddla_error_model_auc", error_model_auc)
            print(f"DDLA Error Model AUC: {error_model_auc:.4f}")
            
            if error_model_auc > (0.5 + threshold):
                retraining_triggered = True
                trigger_method = "DDLA"
                print(f"*** TRIGGERED by DDLA (AUC {error_model_auc:.4f} > {0.5 + threshold:.4f}) ***")
        else:
            print("DDLA check skipped: Model was 100% correct or 100% wrong.")
            mlflow.log_metric("ddla_error_model_auc", 0.5)


        # --- Method 2: HDBSCAN (Cluster Performance) ---
        if not retraining_triggered: 
            print("Running HDBSCAN (cluster) check...")
            try:
                if 'preprocessor' not in locals():
                    preprocessor = baseline_model.named_steps['preprocessor']
                    X_test_drifted_processed = preprocessor.transform(X_test_drifted)
                
                with warnings.catch_warnings(): 
                    warnings.simplefilter("ignore")
                    clusterer = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5, metric='euclidean', allow_single_cluster=True)
                    clusterer.fit(X_test_drifted_processed)
                
                labels = clusterer.labels_
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                print(f"HDBSCAN found {n_clusters} clusters.")
                mlflow.log_metric("hdbscan_num_clusters", n_clusters)
                
                for label in set(labels):
                    if label == -1: continue 
                        
                    cluster_mask = (labels == label)
                    
                    # This is fine: y_test_drifted and y_pred_original are both 0/1
                    cluster_accuracy = accuracy_score(y_test_drifted[cluster_mask], y_pred_original[cluster_mask])
                    
                    if cluster_accuracy < (accuracy_on_drifted - (threshold * 2)): 
                        retraining_triggered = True
                        trigger_method = "HDBSCAN"
                        print(f"*** TRIGGERED by HDBSCAN (Cluster {label} accuracy {cluster_accuracy:.4f} < {accuracy_on_drifted - (threshold*2):.4f}) ***")
                        mlflow.log_metric("hdbscan_trigger_cluster", label)
                        mlflow.log_metric("hdbscan_trigger_cluster_accuracy", cluster_accuracy)
                        break 
            except Exception as e:
                print(f"HDBSCAN clustering failed: {e}")

        mlflow.log_param("retraining_triggered", retraining_triggered)
        mlflow.log_param("trigger_method", trigger_method)

        # --- 5. ADAPT: Retrain Model if Triggered ---
        
        if retraining_triggered:
            print("Retraining triggered. Building new model...")
            
            # X data is fine
            X_retrain = pd.concat([X_train_original, X_test_drifted])
            # y data is now 0/1
            y_retrain = pd.concat([pd.Series(y_train_original), pd.Series(y_test_drifted)])
            
            print(f"New training set size: {len(y_retrain)} samples")
            
            new_preprocessor = get_preprocessor(X_retrain)
            new_rf = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=10)
            
            retrained_pipeline = Pipeline(steps=[
                ('preprocessor', new_preprocessor),
                ('classifier', new_rf)
            ])
            
            # Train the new model (will train on 0/1)
            retrained_pipeline.fit(X_retrain, y_retrain)
            
            print("Retraining complete. Evaluating new model...")
            
            # Evaluate the new model
            y_pred_retrained = retrained_pipeline.predict(X_test_drifted) # Predicts 0/1
            retrained_accuracy = accuracy_score(y_test_drifted, y_pred_retrained)
            retrained_roc_auc = roc_auc_score(y_test_drifted, retrained_pipeline.predict_proba(X_test_drifted)[:, 1])
            
            print(f"Retrained Model Accuracy: {retrained_accuracy:.4f} (Baseline was: {accuracy_on_drifted:.4f})")
            print(f"Retrained Model ROC AUC: {retrained_roc_auc:.4f} (Baseline was: {roc_auc_on_drifted:.4f})")
            
            mlflow.log_metric("retrained_model_accuracy", retrained_accuracy)
            mlflow.log_metric("retrained_model_roc_auc", retrained_roc_auc)
            mlflow.sklearn.log_model(retrained_pipeline, "retrained_model")

        else:
            print("No retraining triggered. Simulation complete.")

In [11]:
run_id = run_initial_training_v2(file_path)
print(run_id)

--- Running Experiment Setup: (v2 - Corrected Logic) ---
MLflow Run ID: 68cb1b590deb49c9ae83ceae0ea8e302


2025/11/04 16:54:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Baseline Model Accuracy: 0.7346
Baseline Model ROC AUC: 0.8135


2025/11/04 16:55:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Logged model pipeline, parameters, metrics, and data artifacts.
68cb1b590deb49c9ae83ceae0ea8e302


In [12]:
run_automated_management_simulation_v2('covariate', 0.5, 'ac7a0e80ec064f6fbe1b8a1b37ec4746')


--- Running Simulation (v2) for: covariate drift ---
Loading assets from setup run: ac7a0e80ec064f6fbe1b8a1b37ec4746


Target labels encoded. Positive class ('Yes') is now 1.
Simulating realistic covariate drift (v2)...
Baseline model accuracy on covariate data: 0.7346
Running DDLA (error model) check...
DDLA Error Model AUC: 0.8415
Running HDBSCAN (cluster) check...
HDBSCAN found 1 clusters.
No retraining triggered. Simulation complete.


In [13]:
run_automated_management_simulation_v2('concept', 0.5, 'ac7a0e80ec064f6fbe1b8a1b37ec4746')


--- Running Simulation (v2) for: concept drift ---
Loading assets from setup run: ac7a0e80ec064f6fbe1b8a1b37ec4746


Target labels encoded. Positive class ('Yes') is now 1.
Simulating concept drift...
Baseline model accuracy on concept data: 0.7140
Running DDLA (error model) check...
DDLA Error Model AUC: 0.7950
Running HDBSCAN (cluster) check...
HDBSCAN found 1 clusters.
No retraining triggered. Simulation complete.


In [14]:
run_automated_management_simulation_v2('combined', 0.5, 'ac7a0e80ec064f6fbe1b8a1b37ec4746')


--- Running Simulation (v2) for: combined drift ---
Loading assets from setup run: ac7a0e80ec064f6fbe1b8a1b37ec4746


Target labels encoded. Positive class ('Yes') is now 1.
Simulating combined drift...
Simulating realistic covariate drift (v2)...
Simulating concept drift...
Baseline model accuracy on combined data: 0.7126
Running DDLA (error model) check...
DDLA Error Model AUC: 0.8057
Running HDBSCAN (cluster) check...
HDBSCAN found 1 clusters.
No retraining triggered. Simulation complete.


In [21]:
run_automated_management_simulation_v2('covariate', 0.1, 'ac7a0e80ec064f6fbe1b8a1b37ec4746')


--- Running Simulation (v2) for: covariate drift ---
Loading assets from setup run: ac7a0e80ec064f6fbe1b8a1b37ec4746


Target labels encoded. Positive class ('Yes') is now 1.
Simulating realistic covariate drift (v2)...
Baseline model accuracy on covariate data: 0.7346
Running DDLA (error model) check...
DDLA Error Model AUC: 0.8415
*** TRIGGERED by DDLA (AUC 0.8415 > 0.6000) ***
Retraining triggered. Building new model...
New training set size: 7043 samples


2025/11/04 17:01:08 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Retraining complete. Evaluating new model...
Retrained Model Accuracy: 0.7346 (Baseline was: 0.7346)
Retrained Model ROC AUC: 0.8252 (Baseline was: 0.8113)


2025/11/04 17:01:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [22]:
run_automated_management_simulation_v2('concept', 0.1, 'ac7a0e80ec064f6fbe1b8a1b37ec4746')


--- Running Simulation (v2) for: concept drift ---
Loading assets from setup run: ac7a0e80ec064f6fbe1b8a1b37ec4746


Target labels encoded. Positive class ('Yes') is now 1.
Simulating concept drift...
Baseline model accuracy on concept data: 0.7012
Running DDLA (error model) check...
DDLA Error Model AUC: 0.8075
*** TRIGGERED by DDLA (AUC 0.8075 > 0.6000) ***
Retraining triggered. Building new model...
New training set size: 7043 samples


2025/11/04 17:01:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Retraining complete. Evaluating new model...
Retrained Model Accuracy: 0.7012 (Baseline was: 0.7012)
Retrained Model ROC AUC: 0.7668 (Baseline was: 0.7414)


2025/11/04 17:01:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [23]:
run_automated_management_simulation_v2('combined', 0.1, 'ac7a0e80ec064f6fbe1b8a1b37ec4746')


--- Running Simulation (v2) for: combined drift ---
Loading assets from setup run: ac7a0e80ec064f6fbe1b8a1b37ec4746


Target labels encoded. Positive class ('Yes') is now 1.
Simulating combined drift...
Simulating realistic covariate drift (v2)...
Simulating concept drift...
Baseline model accuracy on combined data: 0.7055
Running DDLA (error model) check...
DDLA Error Model AUC: 0.8051
*** TRIGGERED by DDLA (AUC 0.8051 > 0.6000) ***
Retraining triggered. Building new model...
New training set size: 7043 samples


2025/11/04 17:01:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Retraining complete. Evaluating new model...
Retrained Model Accuracy: 0.7055 (Baseline was: 0.7055)
Retrained Model ROC AUC: 0.7791 (Baseline was: 0.7487)


2025/11/04 17:01:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
